In [1]:
%pip install -qU google-generativeai langgraph tavily
%pip install -qU tavily-python
%pip install -qU ddgs

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import google.generativeai as genai
from tavily import TavilyClient
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv('GEMINI_API_KEY') 
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

d:\Videos - Cursos\TIME_LATAM\Alvaro\Videos_2255\MULTIAGENTES_LANGGRAPH\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\PC Studio 2\AppData\Local\Temp\ipykernel_21156\2543518455.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [3]:
client = TavilyClient(api_key=TAVILY_API_KEY)
result = client.search("Qué son multiagentes de Inteligencia Artificial?",
                       include_answer=True)

result["answer"]

'Los sistemas multiagente de Inteligencia Artificial son redes de agentes autónomos que colaboran para resolver problemas complejos. Cada agente tiene una función específica y trabaja juntos para lograr objetivos más grandes. Estos sistemas permiten la interacción y cooperación entre múltiples agentes para mejorar la eficiencia y resolución de problemas.'

In [4]:
ciudad = "Tulum"

query = f"""
Enumere los 5 principales restaurantes en {ciudad}, según evaluaciones recientes en TripAdvisor o sitios similares de turismo.
Para cada restaurante, indique:
- Tipo de cocina (ej.: regional, italiana, japonesa)
- Una breve descripción (máx. 2 líneas)
- Calificación promedio (si está disponible)
- Rango de precios

Responda únicamente con datos actualizados y relevantes para turistas.
"""

In [21]:
from ddgs import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        raise e

for link in search(query):
    print(link)

https://www.tripadvisor.com.mx/Restaurants-g150813-Tulum_Yucatan_Peninsula.html
https://www.tripadvisor.com/Restaurants-g150813-Tulum_Yucatan_Peninsula.html
https://www.poresto.com/turismo/gastronomia/2024/5/7/los-5-mejores-restaurantes-tulum-segun-trip-advisor.html
https://www.tripadvisor.es/Restaurants-g150813-Tulum_Yucatan_Peninsula.html
https://nox-agency.com/es/tulum/guides/mejores-restaurantes-tulum
https://www.tripadvisor.co/Restaurants-g150813-Tulum_Yucatan_Peninsula.html


In [6]:
%pip install -qU bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import requests
from bs4 import BeautifulSoup
from ddgs import DDGS
import re

ddg = DDGS()

def scrape_restaurantes_info(url):
    if not url:
        print("Error: URL vacía o no localizada.")
        return None

    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'Accept-Language': 'es-MX,es;q=0.9,en;q=0.8'
}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error al cargar la página {url}: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

search_results = search(query)
if search_results:
    url = search_results[1]
else:
    url = None

soup = scrape_restaurantes_info(url)

print(f"Website: {url}\n\n")

if soup:
    print(str(soup.body.prettify())[:50000])
else:
    print("No fue posible raspar el contenido de la página o la URL no fue encontrada.")

Website: https://www.poresto.com/turismo/gastronomia/2024/5/7/los-5-mejores-restaurantes-tulum-segun-trip-advisor.html


<body class="page-entry">
 <script data-blockingmode="auto" data-cbid="341b5ebd-5c78-43c2-86d0-acbf7725b02b" defer="" id="Cookiebot" src="https://consent.cookiebot.com/uc.js" type="text/javascript">
 </script>
 <section class="block_menu-desktop" itemscope="" itemtype="http://schema.org/SiteNavigationElement">
  <div class="wrap-menu">
   <div class="wrap-logo">
    <meta content="PorEsto!" itemprop="name"/>
    <a class="link" href="/" itemprop="url" title="PorEsto!">
     <img alt="PorEsto!" class="logo" loading="lazy" src="/images/logo185.png">
     </img>
    </a>
   </div>
   <div class="ico-close">
    <svg fill="none" height="23" viewbox="0 0 23 23" width="23" xmlns="http://www.w3.org/2000/svg">
     <path d="M1 22L22.2132 0.786798" stroke="#2F2F2F">
     </path>
     <path d="M1 1L22.2132 22.2132" stroke="#2F2F2F">
     </path>
    </svg>
   </div>
   <form a

In [23]:
restaurante_text_data = []

for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    if tag.name == 'h1' and 'restaurantes' in tag.get_text(" ", strip=True).lower():
        restaurante_text_data.append(f"Título de la Página: {tag.get_text(' ', strip=True)}")

    elif tag.name in ['h2', 'h3'] or (tag.name == 'p' and 'destaque' in tag.get('class', [])):
        restaurante_text_data.append(f"Nombre/Destaque: {tag.get_text(' ', strip=True)}")
        
    elif tag.name == 'p':
        
        if len(tag.get_text(" ", strip=True)) > 50:
            restaurante_text_data.append(f"Contenido: {tag.get_text(' ', strip=True)}")

final_restaurante_data = "\n".join(restaurante_text_data)

final_restaurante_data = re.sub(r'\s+', ' ', final_restaurante_data).strip()

print(f"Website: {url}\n\n")
print(final_restaurante_data)

Website: https://www.poresto.com/turismo/gastronomia/2024/5/7/los-5-mejores-restaurantes-tulum-segun-trip-advisor.html


Contenido: Hombre sobrevive tras quedar más de dos horas en el interior de un cenote en Motul Título de la Página: Los 5 mejores restaurantes en Tulum según Trip Advisor Nombre/Destaque: De acuerdo con Trip Advisor, existen 5 restaurantes en Tulum que destacan del resto por su calidad, servicio y precio Contenido: Trip Advisor suele ser el mejor amigo de los viajeros pues es una plataforma web que reúne miles de opiniones sobre diferentes negocios y servicios del sector turístico en distintas partes del mundo. Esta vez, se dio a conocer una lista de los 5 mejores restaurantes que existen en Tulum, Quintana Roo . Contenido: En este portal web podemos encontrar experiencias de viajeros reales, quienes califican los distintos hoteles, playas, restaurantes, bares, y otros negocios, basándose en la calidad, el servicio, el ambiente, la atención y el precio de los producto

In [24]:
import os
import re
from tavily import TavilyClient

api_key = os.environ.get("TAVILY_API_KEY")
if not api_key:
    raise ValueError("la clave TAVILY_API_KEY no fue encontrada.")

cliente_tavily = TavilyClient(api_key=api_key)

ciudad = "Tulum"
tavily_query = f"restaurantes en {ciudad} tripadvisor con mayor cantidad de reviews y rango de precios"


print("Iniciando la búsqueda agéntica por URLs de Tripadvisor con Tavily...")
tripadvisor_url = None
try:
    tavily_results = cliente_tavily.search(query=tavily_query, max_results=5)

    if tavily_results and tavily_results["results"]:
        print(f"Tavily encontró {len(tavily_results['results'])} resultados. Analizando...")

        for result in tavily_results["results"]:
            url = result["url"]
            
            if "tripadvisor.com" in url or "tripadvisor.com.mx" in url:
                tripadvisor_url = url
                break

        if not tripadvisor_url:
            print("Ninguna URL relevante de Tripadvisor fue encontrada en los primeros resultados.")
            
    else:
        print("Tavily no encontró resultados para la búsqueda agéntica.")

except Exception as e:
    print(f"Error en la búsqueda agéntica con Tavily: {e}. Verifica tu clave de API o tu conexión.")

if tripadvisor_url:
    clean_url = re.sub(r'-oa\d+-', '-', tripadvisor_url)
    tripadvisor_url = clean_url
    print(f"  ✅ URL encontrada limpia de paginación.")

print("-" * 50)
print(f"URL Final de Tripadvisor para el raspado: {tripadvisor_url if tripadvisor_url else 'NO ENCONTRADO'}")
print("-" * 50)

Iniciando la búsqueda agéntica por URLs de Tripadvisor con Tavily...
Tavily encontró 5 resultados. Analizando...
  ✅ URL encontrada limpia de paginación.
--------------------------------------------------
URL Final de Tripadvisor para el raspado: https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html
--------------------------------------------------


In [25]:
%pip install -qU selenium
%pip install -qU webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import WebDriverException, TimeoutException

def scrape_restaurantes_info(url):
    if not url:
        print("Error: URL vacía o no localizada para el raspado.")
        return None

    try:
        service = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(30)
        
    except Exception as e:
        print(f"Error al inicializar el driver de Selenium: {e}")
        return None

    try:
        print(f"Tratando cargar la página con Selenium: {url}")
        driver.get(url)
        
        driver.implicitly_wait(10)

        response_text = driver.page_source

    except TimeoutException:
        print(f"Error de límite de tiempo al cargar la página {url}.")
        return None
    except WebDriverException as e:
        print(f"Error al cargar la página {url} con Selenium: {e}. Puede ser debido a un bloqueo o a un problema de conexión.")
        return None
    finally:
        driver.quit()

    soup = BeautifulSoup(response_text, 'html.parser')
    return soup

soup_tripadvisor = None

if 'tripadvisor_url' in locals() and tripadvisor_url:
    print(f"\nTratando raspar la página identificada: {tripadvisor_url}")
    soup_tripadvisor = scrape_restaurantes_info(tripadvisor_url)

    if soup_tripadvisor:
        print("HTML de la página de Tripadvisor obtenido con éxito!")
        page_title_tag = soup_tripadvisor.find('title')
        if page_title_tag:
            print(f"Título de la página: {page_title_tag.get_text(strip=True)}")
        else:
            print("No fue posible encontrar el título de la página.")
    else:
        print("Falla al raspar la página de Tripadvisor. Verifica la URL o sí la página está bloqueando el acceso.")
else:
    print("No existe URL de Tripadvisor válida para el raspado (obtenido en el bloque 1).")

print("-" * 50)


Tratando raspar la página identificada: https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html
Tratando cargar la página con Selenium: https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html
HTML de la página de Tripadvisor obtenido con éxito!
Título de la página: tripadvisor.com
--------------------------------------------------


In [27]:
from bs4 import BeautifulSoup
import re # Importar regex para una extracción más limpia

# Este es el HTML completo que me proporcionaste - Como si estuviéramos enseñando al agente de búsqueda a visualizar la página
html_completo_ejemplos = """
<!DOCTYPE html><html lang="en-US"><head><link rel="icon" id="favicon" type="image/x-icon" href="https://static.tacdn.com/assets/s/6b5096f1.ico"/><link rel="icon" type="image/svg+xml" href="https://static.tacdn.com/assets/s/3ba4aeda.svg"/><link rel="apple-touch-icon" sizes="180x180" href="https://static.tacdn.com/assets/s/12951248.png"/><link rel="mask-icon" sizes="any" color="#00210c" href="https://static.tacdn.com/assets/s/4f134e39.svg"/><meta name="theme-color" content="#00eb5b"/><meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"/><meta name="format-detection" content="telephone=no"/><meta property="al:ios:app_name" content="TripAdvisor"/><meta property="al:ios:app_store_id" content="284876795"/><meta property="twitter:app:id:ipad" name="twitter:app:id:ipad" content="284876795"/><meta property="twitter:app:id:iphone" name="twitter:app:id:iphone" content="284876795"/><meta property="al:ios:url" content="tripadvisor://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html?m=33762"/><meta property="twitter:app:url:ipad" name="twitter:app:url:ipad" content="tripadvisor://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html?m=33762"/><meta property="twitter:app:url:iphone" name="twitter:app:url:iphone" content="tripadvisor://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html?m=33762"/><meta name="apple-itunes-app" content="app-id=-1"/><script type="application/ld+json">{"@context":"https:\u002F\u002Fschema.org","@graph":[{"@type":"Organization","name":"Tripadvisor","url":"https:\u002F\u002Fwww.tripadvisor.com\u002F","logo":"https:\u002F\u002Fstatic.tacdn.com\u002Fimg2\u002Fbrand_refresh\u002FTripadvisor_logoset_solid_green.svg","sameAs":["https:\u002F\u002Fwww.facebook.com\u002FTripadvisor","https:\u002F\u002Ftwitter.com\u002FTripadvisor","https:\u002F\u002Finstagram.com\u002Ftripadvisor\u002F","https:\u002F\u002Fwww.linkedin.com\u002Fcompany\u002Ftripadvisor"]},{"@type":"WebSite","name":"Tripadvisor","url":"https:\u002F\u002Fwww.tripadvisor.com\u002F","potentialAction":{"@type":"SearchAction","target":{"@type":"EntryPoint","urlTemplate":"https:\u002F\u002Fwww.tripadvisor.com\u002FSearch?q={search_term_string}"},"query":"required name=search_term_string"}}]}</script><meta property="fb:pages" content="5863091683"/><meta property="fb:pages" content="329182043776593"/><meta name="twitter:widgets:csp" content="on"/><script nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279">window.performance&&performance.mark&&performance.getEntriesByType&&0===performance.getEntriesByType("visibility-state").length&&(performance.mark("visible"===document.visibilityState?"visible":"hidden"),document.addEventListener&&document.addEventListener("visibilitychange",function(){performance.mark("visible"===document.visibilityState?"visible":"hidden")},!1));</script><title>THE 10 BEST Moderately-Priced Restaurants in Tulum</title><meta name="viewport" content="width=device-width, initial-scale=1.0, viewport-fit=cover"/><meta name="description" content="Best Moderately Priced Restaurants in Tulum, Riviera Maya: Find Tripadvisor traveler reviews of THE BEST Tulum Mid Range Restaurants and search by price, location, and more."/><link rel="canonical" href="https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en" href="https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-GB" href="https://www.tripadvisor.co.uk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-CA" href="https://www.tripadvisor.ca/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr-CA" href="https://fr.tripadvisor.ca/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="it" href="https://www.tripadvisor.it/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es" href="https://www.tripadvisor.es/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="de" href="https://www.tripadvisor.de/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr" href="https://www.tripadvisor.fr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ja" href="https://www.tripadvisor.jp/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="zh-Hans" href="https://cn.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-IN" href="https://www.tripadvisor.in/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="sv" href="https://www.tripadvisor.se/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="nl" href="https://www.tripadvisor.nl/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="pt" href="https://www.tripadvisor.com.br/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="tr" href="https://www.tripadvisor.com.tr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="da" href="https://www.tripadvisor.dk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-MX" href="https://www.tripadvisor.com.mx/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-IE" href="https://www.tripadvisor.ie/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ar" href="https://ar.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ar-EG" href="https://www.tripadvisor.com.eg/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="de-AT" href="https://www.tripadvisor.at/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="el" href="https://www.tripadvisor.com.gr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-AU" href="https://www.tripadvisor.com.au/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-MY" href="https://www.tripadvisor.com.my/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-NZ" href="https://www.tripadvisor.co.nz/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-PH" href="https://www.tripadvisor.com.ph/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-SG" href="https://www.tripadvisor.com.sg/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-ZA" href="https://www.tripadvisor.co.za/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-AR" href="https://www.tripadvisor.com.ar/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-CL" href="https://www.tripadvisor.cl/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-CO" href="https://www.tripadvisor.co/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-PE" href="https://www.tripadvisor.com.pe/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-VE" href="https://www.tripadvisor.com.ve/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="id" href="https://www.tripadvisor.co.id/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="he" href="https://www.tripadvisor.co.il/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ko" href="https://www.tripadvisor.co.kr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="nb" href="https://no.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="pt-PT" href="https://www.tripadvisor.pt/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ru" href="https://www.tripadvisor.ru/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="th" href="https://th.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="vi" href="https://www.tripadvisor.com.vn/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="zh-Hant" href="https://www.tripadvisor.com.tw/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="de-CH" href="https://www.tripadvisor.ch/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr-CH" href="https://fr.tripadvisor.ch/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="it-CH" href="https://it.tripadvisor.ch/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-HK" href="https://en.tripadvisor.com.hk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr-BE" href="https://fr.tripadvisor.be/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="nl-BE" href="https://www.tripadvisor.be/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="zh-hk" href="https://www.tripadvisor.com.hk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><meta property="og:url" content="https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><meta property="og:site_name" content="Tripadvisor"/><meta property="og:type" content="website"/><meta property="og:locale" content="en-US"/><meta property="og:title" content="THE 10 BEST Moderately-Priced Restaurants in Tulum"/><meta property="og:description" content="Best Moderately Priced Restaurants in Tulum, Riviera Maya: Find Tripadvisor traveler reviews of THE BEST Tulum Mid Range Restaurants and search by price, location, and more."/><meta property="og:image" content="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=600&amp;h=600&amp;s=1"/><meta property="og:image:width" content="600"/><meta property="og:image:height" content="600"/><meta property="twitter:image" content="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=600&amp;h=600&amp;s=1"/><link rel="stylesheet" href="https://static.tacdn.com/assets/35pr0yuitw1c-c.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/p7jq7m.MNync6r.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/1tem5o.hHmbYjL.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/b5kt6r.n2F6kJa.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/633gr8.DQuSDwJ.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/6h4i0r.GoM2Ynr.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/us1vwl.EXnWuEi.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/70j2tf.BwhsA2m.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/eekt6e.u6AkCOt.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/z2nh2s.8hQa4LG.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/mf5oe4.a8E9pkW.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/76j3t0.GC8HV7Y.css" crossorigin=""/><script src="https://static.tacdn.com/assets/0ci59y4imqm7-c.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/37os3oto1kwr-c.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/im/f77fb48474eac182.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/gg6fk5.tkO6FtA.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/mqwhqn.ZtVGKN4.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/8cre3h.gxuh19I.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/p7jq7m.MNync6r.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/1tem5o.hHmbYjL.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/b5kt6r.n2F6kJa.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/w43m8k.bS66dJh.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/y9enps.BdLa8A8.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/n4zdla._ZcRb0A.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/633gr8.DQuSDwJ.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/6h4i0r.GoM2Ynr.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/us1vwl.EXnWuEi.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/5xqg72.tPBR6UO.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/70j2tf.BwhsA2m.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/xfufl0.-vjAwhJ.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/o80y56.qGwLjD9.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/ypf49l.F13Hn5S.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/ti102j.ns_prO6.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/d3i8k1.6Xxde1x.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/mxujx7.UvrWd0b.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/eekt6e.u6AkCOt.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/z2nh2s.8hQa4LG.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/mf5oe4.a8E9pkW.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/v42v3u.g2VkMHb.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script><script src="https://static.tacdn.com/assets/76j3t0.GC8HV7Y.js" async="" crossorigin="" fetchpriority="low" nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279"></script></head><body><div id="lithium-root"><!--$--><div id="ad-banner" aria-hidden="true" class="adsbygoogle ad-slot ad-banner adsbox ad" style="position:absolute;top:0px;bottom:-20px;width:10px;height:10px;background-color:transparent"><img src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACwAAAAAAQABAAACAkQBADs=" alt="Advertisement" style="display:block;width:1px;height:1px"/></div><!--$--><!--/$--><div class="CvDIt _Q f e Gi Ra _s P6"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Skip to main content</span></button></div><div class="ad SxZAK f _h u e j Gi"><div class="ktyWU f _h u e j w Gi"><div id="slot:5x1-728x90:horizon" class="WMKsc w xbgrb o"></div></div></div><span><div></div><header data-component="ta-brand-header" class="aVAoX t z"><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><nav class="MGToN u"><div class="oqeTs u JpEOb"><a class="f _Z wSSLS" href="/"><picture><img class="XpHHt _Z" src="https://static.tacdn.com/img2/brand_refresh_2025/logos/wordmark.svg" alt="Tripadvisor"/></picture></a></div><div class="OzBsv"><div class="HREWK t l s _U S ijEOu"></div><div class="gqnca"><div class="ZjkxF f e Wh z _f oPzxw" data-test-attribute="typeahead-SINGLE_SEARCH_NAV"><form role="search" action="/Search"><input type="hidden" name="searchSessionId" value="001c9e9d0c5b8096.ssid"/><input type="hidden" name="searchNearby" value="false"/><input type="hidden" name="geo" value="150813"/><div class="oKjvH f u Ma Nk MA NM"><div class="F1 f u Q2"><button type="submit" formaction="/Search" class="LhcRH _G _H B- G_ _S t u j H0" title="Search" aria-label="Search" tabindex="-1"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M10.12 4.14a5.99 5.99 0 1 0 0 11.98 5.99 5.99 0 0 0 0-11.98m-7.49 5.99a7.49 7.49 0 1 1 13.299 4.728L21.37 20.3l-1.06 1.061-5.44-5.44A7.49 7.49 0 0 1 2.63 10.13"></path></svg></button><div class="w"><div class="biGQs _P AWdfh"><input type="search" autoComplete="off" autoCorrect="off" autoCapitalize="none" spellcheck="false" required="" name="q" disabled="" class="hUpcN _G G_ B- z F1 w R0 OavzT _O" placeholder="Search" title="Search" role="searchbox" aria-label="Search" aria-autocomplete="list" value=""/></div></div></div></div></form></div></div></div><div class="inHiW JpEOb"><div class="eByLG QA c"><div class="AuQZR f"><div class="f u" data-automation="topNav_rewards"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" href="/Rewards"><span class="biGQs _P ezezH">Rewards</span></a></div><div class="f u" data-automation="topNav_discover"><div class="JLKop w"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH">Discover</span></button></div></div><div class="f u" data-automation="topNav_review"><div class="JLKop w"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH">Review</span></button></div></div></div></div></div><div class="lGhNq QA c JpEOb" data-automation="desktop-cart-and-profile"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH"><span class="q"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M9.31 9.82h4.178c-.069-1.591-.356-2.993-.766-4.017-.237-.593-.501-1.023-.756-1.293-.211-.223-.38-.3-.5-.32h-.133c-.12.02-.289.097-.5.32-.255.27-.519.7-.756 1.293-.41 1.024-.697 2.426-.767 4.017m-.374-5.14q-.135.272-.252.566C8.194 6.472 7.88 8.07 7.81 9.82H5.055a6.39 6.39 0 0 1 3.88-5.14m2.301-1.989a7.883 7.883 0 0 0-7.726 7.88 7.88 7.88 0 0 0 7.884 7.885c.584 0 .871-.014 1.11-.074.124-.031.172-.049.213-.064.058-.02.099-.036.312-.073l-.26-1.477a4 4 0 0 0-.628.159c-.031.007-.132.029-.743.029-.121 0-.313-.06-.566-.327-.255-.27-.519-.699-.756-1.292-.41-1.025-.697-2.426-.767-4.017h4.203a4.7 4.7 0 0 1-.113.843 6 6 0 0 1-.112.413l-.012.04-.004.016c-.034.114-.089.298-.1.478l1.498.088v.006s.007-.035.044-.161l.012-.04c.038-.126.09-.298.136-.505.072-.313.134-.698.152-1.178h2.734a4.2 4.2 0 0 1-.195.949c-.055.16-.11.28-.166.403l-.004.01-.01.02c-.046.101-.132.288-.161.486l1.484.219-.005.026s.012-.032.057-.13l.008-.018c.056-.123.137-.3.216-.529.168-.49.31-1.168.31-2.186a7.88 7.88 0 0 0-7.72-7.879 2 2 0 0 0-.325 0m2.626 1.99q.134.271.252.565c.49 1.226.805 2.824.875 4.574h2.75a6.38 6.38 0 0 0-3.877-5.14M8.94 16.466a8 8 0 0 1-.256-.573c-.49-1.227-.805-2.824-.875-4.574H5.055a6.39 6.39 0 0 0 3.885 5.147"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M20.489 12.343h-8.75v6.648h5.255l3.495 2.325zm-1.5 1.5v4.674l-1.542-1.026H13.24v-3.648z"></path></svg><div class="TzXjz d Vb"></div>USD</span></span></button><a class="rmyCe _G B- z _S c Wc wSSLS w jWkoZ sOtnj" href="/RegistrationController?flow=sign_up_and_save&amp;returnTo=%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html&amp;fullscreen=true&amp;flowOrigin=login&amp;hideNavigation=true&amp;isLithium=true"><span class="biGQs _P ezezH"><div class="eoYut">Sign in</div></span></a></div></nav></div><span><div class="o"><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="uNcsI _T o" data-test-target="global-nav"><div class="fSuCF w q czCLw"><div class="_T w"><div class="OykfZ f _e _h"><div class="KjnXN"><div role="tablist" data-automation="tab-list"><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_Tourism"><span class="biGQs _P OgHoE"><a class="JBbKZ Q1 nadlY" href="/ClientLink?value=a2k4Xy9Ub3VyaXNtLWcxNTA4MTMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEtVmFjYXRpb25zLmh0bWxfSFRU"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.256 7.032A2.441 2.441 0 1 0 9.746 8.9a2.441 2.441 0 0 0 4.51-1.868m-2.615.065a.941.941 0 1 1 .72 1.738.941.941 0 0 1-.72-1.738"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M7.26 5.882 2.748 6.91v14.1l6.418-1.951 5.64 1.714 6.446-1.47V5.708l-4.185.954a5.34 5.34 0 0 0-5.066-3.67 5.34 5.34 0 0 0-4.741 2.89M6.73 7.54a5 5 0 0 0-.053.745c0 1.265.506 2.476 1.163 3.536l-3.592.955v-4.67zm10.592.6q.003.073.002.145c0 1.573-.782 3.063-1.663 4.284l-.077.105v.353l4.168-.95v-4.49zm-3.288 6.368c-.363.386-.706.73-.996 1.019l-.057.057-.002.002-.987.98-.974-.998-.03-.03a44 44 0 0 1-1.072-1.093v3.273l4.168 1.266v-4.46zm.41-2.817c-.778 1.077-1.72 2.028-2.437 2.745-.72-.716-1.676-1.675-2.462-2.765-.814-1.128-1.368-2.294-1.368-3.386 0-2.082 1.717-3.795 3.824-3.795s3.823 1.713 3.823 3.795c0 1.098-.56 2.27-1.38 3.406m5.308 1.924-4.168.95v4.491l4.168-.95zm-15.504.713 4.168-1.108v4.498l-4.168 1.266z"></path></svg>Tulum</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_Hotels"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=YmZPXy9Ib3RlbHMtZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS1Ib3RlbHMuaHRtbF9wT3Q=">Hotels</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_ThingsToDo"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=MENKXy9BdHRyYWN0aW9ucy1nMTUwODEzLUFjdGl2aXRpZXMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEuaHRtbF9PUzY=">Things to Do</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS BmgDU" role="tab" type="button" aria-selected="true" data-automation="navHeader_Restaurants"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=cXFaXy9SZXN0YXVyYW50cy1nMTUwODEzLVR1bHVtX1l1Y2F0YW5fUGVuaW5zdWxhLmh0bWxfODBI">Restaurants</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_Cruises"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/Cruises">Cruises</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_ViewTravelForum"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=WTJmXy9TaG93Rm9ydW0tZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS5odG1sXzMzVA==">Forums</a></span></button></div></div></div><div class="ADnRg _Q t l _U s S Z1"><div class="oeBcu f u _Q t s Y yfScb"><div class="FEPIv"><button class="BrOJk u j z _F _S wSSLS tIqAi Vonfv" type="button" aria-hidden="true" tabindex="-1" aria-label="Previous"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="m16.6 5.65-6.4 6.4 6.4 6.3-1.4 1.4-7.8-7.7 7.8-7.8z"></path></svg></button></div></div><div class="oeBcu f u _Q t s Y MTngI"><div class="FEPIv"><button class="BrOJk u j z _F _S wSSLS tIqAi Vonfv" type="button" aria-hidden="true" tabindex="-1" aria-label="Next"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></button></div></div></div></div></div></div><div class="hNYnL _T"></div></div></div></span></header></span><main><!--$--><div data-test-target="restaurants-list"><div class="SIFnQ D Za" style="bottom:50px;right:100px"><!--$--><!--/$--></div><!--$--><!--/$--><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="f k O Cj Pj PN Pw PA"><div data-automation="breadcrumbs" class="Nm"><div class="buhZJ _T Cj"><a href="/Tourism-g150768-Mexico-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Mexico</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g150805-Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Yucatan Peninsula</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g1575485-Quintana_Roo_Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Quintana Roo</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g659488-Riviera_Maya_Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Riviera Maya</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Tulum</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Tulum Restaurants</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Tulum Mid Range Restaurants</span></span></div><script type="application/ld+json">{"@context":"https:\u002F\u002Fschema.org","@type":"BreadcrumbList","itemListElement":[{"@type":"ListItem","position":1,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g150768-Mexico-Vacations.html","name":"Mexico"},{"@type":"ListItem","position":2,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g150805-Yucatan_Peninsula-Vacations.html","name":"Yucatan Peninsula"},{"@type":"ListItem","position":3,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g1575485-Quintana_Roo_Yucatan_Peninsula-Vacations.html","name":"Quintana Roo"},{"@type":"ListItem","position":4,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g659488-Riviera_Maya_Yucatan_Peninsula-Vacations.html","name":"Riviera Maya"},{"@type":"ListItem","position":5,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html","name":"Tulum"},{"@type":"ListItem","position":6,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurants-g150813-Tulum_Yucatan_Peninsula.html","name":"Tulum Restaurants"},{"@type":"ListItem","position":7,"name":"Tulum Mid Range Restaurants"}]}</script></div><h1 data-automation="masthead_h1" class="Wi _m H0"><div class="biGQs _P VImYz avBIb navcl">Best Moderately Priced Restaurants in Tulum, Riviera Maya</div></h1></div></div><div></div><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="u f k mowmC"><div data-test-target="main_h1"><h2 class="biGQs _P SrOyz">Tulum Mid Range Restaurants</h2></div><div class="Mjzwz u f K"><span data-automation="mapButton"><button class="EAeYu z Xa q W _S Gy Rd Cr _M pGzPy wSSLS" type="button" aria-label="View map"><span class="sZLBj u k Q1"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg><span class="biGQs _P AWdfh">Map</span></span></button></span></div></div><div style="top:0px" class="pcCrB f j K Gx E t QPmrW u mowmC"><div class="biGQs _P SewaP OgHoE">Find a reservation</div><div class="NbxnN f v j"><div class="UvTmw f"><span class="xqGRb" data-automation="enterDate"><div class="jwRbK"><button type="button" class="hGFNH z B1 Z BF Gx _S q o W f u G H wSSLS rOYwG" aria-label="Enter the date." aria-haspopup="dialog"><span class="biGQs _P SewaP xENVe"><svg viewBox="0 0 24 24" width="18px" height="18px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M8.007 2.75a.75.75 0 0 1 .748.752l-.002.748h2.512V3.5a.75.75 0 0 1 1.5 0v.75h2.472l-.002-.748a.75.75 0 1 1 1.5-.004l.002.752h4.018v17H3.245v-17h4.008l.002-.752a.75.75 0 0 1 .752-.748m-.758 3H4.745v14h14.51v-14h-2.513l.006 1.747-1.5.005-.006-1.752h-2.477V7.5h-1.5V5.75H8.749l-.004 1.752-1.5-.004zm-.004 6h5.5v5.5h-5.5zm1.5 1.5v2.5h2.5v-2.5z"></path></svg> <!-- -->Today</span></button></div></span><span data-automation="enterTime"><div class="JLKop w"><div class="jwRbK"><button type="button" class="hGFNH z B1 Z BF Gx _S q o W f u G wSSLS w rOYwG" aria-label="Enter the time. The selected time is 7:00 PM." aria-haspopup="listbox"><span class="biGQs _P SewaP xENVe"><svg viewBox="0 0 24 24" width="18px" height="18px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg> <!-- -->7:00 PM</span></button></div></div></span><span data-automation="enterGuests"><div class="JLKop w"><div class="jwRbK"><button type="button" class="hGFNH z B1 Z BF Gx _S q o W f u G wSSLS w rOYwG" aria-label="Enter the number of guests. The selected number of guests is 2." aria-haspopup="listbox"><span class="biGQs _P SewaP xENVe"><svg viewBox="0 0 24 24" width="18px" height="18px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M8 4.75a1.5 1.5 0 1 0 0 3 1.5 1.5 0 0 0 0-3m-3 1.5a3 3 0 1 1 6 0 3 3 0 0 1-6 0m11 .5a1.5 1.5 0 1 0 0 3 1.5 1.5 0 0 0 0-3m-3 1.5a3 3 0 1 1 6 0 3 3 0 0 1-6 0m-5 2.5a6.91 6.91 0 0 1 5.36 2.493 6.96 6.96 0 0 1 8.244 2.312l.146.199v4.996H2.25v-6.997l.147-.199A6.91 6.91 0 0 1 8 10.75m3.75 8.5h8.5v-2.995a5.46 5.46 0 0 0-8.5 0zm-1.5 0h-6.5v-4.994a5.41 5.41 0 0 1 4.245-2.006h.01a5.4 5.4 0 0 1 4.006 1.73 7 7 0 0 0-1.615 1.575l-.146.199z"></path></svg> <span>2</span></span></button></div></div></span></div><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ sOtnj" data-automation="findARestaurant" type="button"><span class="biGQs _P ezezH">Search</span></button></div></div><div class="Mjzwz u f K tyUdl"><div class="u f"><button class="Datwj z W _S Gn B1 Z BF Rd _M j u Nfawr wSSLS XuzGl" type="button"><div class="SXRwk j u"><span class="biGQs _P FvqAl FGgoA AWdfh"><span data-automation="appliedBandFilter2">Mid-range</span></span><span class="NK"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M10.94 12 4.47 5.53l1.06-1.06L12 10.94l6.47-6.47 1.06 1.06L13.06 12l6.47 6.47-1.06 1.06L12 13.06l-6.47 6.47-1.06-1.06z"></path></svg></span></div></button></div><button class="xyffN _G B- _S _W u _T c G_ wSSLS eeawa" type="button" data-automation="clearAllfilters"><span class="biGQs _P ezezH">Clear all filters</span></button></div><div class="isRBk"><hr class="RMdFe tyUdl"/><div class="fYwEb izCpD"><div class="f u Q2"><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="m16.6 5.65-6.4 6.4 6.4 6.3-1.4 1.4-7.8-7.7 7.8-7.8z"></path></svg><span class="biGQs _P VImYz AWdfh">Tulum restaurants</span></a></div><div class="u f Q2"><div class="biGQs _P VImYz AWdfh" data-automation="resultsTotal">397<!-- --> results</div><div class="NB"><span><div class="f u"><div class="biGQs _P VImYz egaXP kSNRl ZNjnF">Sort:<!-- --> </div><div class="lOvTR"><button class="xyffN _G B- _S _W u _T c G_ wSSLS eeawa zMkft" type="button"><span class="biGQs _P ezezH">Relevance</span><div class="YQrIS _W PH"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div><div class="DgJpu"><button type="button" class="ypcsE _S wSSLS" aria-labelledby=":lithium-Rfjb8s9f9bvlq:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb rihVH UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-Rfjb8s9f9bvlq:" class="KrpDA o">Restaurants ranked by how well they match your selections and traveler reviews.</span></div></div></span></div></div></div><div class="seiBA _T wCxWg" data-automation="LeftRailContainer"><div class="wjLQL _T Ft" data-automation="LeftRail"><div class="NbqPV e iESum"><div class="WVPPt y S5 Gi f e"><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Establishment type</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R2pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R2pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.10591" aria-labelledby="establishmentTypes.10591_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.10591" value="10591"/></div><div class="hTbJa f H o"><label id="establishmentTypes.10591_label" for="establishmentTypes.10591" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Restaurants</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.11776" aria-labelledby="establishmentTypes.11776_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.11776" value="11776"/></div><div class="hTbJa f H o"><label id="establishmentTypes.11776_label" for="establishmentTypes.11776" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Bars &amp; Pubs</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.9900" aria-labelledby="establishmentTypes.9900_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.9900" value="9900"/></div><div class="hTbJa f H o"><label id="establishmentTypes.9900_label" for="establishmentTypes.9900" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Coffee &amp; Tea</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.9909" aria-labelledby="establishmentTypes.9909_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.9909" value="9909"/></div><div class="hTbJa f H o"><label id="establishmentTypes.9909_label" for="establishmentTypes.9909" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Dessert</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button" aria-expanded="false" aria-controls="filter-expando-establishmentTypes"><span class="biGQs _P ezezH">Show more<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Meal type</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R4pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R4pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10597" aria-labelledby="mealTypes.10597_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10597" value="10597"/></div><div class="hTbJa f H o"><label id="mealTypes.10597_label" for="mealTypes.10597" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Breakfast</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10606" aria-labelledby="mealTypes.10606_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10606" value="10606"/></div><div class="hTbJa f H o"><label id="mealTypes.10606_label" for="mealTypes.10606" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Brunch</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10598" aria-labelledby="mealTypes.10598_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10598" value="10598"/></div><div class="hTbJa f H o"><label id="mealTypes.10598_label" for="mealTypes.10598" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Lunch</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10599" aria-labelledby="mealTypes.10599_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10599" value="10599"/></div><div class="hTbJa f H o"><label id="mealTypes.10599_label" for="mealTypes.10599" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Dinner</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Cuisines</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R6pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R6pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.5110" aria-labelledby="cuisines.5110_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.5110" value="5110"/></div><div class="hTbJa f H o"><label id="cuisines.5110_label" for="cuisines.5110" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Mexican</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.10643" aria-labelledby="cuisines.10643_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.10643" value="10643"/></div><div class="hTbJa f H o"><label id="cuisines.10643_label" for="cuisines.10643" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Seafood</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.10640" aria-labelledby="cuisines.10640_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.10640" value="10640"/></div><div class="hTbJa f H o"><label id="cuisines.10640_label" for="cuisines.10640" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Bar</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.10679" aria-labelledby="cuisines.10679_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.10679" value="10679"/></div><div class="hTbJa f H o"><label id="cuisines.10679_label" for="cuisines.10679" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Healthy</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Show all</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Dishes</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R8pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R8pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.20515" aria-labelledby="dishes.20515_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.20515" value="20515"/></div><div class="hTbJa f H o"><label id="dishes.20515_label" for="dishes.20515" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Cocido</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.19954" aria-labelledby="dishes.19954_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.19954" value="19954"/></div><div class="hTbJa f H o"><label id="dishes.19954_label" for="dishes.19954" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Tacos</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.22444" aria-labelledby="dishes.22444_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.22444" value="22444"/></div><div class="hTbJa f H o"><label id="dishes.22444_label" for="dishes.22444" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Beef Tacos</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.10937" aria-labelledby="dishes.10937_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.10937" value="10937"/></div><div class="hTbJa f H o"><label id="dishes.10937_label" for="dishes.10937" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Shrimp</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Show all</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH"><div class="f">Awards<div class="TtrFj"><div class="biGQs _P ezezH"><span class="NK"><div class="DgJpu"><button type="button" class="ypcsE _S wSSLS" aria-labelledby=":lithium-R4bapdb8s9f9bvlq:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-R4bapdb8s9f9bvlq:" class="KrpDA o">Travelers’ Choice Awards winners (including the “Best of the Best” title) are among the top 10% of listings on Tripadvisor, according to the reviews and opinions of travelers across the globe.</span></div></span></div></div></div></h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Rapdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rapdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="travelersChoice.coe" aria-labelledby="travelersChoice.coe_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="travelersChoice.coe" value="coe"/></div><div class="hTbJa f H o"><label id="travelersChoice.coe_label" for="travelersChoice.coe" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="f Q1 u"><svg width="24" height="24" viewBox="0 0 24 24" fill="none" class="iezEl I" aria-hidden="true"><mask id=":lithium-R2ulapdb8s9f9bvlq:" fill="#fff"><path d="M0 12C0 5.373 5.373 0 12 0s12 5.373 12 12-5.373 12-12 12S0 18.627 0 12"></path></mask><path d="M0 12C0 5.373 5.373 0 12 0s12 5.373 12 12-5.373 12-12 12S0 18.627 0 12" fill="#00EB5B"></path><path d="M12 24v-1C5.925 23 1 18.075 1 12h-2c0 7.18 5.82 13 13 13zm12-12h-1c0 6.075-4.925 11-11 11v2c7.18 0 13-5.82 13-13zM12 0v1c6.075 0 11 4.925 11 11h2c0-7.18-5.82-13-13-13zm0 0v-1C4.82-1-1 4.82-1 12h2C1 5.925 5.925 1 12 1z" fill="#002B11" mask="url(#:lithium-R2ulapdb8s9f9bvlq:)"></path><path d="M12 5.294a6.202 6.202 0 1 0 .001 12.404 6.202 6.202 0 0 0 0-12.404m0 11.71a5.508 5.508 0 1 1 0-11.016 5.508 5.508 0 0 1 0 11.015m-1.38-5.088a.759.759 0 1 1-1.518 0 .759.759 0 0 1 1.518 0m4.277 0a.759.759 0 1 1-1.517 0 .759.759 0 0 1 1.517 0m.683-1.58.7-.762H14.73a4.8 4.8 0 0 0-2.73-.84 4.8 4.8 0 0 0-2.725.84H7.72l.7.761a2.14 2.14 0 1 0 2.896 3.15l.685.747.686-.746a2.14 2.14 0 1 0 2.895-3.15m-5.72 3.027a1.448 1.448 0 1 1 0-2.896 1.448 1.448 0 0 1 0 2.896M12 11.874c0-.952-.693-1.77-1.607-2.12a4.2 4.2 0 0 1 1.606-.32c.57 0 1.113.114 1.607.32-.913.35-1.606 1.168-1.606 2.12m2.139 1.49a1.448 1.448 0 1 1 0-2.897 1.448 1.448 0 0 1 0 2.896m-7.046 3.099s-.314-.788-1.163-1.585c-.85-.798-1.92-.943-1.92-.943s.381.85 1.285 1.699c.904.848 1.798.83 1.798.83m2.141 1.174s-.437-.425-1.233-.735-1.551-.16-1.551-.16.499.45 1.346.78 1.438.115 1.438.115m2.077.421s-.454-.187-1.13-.163a2.3 2.3 0 0 0-1.18.392s.507.185 1.227.16c.722-.023 1.083-.389 1.083-.389m-5.154-3.502s-.146-.84-.812-1.795a3.84 3.84 0 0 0-1.679-1.315s.198.915.907 1.932 1.584 1.178 1.584 1.178m-.837-2.559s.163-.851-.11-1.994A3.85 3.85 0 0 0 4.13 8.16s-.141.94.15 2.157c.292 1.217 1.04 1.679 1.04 1.679" fill="#002B11"></path><path d="M16.906 16.462s.314-.788 1.164-1.585c.849-.798 1.919-.943 1.919-.943s-.381.85-1.285 1.699c-.904.848-1.798.83-1.798.83m-2.139 1.174s.436-.425 1.232-.735 1.551-.16 1.551-.16-.498.45-1.346.78c-.847.33-1.437.115-1.437.115m-2.077.421s.453-.187 1.13-.163 1.18.392 1.18.392-.507.185-1.228.16c-.722-.023-1.083-.389-1.083-.389m5.154-3.502s.146-.84.812-1.795a3.84 3.84 0 0 1 1.679-1.315s-.198.915-.907 1.932-1.584 1.178-1.584 1.178m.837-2.559s-.163-.851.11-1.994c.274-1.143 1.078-1.842 1.078-1.842s.141.94-.15 2.157-1.039 1.679-1.039 1.679M12 18.706a.345.345 0 1 0 0-.689.345.345 0 0 0 0 .689" fill="#002B11"></path></svg>Travelers’ Choice</div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Price</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Rcpdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rcpdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="priceTypes.10953" aria-labelledby="priceTypes.10953_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="priceTypes.10953" value="10953"/></div><div class="hTbJa f H o"><label id="priceTypes.10953_label" for="priceTypes.10953" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Cheap Eats</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="priceTypes.10955" aria-labelledby="priceTypes.10955_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="true" name="priceTypes.10955" checked="" value="10955"/></div><div class="hTbJa f H o"><label id="priceTypes.10955_label" for="priceTypes.10955" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Mid-range</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="priceTypes.10954" aria-labelledby="priceTypes.10954_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="priceTypes.10954" value="10954"/></div><div class="hTbJa f H o"><label id="priceTypes.10954_label" for="priceTypes.10954" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Fine Dining</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Online options &amp; offers</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Repdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Repdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="onlineOptions.20693" aria-labelledby="onlineOptions.20693_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="onlineOptions.20693" value="20693"/></div><div class="hTbJa f H o"><label id="onlineOptions.20693_label" for="onlineOptions.20693" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Online Reservations </span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Traveler rating</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Rgpdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rgpdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="minimumTravelerRating.TRAVELER_RATING_LOW" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_LOW_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_LOW" value="TRAVELER_RATING_LOW"/></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_LOW_label" for="minimumTravelerRating.TRAVELER_RATING_LOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-R5qlgpdb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R5qlgpdb8s9f9bvlq:">3 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P VImYz biKBZ AWdfh">&amp; up</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="minimumTravelerRating.TRAVELER_RATING_MEDIUM" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_MEDIUM" value="TRAVELER_RATING_MEDIUM"/></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" for="minimumTravelerRating.TRAVELER_RATING_MEDIUM" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-R5slgpdb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R5slgpdb8s9f9bvlq:">4 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P VImYz biKBZ AWdfh">&amp; up</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="minimumTravelerRating.TRAVELER_RATING_HIGH" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_HIGH_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_HIGH" value="TRAVELER_RATING_HIGH"/></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_HIGH_label" for="minimumTravelerRating.TRAVELER_RATING_HIGH" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-R5ulgpdb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R5ulgpdb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg></div></div></div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Open now</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Ripdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Ripdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="openNow.OPEN_NOW" aria-labelledby="openNow.OPEN_NOW_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="openNow.OPEN_NOW" value="OPEN_NOW"/></div><div class="hTbJa f H o"><label id="openNow.OPEN_NOW_label" for="openNow.OPEN_NOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Open now</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Dietary restrictions</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Rkpdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rkpdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diets.10665" aria-labelledby="diets.10665_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diets.10665" value="10665"/></div><div class="hTbJa f H o"><label id="diets.10665_label" for="diets.10665" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Vegetarian friendly</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diets.10697" aria-labelledby="diets.10697_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diets.10697" value="10697"/></div><div class="hTbJa f H o"><label id="diets.10697_label" for="diets.10697" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Vegan options</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diets.10992" aria-labelledby="diets.10992_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diets.10992" value="10992"/></div><div class="hTbJa f H o"><label id="diets.10992_label" for="diets.10992" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Gluten free options</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">MICHELIN Guide</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Rmpdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rmpdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="michelinAwards.21964" aria-labelledby="michelinAwards.21964_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="michelinAwards.21964" data-automation="michelinGuideFilter" value="21964"/></div><div class="hTbJa f H o"><label id="michelinAwards.21964_label" for="michelinAwards.21964" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">MICHELIN Guide</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="michelinAwards.21969" aria-labelledby="michelinAwards.21969_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="michelinAwards.21969" value="21969"/></div><div class="hTbJa f H o"><label id="michelinAwards.21969_label" for="michelinAwards.21969" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">MICHELIN Bib Gourmand</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Great for</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Ropdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Ropdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.10604" aria-labelledby="styles.10604_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.10604" value="10604"/></div><div class="hTbJa f H o"><label id="styles.10604_label" for="styles.10604" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Families with children</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.10614" aria-labelledby="styles.10614_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.10614" value="10614"/></div><div class="hTbJa f H o"><label id="styles.10614_label" for="styles.10614" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Romantic</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.11777" aria-labelledby="styles.11777_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.11777" value="11777"/></div><div class="hTbJa f H o"><label id="styles.11777_label" for="styles.11777" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Kids</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.10607" aria-labelledby="styles.10607_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.10607" value="10607"/></div><div class="hTbJa f H o"><label id="styles.10607_label" for="styles.10607" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Special occasions</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button" aria-expanded="false" aria-controls="filter-expando-styles"><span class="biGQs _P ezezH">Show more<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Features</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Rqpdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rqpdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.10852" aria-labelledby="diningOptions.10852_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.10852" value="10852"/></div><div class="hTbJa f H o"><label id="diningOptions.10852_label" for="diningOptions.10852" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Seating</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.16547" aria-labelledby="diningOptions.16547_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.16547" value="16547"/></div><div class="hTbJa f H o"><label id="diningOptions.16547_label" for="diningOptions.16547" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Table Service</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.10602" aria-labelledby="diningOptions.10602_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.10602" data-automation="reservationsFilter" value="10602"/></div><div class="hTbJa f H o"><label id="diningOptions.10602_label" for="diningOptions.10602" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Reservations</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.10862" aria-labelledby="diningOptions.10862_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.10862" value="10862"/></div><div class="hTbJa f H o"><label id="diningOptions.10862_label" for="diningOptions.10862" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Serves Alcohol</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Show all</span></button></div></div></div></div></div></span></div></div><div class="oqGNy" data-automation="LeftRailSlot"><div class="uGSLz f e"><div class="IUxgK"><div style="top:80px" class="CsVUk E"><div class="YcOBw mFLzV"><div class="Eufjb j u ChFsW Xd o S" id="slot:300x600:rail1"></div></div></div></div><div class="IUxgK"><div style="top:80px" class="CsVUk E"><div class="YcOBw mFLzV"><div class="Eufjb j u ChFsW Xd o S" id="slot:300x600:rail3"></div></div></div></div></div></div></div><div class="LvzMf _T" data-automation="LeftRailMain"><div class="klkOK"><!--$--><!--/$--><div class="rfqBq f e"><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-24855625"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Alma Verde Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556g6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556g6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->348<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Cafe</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.almaverde.com.mx/copia-de-copia-de-menu-tulum" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We ordered the <b>avocado toast</b> and the eggs Benedict, both were absolutely deli...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amazing breakfast</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="365" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-25294307"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Xibak Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gaalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->88<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">International</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Opens in 4 min</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://beta.menuflow.com/900001450" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Nice Dinner</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Perfect Dining Experience in Aldea Zama!</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-2286855"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">1. La Zebra Beach Restaurant And Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gealb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->6,445<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>4.6<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://lazebratulum.com/dining-2/#menu" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Provides a variety of rooms, restaurants, and <b>bar</b> as well as beach area to en...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Edgar, <b>tacos</b>, and drinks… amazing!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline1" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23483492"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">2. Fit-a-licious</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gialb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->913<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, International</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl"><b>Arepas</b>, sough dough French toast, quesadillas and amazing Gluten free salmon...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Had the <b>guacamole</b> and smoked salmon rye...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-1144210"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">3. Mezzanine Thai Restaurant &amp; Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gmalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gmalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->4,783<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>2<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Asian, Thai</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://mezzaninetulum.com/menus" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">As was my tofu <b>green curry</b>.</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... the Tulum <b>salad</b>, tortillas, som tam...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-12091598"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">4. Restaurante Estrada</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gqalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,566<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Caribbean, Latin</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Fantastic <b>seafood</b> restaurant on the way back from the cenotes - would highly...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... day was superb as was the <b>guacamole</b>.</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27096664"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">5. Sexy Pizza Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gualb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->401<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bar, Pizza</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amazing!  Friendly and yummy!</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Marco was kind and Pizza delicious</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline2" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23597112"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Mamazul Tulum Mezcaleria</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.6</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556h2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556h2alb8s9f9bvlq:">4.6 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->52<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Best <b>tacos</b> ever</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Mezcal yes please</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27105003"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">6. Lula&#x27;s Kitchen</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556h6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556h6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->795<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, Mediterranean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://lulahoteltulum.com/eat-drink/" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">I highly recommend the shrimp <b>tacos</b>!</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>French toast</b>, bowls, and all of the...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23719230"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">7. Distrito Panamera</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556haalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556haalb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->175<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bar, Pizza</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>room</b> was very comfortable—the bed was nice and firm and the air condition...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Distrito Awesome!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-20296875"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">8. La Negra Tomasa</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556healb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556healb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,240<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Brew Pub</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://lanegratomasatulum.webnode.mx/" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The surf-and-turf tostada, octopus, and fried <b>tuna</b> tacos were incredibly tend...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Beautiful place for family dinner</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline3" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10066419"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">9. La Popular - Nômade Temple Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hialb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->411<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://cutt.ly/gbJo5nL" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excellent <b>seafood</b> by the sea</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>ceviche</b> was a highlight.</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-21408449"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">10. Tantra Tulum Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hmalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hmalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,678<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>3.7<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, International</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl"><b>Anniversary</b> at tantra tulum</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Tantra is a must visit!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10252068"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">11. Mi Amor Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hqalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->972<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">International, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.tulumhotelmiamor.com/restaurant-bar/#menu" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Paradise best <b>guacamole</b> in Tulum</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The best <b>cheesecake</b> dessert and amazing...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-24855625"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">12. Alma Verde Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hualb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->348<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Cafe</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.almaverde.com.mx/copia-de-copia-de-menu-tulum" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We ordered the <b>avocado toast</b> and the eggs Benedict, both were absolutely deli...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amazing breakfast</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline4" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-15294346"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">13. Casa Maria</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556i2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556i2alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,049<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://qr.imenupro.com/hod-32" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We had grilled <b>octopus</b> and baja style fish tacos.</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... and mixed <b>ceviche</b> Mexican style!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27477578"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">14. Piaggia Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556i6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556i6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->340<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Italian, Mediterranean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://toptablegroup.com/piaggia-tulum" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We had the açaí bowl and the <b>French toast</b>.</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Best pizza in Tulum</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10434099"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">15. Fresco&#x27;s</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iaalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,101<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>3.9<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Latin</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="http://www.frescostulum.com/menus" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">AMAZING!! Best breakfast in tulum! This place is so cute and the service is...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Healthy, fresh and DELICIOUS! Immaculate vibes, impeccable service, and SO SO SO FRESH!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23432753"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Parallel 20° by Kimpton Aluna Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iealb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->90<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">International, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">He even prepared something out of menu for me! Alex in the morning buffet is...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... I think said he was from <b>Mexico City</b>.</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23652674"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">16. Onyx Tulum Restaurant Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iialb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->728<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>ceviche</b> and mushroom tacos especially were great, and the Onyx Mescalita...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We got the <b>guacamole</b> for an app, steak...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10169568"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">17. El Suspiro</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556imalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556imalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->163<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Latin</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Alejandro was our Server with immediate attention and very considerable.!!! Our...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... pescado a <b>la parrilla</b> y el ceviche:...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-26265692"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">18. Golden Cham Thai</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iqalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->321<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, Asian</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed today</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We had shrimp steamed <b>dumplings</b> & Thai...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Fabulous in all ways!</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23180563"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">19. Ático Rooftop Bar &amp; Lounge</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iualb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->158<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The restaurant and rooftop staff were also fabulous and great to chat with...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Great Service</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline5" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-13201059"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">20. Loco Tulum Mediterranean Kitchen</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556j2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556j2alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->960<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>5.1<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, Mediterranean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We enjoyed the pita & <b>hummus</b>, tuna tataki, shrimp tacos, and Caesar salad—eve...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">I’m very speechless how was the service...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23780639"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">21. Rosanegra Beach Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556j6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556j6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->238<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>4.2<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Latin, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed today</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">A perfect day at the <b>tulum beach</b></span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">GREAT!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="374" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23459171"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">22. Cervecería Tulum Brewery Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jaalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,493<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The lager is delicious and the <b>tuna</b> tiradito is killer.</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Good dj, nice ambiance and decoration...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23524261"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">23. Bio-natural Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jealb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->305<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://bio-natural.mx/en/menu/" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vegan friendly! Thanks to Harry and Antonio great people and very friendly ☺️</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">5 star for Bio natural</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline6" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23280755"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">24. Confessions Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jialb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->997<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bars &amp; Pubs, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed today</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Hands down best night scene in tulum ask for them!! Confessions is an amazing...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Long and large <b>bar</b>.</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-19458762"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">25. Curry Thai Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jmalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jmalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->732<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Asian, Thai</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Great <b>Thai</b> food in Tulum and Fermin was a fantastic waiter, super attentive,...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We’ve stayed in tulum now for a few...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-26128325"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">26. Copal Gastro Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jqalb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->663<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://copalgastrobar.com/wp-content/uploads/2023/09/SPRECIO-COPAL-MENU-INGLES.pdf" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Fantastic fish <b>tacos</b></span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Must Visit Restaurant</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-1775007"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">27. Ziggy Beach Club &amp; Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jualb8s9f9bvlq:">4.5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->2,186<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>3.9<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Caribbean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.ziggybeachtulum.com/menu" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Stayed at the beach tulum and ate at ziggy’s for lunch and dinner many times...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">For the main dish had the <b>octopus</b>, grea...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline7" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23259804"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">28. Mamazzita Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556k2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556k2alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,184<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bars &amp; Pubs, Mexican</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="http://bit.ly/2PV0Uln" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Great service, Carlos was our server. The ambiance and atmosphere is really...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... wavy <b>meatballs</b>, prime rib tacos & s...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-19855557"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">29. Las Hijas De La Tostada Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556k6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556k6alb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->178<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://las-hijas-de-la-tostada-4.ola.click/products" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Not too sweet and quality ingredients. I had a hibiscus/mescal drink and a...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... and salmon <b>tostada</b> and both were su...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-21248033"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">30. Funky Geisha Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556kaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556kaalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,457<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Japanese, Sushi</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.funkygeisha.com.mx/en/menu-comida-asiatica-thai-japonesa" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... the Panang curry and all of our <b>sushi</b> (order the kimchi <b>sushi</b>!) was delic...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">A great restaurant in Tulum</span></div></a></div></div></div></div></div></div></div><div class="qAZoU c"><div class="GMYGA"><div class="mkNRT j"><div class="HXmEd"><div class="IGLCo"><a class="BrOJk u j z _F _S wSSLS tIqAi unMkR" data-smoke-attr="pagination-next-arrow" aria-label="Next page" href="/Restaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></a></div></div></div><div class="LaMdn j"><div class="aMnVW"><div class="Yzhnw P"><button class="BrOJk u j z _F _S wSSLS tIqAi iNBVo SSqtP" type="button" disabled="" aria-label="1"><span class="kLqdM"><span class="biGQs _P ezezH">1</span></span></button></div><div class="Yzhnw P"><a class="BrOJk u j z _F _S wSSLS tIqAi iNBVo" aria-label="2" href="/Restaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html"><span class="kLqdM"><span class="biGQs _P ezezH">2</span></span></a></div><div class="Yzhnw P"><div class="biGQs _P ezezH"><div class="sUgyD">…</div></div></div></div></div></div></div><div class="qAZoU c"><div class="biGQs _P VImYz ZNjnF"><div class="Ci">Showing results <!-- -->1<!-- -->-<!-- -->30<!-- --> of <!-- -->397</div></div></div></div></div></div></div></div></div><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="Pk PY Px PK"><div class="oTksw EFdEd"><div id="slot:970x250-728x90:footer" class="Eufjb j u ChFsW Xd o S"></div></div></div></div></div><div data-automation="restaurant-list-jsonld"><script type="application/ld+json">{"@context":"https:\u002F\u002Fschema.org","@type":"ItemList","itemListOrder":"https:\u002F\u002Fschema.org\u002FItemListOrderAscending","itemListElement":[{"@type":"ListItem","position":1,"item":{"@type":"Restaurant","name":"La Zebra Beach Restaurant And Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":6445},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002Fac\u002Ff1\u002Fd5\u002Frestaurant-interior-and.jpg"],"telephone":"+52 984 115 4726","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum A Boca Paila Km 8.2 La Zebra Hotel, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.14566,"longitude":-87.45949}}},{"@type":"ListItem","position":2,"item":{"@type":"Restaurant","name":"Fit-a-licious","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":913},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F23\u002Fd8\u002F44\u002Fa9\u002Four-new-look-come-check.jpg"],"telephone":"+52 984 176 0165","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Aldea Zama 26, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.199438,"longitude":-87.46239}}},{"@type":"ListItem","position":3,"item":{"@type":"Restaurant","name":"Mezzanine Thai Restaurant & Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":4783},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F25\u002F22\u002F34\u002F80\u002Fphad-thai.jpg"],"telephone":"+52 984 115 4728","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum-Boca Paila Zona Hotelera, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.197395,"longitude":-87.43733}}},{"@type":"ListItem","position":4,"item":{"@type":"Restaurant","name":"Restaurante Estrada","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1566},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F30\u002Ff9\u002F26\u002F4d\u002Festrada.jpg"],"telephone":"+52 984 124 0011","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Calle Centauro Sur Entre Gama Oriente y Asteroide Oriente, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.209045,"longitude":-87.45844}}},{"@type":"ListItem","position":5,"item":{"@type":"Restaurant","name":"Sexy Pizza Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":401},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2c\u002F1a\u002Ff1\u002Fb0\u002Fcaption.jpg"],"telephone":"+52 984 177 7336","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77776","streetAddress":"av Tulum 16 Av Principal de Tulum, Tulum 77776 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.212246,"longitude":-87.4588}}},{"@type":"ListItem","position":6,"item":{"@type":"Restaurant","name":"Lula's Kitchen","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":795},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002F22\u002Ff4\u002F4a\u002Fterrace-view.jpg"],"telephone":"+52 984 210 3028","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Carr. Tulum-Boca Paila km 8.1 Lula Seaside Boutique Hotel, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.145996,"longitude":-87.45929}}},{"@type":"ListItem","position":7,"item":{"@type":"Restaurant","name":"Distrito Panamera","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":175},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002F4c\u002F45\u002F2a\u002Fcaption.jpg"],"telephone":"+529982277902","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila Km 8. 5, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.212597,"longitude":-87.458786}}},{"@type":"ListItem","position":8,"item":{"@type":"Restaurant","name":"La Negra Tomasa","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1240},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1c\u002F63\u002F4d\u002F59\u002Fphoto0jpg.jpg"],"telephone":"+52 984 156 9555","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Andromeda Oriente # 14, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.211016,"longitude":-87.459915}}},{"@type":"ListItem","position":9,"item":{"@type":"Restaurant","name":"La Popular - N\u00F4made Temple Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":411},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002Fd8\u002F44\u002Fe1\u002Fla-popular.jpg"],"telephone":"+52 984 745 0314","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77880","streetAddress":"Carr. Tulum-Boca Paila Km 10.5 Tulum Beach, Tulum 77880 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.13606,"longitude":-87.46304}}},{"@type":"ListItem","position":10,"item":{"@type":"Restaurant","name":"Tantra Tulum Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1678},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2a\u002F28\u002Fa4\u002F11\u002Ftantra-tulum.jpg"],"telephone":"+52 998 690 0252","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila KM 7 Tantra Bohemian Luxury Beach Club, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.158932,"longitude":-87.455475}}},{"@type":"ListItem","position":11,"item":{"@type":"Restaurant","name":"Mi Amor Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":972},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F29\u002Fc9\u002Fd0\u002Fad\u002Frestaurant.jpg"],"telephone":"+52 1 984 188 4273","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila Km. 4.1 Zona Hotelera, Parque Nacional, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.194103,"longitude":-87.43926}}},{"@type":"ListItem","position":12,"item":{"@type":"Restaurant","name":"Alma Verde Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":348},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2a\u002Fef\u002Fff\u002Fc4\u002Fcaption.jpg"],"telephone":"+52 984 231 7473","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"M 2 Av. Cob\u00E1 Sur Lote 3 Plaza Hunab, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.203936,"longitude":-87.45003}}},{"@type":"ListItem","position":13,"item":{"@type":"Restaurant","name":"Casa Maria","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1049},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002F6a\u002Fea\u002Fb8\u002Ffish-tacos.jpg"],"telephone":"+52 984 876 2115","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Boca Paila Km 2.5 Diamante Hotel & Resturant, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.195839,"longitude":-87.43832}}},{"@type":"ListItem","position":14,"item":{"@type":"Restaurant","name":"Piaggia Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":340},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002Fe0\u002F3a\u002F6c\u002Fpiaggia-tulum-is-an-italo.jpg"],"telephone":"+52 984 310 7074","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Calla Gamma Ote Mz 4 Lote 1, Local 8a, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.212048,"longitude":-87.45109}}},{"@type":"ListItem","position":15,"item":{"@type":"Restaurant","name":"Fresco's","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1101},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2f\u002F93\u002F34\u002F2d\u002Ffresco-s-restaurant-entrnace.jpg"],"telephone":"+52 984 871 1141","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum-Boca Paila km 7 Zona Hotelera, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.156944,"longitude":-87.45616}}},{"@type":"ListItem","position":16,"item":{"@type":"Restaurant","name":"Onyx Tulum Restaurant Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":728},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1e\u002F5b\u002Fdf\u002F3c\u002Fview-of-the-restaurant.jpg"],"telephone":"+52 984 209 1788","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Calle Ori\u00F3n Sur M4 L-009, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.211458,"longitude":-87.460495}}},{"@type":"ListItem","position":17,"item":{"@type":"Restaurant","name":"El Suspiro","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":163},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F17\u002F8f\u002F84\u002Fed\u002Fnuestro-original-ceviche.jpg"],"telephone":"+52 998 279 1370","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Avenida Coba, Mz 36, lote 4 y 5 Mz 36, Lote 4 y 5, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.214872,"longitude":-87.45359}}},{"@type":"ListItem","position":18,"item":{"@type":"Restaurant","name":"Golden Cham Thai","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":321},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2a\u002F02\u002Fff\u002Fdb\u002Fit-s-a-great-place-to.jpg"],"telephone":"+52 984 120 3206","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Av. Tulum Entre Saturno y Luna Sur, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.209072,"longitude":-87.46888}}},{"@type":"ListItem","position":19,"item":{"@type":"Restaurant","name":"\u00C1tico Rooftop Bar & Lounge","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":158},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1c\u002Fd9\u002Ff1\u002F51\u002Finfinity-pool-at-atico.jpg"],"telephone":"+52 984 689 0506","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Mza 05 Av. Coba Lot 01, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.205343,"longitude":-87.450386}}},{"@type":"ListItem","position":20,"item":{"@type":"Restaurant","name":"Loco Tulum Mediterranean Kitchen","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":960},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1a\u002Fb3\u002F9e\u002Ff9\u002Flocos-at-loco-tulum-km.jpg"],"telephone":"+52 984 309 0149","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum Boca Paila Km 9.3, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.137993,"longitude":-87.46335}}},{"@type":"ListItem","position":21,"item":{"@type":"Restaurant","name":"Rosanegra Beach Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":238},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F22\u002F0c\u002F1b\u002F38\u002Fbest-beach-club-in-town.jpg"],"telephone":"+52 998 690 0283","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-boca Paila 5-km 7, Tulum Beach Zona Hotelera, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.151724,"longitude":-87.45831}}},{"@type":"ListItem","position":22,"item":{"@type":"Restaurant","name":"Cervecer\u00EDa Tulum Brewery Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1493},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002F6c\u002Fee\u002F98\u002Fla-mejor-cocina-mexicana.jpg"],"telephone":"+52 984 218 0506","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77789","streetAddress":"Carretera Canc\u00FAn - Chetumal Km. 230 Parque Nacional Del Jaguar De Tulum, Tulum 77789 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.232265,"longitude":-87.42345}}},{"@type":"ListItem","position":23,"item":{"@type":"Restaurant","name":"Bio-natural Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":305},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F23\u002Fde\u002F72\u002Fc3\u002Fbio-natural-tulum.jpg"],"telephone":"+52 984 168 6138","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Av Coba Plaza Mayan Monkey, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.204794,"longitude":-87.45029}}},{"@type":"ListItem","position":24,"item":{"@type":"Restaurant","name":"Confessions Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":997},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F23\u002F3f\u002Fc7\u002Fe1\u002Fconfessions.jpg"],"telephone":"+52 998 690 0275","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum- Boca Paila Zona Hotelera, Km. 7, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.159094,"longitude":-87.4552}}},{"@type":"ListItem","position":25,"item":{"@type":"Restaurant","name":"Curry Thai Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":732},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002F8b\u002F3c\u002F58\u002Fstaff.jpg"],"telephone":"+52 984 239 3265","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"C. 7 Sur La Veleta, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.202757,"longitude":-87.47367}}},{"@type":"ListItem","position":26,"item":{"@type":"Restaurant","name":"Copal Gastro Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":663},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F29\u002F7c\u002F05\u002F1c\u002Fcaption.jpg"],"telephone":"+52 984 181 6970","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Calle Ixchel esquina Av. Juanek, Aldea Zama, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.20358,"longitude":-87.46014}}},{"@type":"ListItem","position":27,"item":{"@type":"Restaurant","name":"Ziggy Beach Club & Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.5","reviewCount":2186},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002F1e\u002F3d\u002F74\u002Fbeach-club.jpg"],"telephone":"+52 984 871 1145","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Carr. Tulum-Boca Paila km 7.5 Zona Hotelera, Tulum Beach, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.156145,"longitude":-87.45642}}},{"@type":"ListItem","position":28,"item":{"@type":"Restaurant","name":"Mamazzita Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1184},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F22\u002F33\u002Fb4\u002F1c\u002Fmain-entrance-to-mamazzita.jpg"],"telephone":"+52 998 690 0274","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum Boca Paila, Km. 7 Zona Hotelera, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.15775,"longitude":-87.45616}}},{"@type":"ListItem","position":29,"item":{"@type":"Restaurant","name":"Las Hijas De La Tostada Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":178},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002Fa4\u002F5d\u002F94\u002Fshrimp-burger.jpg"],"telephone":"+52 984 235 0690","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Tulum Centro, Col Huracanes, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.211218,"longitude":-87.461555}}},{"@type":"ListItem","position":30,"item":{"@type":"Restaurant","name":"Funky Geisha Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":1457},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1c\u002F61\u002F87\u002Fd9\u002Flounge.jpg"],"telephone":"+52 998 690 0255","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila Km 7.8 Tulum Beach - Hotel Zone, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.149506,"longitude":-87.4594}}}]}</script></div><!--/$--></main><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="Kfkql Pg PN Pt PA Cj"><div class="Pg" data-test-target="global-nav-stacked"><div class=""><button aria-controls=":lithium-Rb9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-Rb9j9bvlq:" class="Duuud"><a href="/Tourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Tourism</span></a><a href="/Hotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotels</span></a><a href="/Hotels-g150813-c2-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Bed and Breakfast</span></a><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Restaurants</span></a><a href="/Attractions-g150813-Activities-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Things to Do in Tulum</span></a><a href="/ShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Travel Forum</span></a><a href="/LocationPhotos-g150813-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Photos</span></a><a href="/LocalMaps-g150813-Tulum-Area.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Map</span></a></div><button aria-controls=":lithium-Rj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-Rj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Tulum Hotels</span></a><a href="/SmartDeals-g150813-Tulum_Yucatan_Peninsula-Hotel-Deals.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotel Deals</span></a><a href="/LastMinute-g150813-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Last Minute Hotels in Tulum</span></a><a href="/HotelsNearMe" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels Near Me</span></a><button aria-controls=":lithium-R2qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">By Hotel Type</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R2qj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfd2-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Motels</span></a><a href="/Hotels-g150813-c3-zff26-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hostels</span></a><a href="/Hotels-g150813-c3-zff29-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Campgrounds</span></a><a href="/Hotels-g150813-zff105-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Glamping Stays in Tulum</span></a><a href="/Hotels-g150813-zff10-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Beach Hotels</span></a><a href="/Hotels-g150813-zff7-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Business Hotels</span></a><a href="/Hotels-g150813-zff4-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Family Hotels</span></a><a href="/Hotels-g150813-zff24-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Green Hotels</span></a><a href="/Hotels-g150813-zff8-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Resorts</span></a><a href="/Hotels-g150813-zff12-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Luxury Hotels</span></a><a href="/Hotels-g150813-zff3-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Romantic Hotels in Tulum</span></a><a href="/Hotels-g150813-zff14-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Casinos</span></a><a href="/Hotels-g150813-zff13-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Spa Resorts</span></a></div><button aria-controls=":lithium-R3aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">By Hotel Class</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R3aj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfc5-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">5-Star Hotels in Tulum</span></a><a href="/Hotels-g150813-zfc3-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">3-Star Hotels in Tulum</span></a><a href="/Hotels-g150813-zfc4-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">4-Star Hotels in Tulum</span></a></div><button aria-controls=":lithium-R3qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">By Hotel Brand</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R3qj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfb22174-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Ahau Collection Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb11779-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">OYO Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb11763-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Luxury Bahia Principe Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb16281-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Oasis Hotels &amp; Resorts in Tulum</span></a><a href="/Hotels-g150813-zfb9205-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">InterContinental (IHG) Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb9208-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Marriott Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb11845-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Colibri Boutique Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb9204-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hilton Hotels in Tulum</span></a></div><button aria-controls=":lithium-R4aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Amenities</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R4aj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfa9-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Pet Friendly Hotels in Tulum</span></a><a href="/Hotels-g150813-zfa3-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotels with Pools</span></a><a href="/Hotels-g150813-zfa7-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotels with Free Parking</span></a></div><button aria-controls=":lithium-R4qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Tulum Categories</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R4qj9j9bvlq:" class="Duuud"><a href="/HotelsList-Tulum-Boutique-Hotels-zfp1512.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Boutique Hotels in Tulum</span></a><a href="/HotelsList-Tulum-Cheap-Hotels-zfp11040.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Discount Hotels in Tulum</span></a><a href="/HotelsList-Tulum-Honeymoon-Resorts-zfp13996.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Honeymoon Resorts in Tulum</span></a><a href="/HotelsList-Tulum-Wedding-Hotels-zfp16067.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Wedding Hotels in Tulum</span></a><a href="/HotelsList-Tulum-All-Inclusive-Hotels-zfp41115.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Inclusive Hotels in Tulum</span></a><a href="/HotelsList-Tulum-All-Inclusive-Resorts-zfp1189394.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Inclusive Resorts in Tulum</span></a><a href="/HotelsList-Tulum-Adults-Only-And-Adult-Friendly-Resorts-zfp28929004.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Adults Only and Adult Friendly Resorts Tulum</span></a><a href="/HotelsList-Tulum-Golf-Hotels-zfp28929053.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Golf Hotels</span></a><a href="/HotelsList-Tulum-Singles-Resorts-zfp28929102.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Singles Resorts</span></a><a href="/HotelsList-Tulum-Spring-Break-Resorts-zfp28929151.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Spring Break Resorts in Tulum</span></a></div><button aria-controls=":lithium-R5aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Landmarks</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R5aj9j9bvlq:" class="Duuud"><a href="/HotelsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Tulum Archaeological Site</span></a><a href="/HotelsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Akumal Beach</span></a><a href="/HotelsNear-g150813-d1478385-Cenotes_Dos_Ojos-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Cenotes Dos Ojos</span></a><a href="/HotelsNear-g150813-d3321137-Playa_Paraiso-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Playa Paraiso</span></a><a href="/HotelsNear-g150813-d669578-Gran_Cenote-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Gran Cenote</span></a><a href="/HotelsNear-g499445-d153288-Yal_ku_Lagoon-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Yal-ku Lagoon</span></a><a href="/HotelsNear-g499445-d153913-Aktun_Chen-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Aktun Chen</span></a><a href="/HotelsNear-g499445-d10441459-Akumal_Monkey_Sanctuary_Rescued_Animals-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Akumal Monkey Sanctuary &amp; Rescued Animals</span></a><a href="/HotelsNear-g150813-d3387166-Cenotes_Sac_Actun-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Cenotes Sac Actun</span></a><a href="/HotelsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near The Yacht Experiences</span></a></div><button aria-controls=":lithium-R5qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Airports</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R5qj9j9bvlq:" class="Duuud"><a href="/HotelsNear-g150807-qCUN-Cancun_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Cancun Airport (CUN)</span></a><a href="/HotelsNear-g150813-qTQO-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Felipe Carrillo Puerto International Airport (TQO)</span></a><a href="/HotelsNear-g150812-qPCM-Playa_del_Carmen_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Aerodromo de Playa Del Carmen (PCM)</span></a><a href="/HotelsNear-g150812-qPCM-zfd2-Playa_del_Carmen_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Motels near Aerodromo de Playa Del Carmen (PCM)</span></a></div><button aria-controls=":lithium-R6aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Hotel Categories</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R6aj9j9bvlq:" class="Duuud"><a href="/HotelCategory-zft9435-Ryokan.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Ryokan</span></a><a href="/HotelCategory-zft6216-Family_Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Family Hotels</span></a><a href="/HotelCategory-zft6216,9672-Family_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Family Resorts</span></a><a href="/HotelCategory-zft9672,19075-Ski_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Ski Resorts</span></a><a href="/HotelCategory-zft9672,20284-Singles_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Singles Resorts</span></a><a href="/HotelCategory-zft9172,9189-All_Inclusive_Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular All-Inclusive Hotels</span></a><a href="/HotelCategory-zft9624,9672-Honeymoon_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Honeymoon Resorts</span></a><a href="/HotelCategory-zft6216,9172,9672-All_Inclusive_Family_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular All-Inclusive Family Resorts</span></a><a href="/HotelCategory-zft9172,9672,18884-Adults_Only_All_Inclusive_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Adults Only All-Inclusive Resorts</span></a><a href="/HotelCategory-zft9171-Casino_Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Casino Hotels</span></a></div></div><a href="/Attractions-g150813-Activities-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Things to Do</span></a><button aria-controls=":lithium-R139j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R139j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Tulum Restaurants</span></a><button aria-controls=":lithium-R1b39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Types of Food</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R1b39j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-c10698-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Argentinian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10686-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Street Food Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10631-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Peruvian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10772-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hawaiian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10679-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Healthy Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10639-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Latin Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10646-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Fast Food Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfz10992-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Gluten Free Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfz10665-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Vegetarian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfz10697-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Vegan Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c36-zfp6-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Spanish Restaurants with Outdoor Seating in Tulum</span></a><a href="/Restaurants-g150813-c37-zfp8-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Late Night Steakhouses in Tulum</span></a><a href="/Restaurants-g150813-c38-zfp30-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Sushi Restaurants for Lunch in Tulum</span></a></div><button aria-controls=":lithium-R1r39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Dishes</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R1r39j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-zfd10758-Tulum_Yucatan_Peninsula-Donuts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Donuts in Tulum</span></a><a href="/Restaurants-g150813-zfd10902-Tulum_Yucatan_Peninsula-French_Toast.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best French Toast in Tulum</span></a><a href="/Restaurants-g150813-zfd10890-Tulum_Yucatan_Peninsula-Chow_Mein.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Chow Mein in Tulum</span></a><a href="/Restaurants-g150813-zfd11724-Tulum_Yucatan_Peninsula-Unagi_Dojo_Eel_Soil.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Unagi &amp; Dojo (Eel &amp; Soil) in Tulum</span></a><a href="/Restaurants-g150813-zfd19960-Tulum_Yucatan_Peninsula-Cuban_Sandwich.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Cuban Sandwich in Tulum</span></a><a href="/Restaurants-g150813-zfd20040-Tulum_Yucatan_Peninsula-Mac_and_cheese.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Mac and cheese in Tulum</span></a><a href="/Restaurants-g150813-zfd20336-Tulum_Yucatan_Peninsula-Steak_Frites.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Steak Frites in Tulum</span></a><a href="/Restaurants-g150813-zfd10878-Tulum_Yucatan_Peninsula-Burrito.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Burritos in Tulum</span></a><a href="/Restaurants-g150813-zfd10905-Tulum_Yucatan_Peninsula-Gyros.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Gyros in Tulum</span></a><a href="/Restaurants-g150813-zfd10894-Tulum_Yucatan_Peninsula-Crab_Cakes.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Crab Cakes in Tulum</span></a><a href="/Restaurants-g150813-zfd20183-Tulum_Yucatan_Peninsula-Red_Curry.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Red Curry in Tulum</span></a><a href="/Restaurants-g150813-zfd20156-Tulum_Yucatan_Peninsula-Veggie_Burger.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Veggie Burger in Tulum</span></a><a href="/Restaurants-g150813-zfd20137-Tulum_Yucatan_Peninsula-Tres_Leches.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Tres Leches in Tulum</span></a><a href="/Restaurants-g150813-zfd20161-Tulum_Yucatan_Peninsula-Cucumber_Salad.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Cucumber Salad in Tulum</span></a><a href="/Restaurants-g150813-zfd20189-Tulum_Yucatan_Peninsula-Lettuce_Wraps.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Lettuce Wraps in Tulum</span></a></div><button aria-controls=":lithium-R2b39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Restaurant Categories</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R2b39j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-zfp2-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Breakfast Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp30-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Lunch Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp58-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Dinner Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp43-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants with a View in Tulum</span></a><a href="/Restaurants-g150813-zfp9-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants for Group Dining in Tulum</span></a><a href="/Restaurants-g150813-zfp49-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Buffet Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp8-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Late Night Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp6-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants with Outdoor Seating in Tulum</span></a><a href="/Restaurants-g150813-zfp19-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Food Delivery Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp5-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Kid Friendly Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfg9901-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Bakeries in Tulum</span></a><a href="/Restaurants-g150813-zfg9900-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Coffee &amp; Tea in Tulum</span></a><a href="/Restaurants-g150813-zfg9909-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Desserts in Tulum</span></a></div><button aria-controls=":lithium-R2r39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Hotels</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R2r39j9bvlq:" class="Duuud"><a href="/RestaurantsNear-g150813-d7223463-Lula_Seaside_Boutique_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Lula Seaside Boutique Hotel </span></a><a href="/RestaurantsNear-g150813-d23825874-Wakax_Hacienda_Cenote_Boutique_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Wakax Hacienda - Cenote &amp; Boutique Hotel</span></a><a href="/RestaurantsNear-g150813-d19006698-Hotel_Ma_xanab_Tulum-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Hotel Ma&#x27;xanab Tulum</span></a><a href="/RestaurantsNear-g150813-d7183451-Mi_Amor_Colibri_Boutique_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Mi Amor Colibri Boutique Hotel</span></a><a href="/RestaurantsNear-g150813-d557443-Mezzanine_Tulum_a_Small_Luxury_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Mezzanine Tulum, a Small Luxury Hotel</span></a><a href="/RestaurantsNear-g150813-d17495992-Kanan_Tulum-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Kanan Tulum</span></a><a href="/RestaurantsNear-g23240074-d296667-La_Zebra_an_SLH_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near La Zebra, an SLH Hotel</span></a><a href="/RestaurantsNear-g499445-d8285376-Secrets_Akumal_Riviera_Maya-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Secrets Akumal Riviera Maya</span></a><a href="/RestaurantsNear-g23240074-d557040-Suenos_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Suenos Tulum</span></a><a href="/RestaurantsNear-g23240074-d3979959-The_Beach_Tulum_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near The Beach Tulum Hotel</span></a></div><button aria-controls=":lithium-R3b39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Landmarks</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R3b39j9bvlq:" class="Duuud"><a href="/RestaurantsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Tulum Archaeological Site</span></a><a href="/RestaurantsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Akumal Beach</span></a><a href="/RestaurantsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near The Yacht Experiences</span></a><a href="/RestaurantsNear-g150813-d23065077-My_Quest_Concierge_Private_Tours-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near My Quest Concierge Private Tours</span></a><a href="/RestaurantsNear-g150813-d23852094-Dlux_Transfer-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Dlux Transfer</span></a><a href="/RestaurantsNear-g499445-d23928640-Marine_Life_Akumal_MX-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Marine Life Akumal MX</span></a><a href="/RestaurantsNear-g150813-d23285262-Mayikal_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Mayikal Experiences</span></a><a href="/RestaurantsNear-g150813-d26344863-Parque_Ecoturistico_Cenote_Aktun_Bej-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Parque Ecoturistico Cenote Aktun Bej</span></a><a href="/RestaurantsNear-g150813-d26394539-Tulum_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Tulum Experiences</span></a><a href="/RestaurantsNear-g150813-d20967421-Playacht_Private_Yacht_Rentals-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Playacht - Private Yacht Rentals</span></a></div></div><a href="/Articles" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Travel Stories</span></a><a href="/Cruises" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Cruises</span></a><button aria-controls=":lithium-R1r9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">More</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R1r9j9bvlq:" class="Duuud"><a href="/Attractions-g150813-Activities-c42-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tours</span></a><a href="/AddListing" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Add a Place</span></a><a href="/ShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Travel Forum</span></a><a href="/Airlines" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Airlines</span></a><a href="/TravelersChoice" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Travelers&#x27; Choice</span></a><a href="https://www.tripadvisorsupport.com/hc/en-us" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Help Center</span></a><a href="/Trips" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Trip Planner</span></a></div></div></div></div></div><footer data-component="ta-brand-footer" class="Gz Cj Pl PN Py PA"><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="dLJNv"><div class="MfOiD"><div class="MQpxt f _Y"><picture><img class="HAOGn F0 Nf" src="https://static.tacdn.com/img2/brand_refresh_2025/logos/logo.svg" alt="Tripadvisor"/></picture><div><div class="CDuLn"><div class="biGQs _P VImYz navcl">© <!-- -->2025<!-- --> Tripadvisor LLC<!-- --> <!-- -->All rights reserved.</div></div><div class="VPSQM f K"><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="https://tripadvisor.mediaroom.com/us-terms-of-use" rel="noopener" target="_self"><span class="biGQs _P ezezH">Terms of Use</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="https://tripadvisor.mediaroom.com/us-privacy-policy" rel="noopener" target="_self"><span class="biGQs _P ezezH">Privacy and Cookies Statement</span></a><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" type="button"><span class="biGQs _P ezezH">Cookie consent</span></button><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="/SiteIndex" rel="noopener" target="_self"><span class="biGQs _P ezezH">Site Map</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="/pages/serviceEN.html" rel="noopener" target="_self"><span class="biGQs _P ezezH">How the site works</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="https://tripadvisor.mediaroom.com/US-contact-us" rel="noopener" target="_blank"><span class="biGQs _P ezezH">Contact us</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="/Trust-lgF5hKLTqw3U.html" rel="noopener" target="_blank"><span class="biGQs _P ezezH">Accessibility Statement</span></a></div></div></div><div class="_T FKffI bmUTE"><div class="fIrGe _T bgMZj" style="-webkit-line-clamp:2;line-break:normal;cursor:auto"><div class="biGQs _P VImYz ZNjnF"><span class="JguWG"><p>This is the version of our website addressed to speakers of <!-- -->English<!-- --> in <!-- -->the United States<!-- -->. If you are a resident of another country or region, please select the appropriate version of Tripadvisor for your country or region in the drop-down menu.</p><br/><br/><p>Tripadvisor  LLC makes no guarantees for availability of prices advertised on our sites and applications. Listed prices may require a stay of a particular length or have blackout dates, qualifications or other applicable restrictions. Tripadvisor LLC is not responsible for any content on external web sites that are not owned or operated by Tripadvisor .<!-- --> <!-- -->Indicative hotel prices displayed on our “Explore” pages are estimates extrapolated from historic pricing data.</p><br/><br/><p>Tripadvisor LLC is not a booking agent or tour operator. When you book with one of our partners, please be sure to check their site for a full disclosure of all applicable fees.</p><br/><br/></span></div></div><div class="lszDU"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Read more</span><div class="CECjK _W"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div></div></div><div class="sVVKe"><div class="JLKop w"><button type="button" class="EVXMG _G f k Q2 u _u w Gn Z B2 BF Cq Ra _S wSSLS" aria-haspopup="listbox" aria-label="Currency: $ USD"><span class="biGQs _P ezezH">$ USD</span><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div class="JLKop w"><button type="button" class="EVXMG _G f k Q2 u _u w Gn Z B2 BF Cq Ra _S wSSLS" aria-haspopup="menu" aria-label="Region: United States"><span class="biGQs _P ezezH">United States</span><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div></div></div></div></footer><!--$--><!--/$--><!--$--><!--/$--><!--/$--></div><script nonce="f69050f0-d0a1-4b51-bfeb-2888452bd279" async src="data:text/javascript,(this.%24WP%3Dthis.%24WP%7C%7C%5B%5D).push(function(e)%7Bvar%20t%3De.clientEntrypoint%3Breturn%20delete%20e.clientEntrypoint%2C%5B%22bootstrap%22%2Cfunction(t%2Cn)%7Bvar%20r%3Breturn%5Bfunction()%7Br(document.getElementById(%22lithium-root%22)%2Ce%2Cnull%3D%3D%3De.lithiumBootstrap.renderError)%2C%22serviceWorker%22in%20navigator%26%26navigator.serviceWorker.getRegistrations().then(function(e)%7Be.forEach(function(e)%7Be.unregister()%7D)%7D)%2Cwindow.indexedDB%26%26indexedDB.deleteDatabase%26%26indexedDB.deleteDatabase(%22keyval-store%22)%7D%2C%5Bfunction(e)%7Br%3De.default%7D%5D%5D%7D%2C%5Bt%5D%5D%7D(JSON.parse(%22%7B%5C%22userAgent%5C%22%3A%7B%5C%22userAgentCategory%5C%22%3A%5C%22DESKTOP%5C%22%2C%5C%22os%5C%22%3A%7B%5C%22family%5C%22%3A%5C%22WINDOWS%5C%22%2C%5C%22majorVersion%5C%22%3A-1%7D%2C%5C%22browser%5C%22%3A%7B%5C%22family%5C%22%3A%5C%22CHROME%5C%22%2C%5C%22majorVersion%5C%22%3A143%7D%2C%5C%22isWebview%5C%22%3Afalse%2C%5C%22crawlerCategory%5C%22%3Anull%2C%5C%22nativeVersion%5C%22%3Anull%7D%2C%5C%22domainName%5C%22%3A%5C%22www.tripadvisor.com%5C%22%2C%5C%22cdnPrefix%5C%22%3A%5C%22https%3A%2F%2Fstatic.tacdn.com%5C%22%2C%5C%22locale%5C%22%3A%5C%22en-US%5C%22%2C%5C%22userId%5C%22%3Anull%2C%5C%22sessionId%5C%22%3A%5C%229F2A39C10AC80C288E8FA7C29CCF35D3%5C%22%2C%5C%22route%5C%22%3A%5B%7B%5C%22page%5C%22%3A%5C%22Restaurants%5C%22%2C%5C%22params%5C%22%3A%7B%5C%22geoId%5C%22%3A150813%2C%5C%22diningOptionId%5C%22%3A10955%7D%2C%5C%22path%5C%22%3A%5C%22%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%22%2C%5C%22fragment%5C%22%3A%5C%22%5C%22%7D%2C%7B%5C%22uid%5C%22%3A%5C%224a4f2ed6-0a7c-4d54-a2e8-4073767cfd12%5C%22%7D%5D%2C%5C%22cookies%5C%22%3A%7B%7D%2C%5C%22webviewCurrencyOverride%5C%22%3Anull%2C%5C%22serverSideCanonicalized%5C%22%3Atrue%2C%5C%22timeOrigin%5C%22%3A1766606129462%2C%5C%22documentInstalledChunkIds%5C%22%3A%5B%5C%22h2v4d7%5C%22%2C%5C%22iucilr%5C%22%2C%5C%22mg6599%5C%22%2C%5C%22pwxlu9%5C%22%2C%5C%22gg6fk5%5C%22%2C%5C%22bfou8q%5C%22%2C%5C%2221h32l%5C%22%2C%5C%22qndsds%5C%22%2C%5C%22o4yt4q%5C%22%2C%5C%2202sgqj%5C%22%2C%5C%22a349lq%5C%22%2C%5C%22lq6xsf%5C%22%2C%5C%22mb2nze%5C%22%2C%5C%2283pm0d%5C%22%2C%5C%22q18gbm%5C%22%2C%5C%229b9b10%5C%22%2C%5C%22973cci%5C%22%2C%5C%227thqz4%5C%22%2C%5C%22p5p2yo%5C%22%2C%5C%22fgwvq1%5C%22%2C%5C%22ex1xu3%5C%22%2C%5C%22qjohxf%5C%22%2C%5C%225vmpsm%5C%22%2C%5C%22ys6v1o%5C%22%2C%5C%22al4vil%5C%22%2C%5C%22cyrz7y%5C%22%2C%5C%22xo1is3%5C%22%2C%5C%2252far5%5C%22%2C%5C%22nnprhg%5C%22%2C%5C%226hf0eg%5C%22%2C%5C%22w1o2w8%5C%22%2C%5C%22iog21h%5C%22%2C%5C%2260mj7v%5C%22%2C%5C%22fkji80%5C%22%2C%5C%22w1vb4i%5C%22%2C%5C%22r73jw8%5C%22%2C%5C%229m9122%5C%22%2C%5C%22jppfwt%5C%22%2C%5C%22i0z8n1%5C%22%2C%5C%22qkjm8h%5C%22%2C%5C%22towdyv%5C%22%2C%5C%22cb79np%5C%22%2C%5C%22l7164z%5C%22%2C%5C%22vpo3wt%5C%22%2C%5C%22llqsfm%5C%22%2C%5C%22qlcuu5%5C%22%2C%5C%22fbuhsy%5C%22%2C%5C%222mrepu%5C%22%2C%5C%22s6q22n%5C%22%2C%5C%22nntg4s%5C%22%2C%5C%22zzfm0c%5C%22%2C%5C%226tkc5e%5C%22%2C%5C%22ih0468%5C%22%2C%5C%22nv7v96%5C%22%2C%5C%22ku7suo%5C%22%2C%5C%2294v69w%5C%22%2C%5C%22nv8ze5%5C%22%2C%5C%22g9ot0r%5C%22%2C%5C%22essqvy%5C%22%2C%5C%22gw8uw6%5C%22%2C%5C%2234bu7w%5C%22%2C%5C%22lr6tb6%5C%22%2C%5C%22f1x7n0%5C%22%2C%5C%22voa3hw%5C%22%2C%5C%22254k2b%5C%22%2C%5C%22mqwhqn%5C%22%2C%5C%22q7fbfw%5C%22%2C%5C%22ns7yrw%5C%22%2C%5C%2249c7br%5C%22%2C%5C%226nmgrj%5C%22%2C%5C%22hckdpv%5C%22%2C%5C%22fgi73t%5C%22%2C%5C%22b5nukb%5C%22%2C%5C%22kb5au4%5C%22%2C%5C%227dp9gx%5C%22%2C%5C%224uqz87%5C%22%2C%5C%22a7906a%5C%22%2C%5C%22eap1p5%5C%22%2C%5C%225j1idx%5C%22%2C%5C%227u7gib%5C%22%2C%5C%22q13wda%5C%22%2C%5C%22bp96i9%5C%22%2C%5C%2255gplo%5C%22%2C%5C%22b478w0%5C%22%2C%5C%22eyzj96%5C%22%2C%5C%22s2x8on%5C%22%2C%5C%22ueacbr%5C%22%2C%5C%22trf9tg%5C%22%2C%5C%22o3mm96%5C%22%2C%5C%22nbupuc%5C%22%2C%5C%22ezmx4w%5C%22%2C%5C%22dpzonz%5C%22%2C%5C%22w1yg77%5C%22%2C%5C%22jgmezl%5C%22%2C%5C%22p3y1zq%5C%22%2C%5C%2232pwc7%5C%22%2C%5C%22dkml3k%5C%22%2C%5C%22198ozm%5C%22%2C%5C%22p9ucm8%5C%22%2C%5C%221mwjfl%5C%22%2C%5C%22tsg6fc%5C%22%2C%5C%223w1vrt%5C%22%2C%5C%223btuy0%5C%22%2C%5C%22gwsdl1%5C%22%2C%5C%227yaz66%5C%22%2C%5C%22xg719z%5C%22%2C%5C%22uto6xl%5C%22%2C%5C%22rr68op%5C%22%2C%5C%22gzffp2%5C%22%2C%5C%227rx7bi%5C%22%2C%5C%22q4uvd4%5C%22%2C%5C%22covaqn%5C%22%2C%5C%22t3fmar%5C%22%2C%5C%22n6iz1t%5C%22%2C%5C%225n3iuq%5C%22%2C%5C%22vb6y54%5C%22%2C%5C%22epp8eq%5C%22%2C%5C%227nq86i%5C%22%2C%5C%22bxtwp3%5C%22%2C%5C%22zq967r%5C%22%2C%5C%224ilds0%5C%22%2C%5C%22ntn269%5C%22%2C%5C%22xm2jzo%5C%22%2C%5C%22my7i5t%5C%22%2C%5C%223yhbda%5C%22%2C%5C%22wt5b21%5C%22%2C%5C%22i3sz5h%5C%22%2C%5C%22frf6ac%5C%22%2C%5C%22kq4525%5C%22%2C%5C%22mqfj4j%5C%22%2C%5C%2200mpxl%5C%22%2C%5C%22la678j%5C%22%2C%5C%22pydmep%5C%22%2C%5C%2268fm7o%5C%22%2C%5C%222r4qug%5C%22%2C%5C%22p98jyo%5C%22%2C%5C%228e77a5%5C%22%2C%5C%22livpad%5C%22%2C%5C%22ac38c0%5C%22%2C%5C%22mhkl02%5C%22%2C%5C%22uf95jv%5C%22%2C%5C%221b5l5d%5C%22%2C%5C%22eq3hjy%5C%22%2C%5C%22dcz41w%5C%22%2C%5C%225rvmf1%5C%22%2C%5C%220a0jsz%5C%22%2C%5C%22dlu2yg%5C%22%2C%5C%22teh9fa%5C%22%2C%5C%229lrsvx%5C%22%2C%5C%222tx7su%5C%22%2C%5C%2275vfyw%5C%22%2C%5C%22u1kaei%5C%22%2C%5C%22556vjb%5C%22%2C%5C%22u9jlwp%5C%22%2C%5C%22etaac9%5C%22%2C%5C%22cmm6zr%5C%22%2C%5C%22ahn2ts%5C%22%2C%5C%22bzl252%5C%22%2C%5C%22ay1l3c%5C%22%2C%5C%220v7vxf%5C%22%2C%5C%22008v8e%5C%22%2C%5C%22ue3yko%5C%22%2C%5C%22it6k5j%5C%22%2C%5C%22og41x6%5C%22%2C%5C%225azdze%5C%22%2C%5C%22cwl85x%5C%22%2C%5C%22gfv5nk%5C%22%2C%5C%22lfeddg%5C%22%2C%5C%22u3wokd%5C%22%2C%5C%22227h6c%5C%22%2C%5C%220ad67e%5C%22%2C%5C%22lgvzhu%5C%22%2C%5C%22kqywgi%5C%22%2C%5C%22qx89h8%5C%22%2C%5C%22rdycz1%5C%22%2C%5C%22gsd8gu%5C%22%2C%5C%224euaes%5C%22%2C%5C%22nylcph%5C%22%2C%5C%22ljqkwp%5C%22%2C%5C%22s1ymgp%5C%22%2C%5C%22gjtatx%5C%22%2C%5C%22gudzb7%5C%22%2C%5C%22j56vim%5C%22%2C%5C%220khkbz%5C%22%2C%5C%229ygz84%5C%22%2C%5C%225jr1ur%5C%22%2C%5C%227a7jm3%5C%22%2C%5C%22sciq8z%5C%22%2C%5C%22dvdzap%5C%22%2C%5C%22cbdk2b%5C%22%2C%5C%22v3e972%5C%22%2C%5C%22s557se%5C%22%2C%5C%22aj7avv%5C%22%2C%5C%22rh2yho%5C%22%2C%5C%22r3rvh1%5C%22%2C%5C%22lme1i4%5C%22%2C%5C%22ou1xb8%5C%22%2C%5C%22gbnboa%5C%22%2C%5C%225gpqhx%5C%22%2C%5C%22q7wvkq%5C%22%2C%5C%22uvk21c%5C%22%2C%5C%22mtbtnn%5C%22%2C%5C%22gyzem5%5C%22%2C%5C%22ulejzu%5C%22%2C%5C%22j8lyt5%5C%22%2C%5C%22g05qlv%5C%22%2C%5C%22uujvhg%5C%22%2C%5C%22wstkum%5C%22%2C%5C%22t9nvqo%5C%22%2C%5C%22dsbyen%5C%22%2C%5C%22u5g3tb%5C%22%2C%5C%22mdhor2%5C%22%2C%5C%22sljmfx%5C%22%2C%5C%22r3kgcy%5C%22%2C%5C%22vjlqzu%5C%22%2C%5C%22doaw04%5C%22%2C%5C%2298inwa%5C%22%2C%5C%22xj2t7q%5C%22%2C%5C%22vumu0s%5C%22%2C%5C%22bt3bv2%5C%22%2C%5C%224x6ber%5C%22%2C%5C%22904jnz%5C%22%2C%5C%22b8gyi1%5C%22%2C%5C%22vl1wbo%5C%22%2C%5C%22tbkd43%5C%22%2C%5C%22btbawi%5C%22%2C%5C%228iun0t%5C%22%2C%5C%22nexzid%5C%22%2C%5C%22y9ubrl%5C%22%2C%5C%22tp1g68%5C%22%2C%5C%22t05m6z%5C%22%2C%5C%22j8mzb2%5C%22%2C%5C%22pwoff0%5C%22%2C%5C%22j77a03%5C%22%2C%5C%22evvugf%5C%22%2C%5C%22nlswq2%5C%22%2C%5C%22ue4abb%5C%22%2C%5C%22q213pt%5C%22%2C%5C%2298810w%5C%22%2C%5C%22sor0m3%5C%22%2C%5C%229w33la%5C%22%2C%5C%22sdpjgr%5C%22%2C%5C%22rdeq8t%5C%22%2C%5C%22te6757%5C%22%2C%5C%22cngeux%5C%22%2C%5C%22v1gmh9%5C%22%2C%5C%22nunr2w%5C%22%2C%5C%22c6w8qp%5C%22%2C%5C%221rki99%5C%22%2C%5C%22meqo19%5C%22%2C%5C%2222xnaq%5C%22%2C%5C%22uedfnc%5C%22%2C%5C%22a6i9w3%5C%22%2C%5C%226qmqhw%5C%22%2C%5C%22fa3gob%5C%22%2C%5C%2202kko1%5C%22%2C%5C%2248k7uc%5C%22%2C%5C%22nnmev3%5C%22%2C%5C%22ffrmny%5C%22%2C%5C%22kq4q2w%5C%22%2C%5C%2290bo0h%5C%22%2C%5C%22joj99y%5C%22%2C%5C%229zpurr%5C%22%2C%5C%22xp34yw%5C%22%2C%5C%227b6vjf%5C%22%2C%5C%22acx4g3%5C%22%2C%5C%22cra0rz%5C%22%2C%5C%22q5b1x4%5C%22%2C%5C%22e0avcx%5C%22%2C%5C%22ejmcf4%5C%22%2C%5C%22r144n1%5C%22%2C%5C%22bmouzl%5C%22%2C%5C%22tvte1l%5C%22%2C%5C%22rrxjvt%5C%22%2C%5C%2265rb8q%5C%22%2C%5C%22ltllz9%5C%22%2C%5C%22x9dv95%5C%22%2C%5C%2279a2um%5C%22%2C%5C%22mdvsf7%5C%22%2C%5C%2279x4p0%5C%22%2C%5C%22835myt%5C%22%2C%5C%22d8oes5%5C%22%2C%5C%22uzmmz7%5C%22%2C%5C%22ky3i2c%5C%22%2C%5C%22gcjubj%5C%22%2C%5C%22o5y7tg%5C%22%2C%5C%22yw8i09%5C%22%2C%5C%22u5pild%5C%22%2C%5C%229rb8bq%5C%22%2C%5C%223dy54m%5C%22%2C%5C%22eww825%5C%22%2C%5C%22nute5e%5C%22%2C%5C%22wenm84%5C%22%2C%5C%22lnxtb7%5C%22%2C%5C%22wgun1f%5C%22%2C%5C%22chbmyv%5C%22%2C%5C%22qumvh0%5C%22%2C%5C%22b4ge9e%5C%22%2C%5C%22b9nflw%5C%22%2C%5C%222rns1i%5C%22%2C%5C%22k3csuh%5C%22%2C%5C%22b9lpgd%5C%22%2C%5C%22t3d7hy%5C%22%2C%5C%22mvpl5y%5C%22%2C%5C%22a2xbq6%5C%22%2C%5C%224ecb70%5C%22%2C%5C%22j9pg61%5C%22%2C%5C%22iq7kpa%5C%22%2C%5C%22yau6qy%5C%22%2C%5C%22lt9m20%5C%22%2C%5C%22bckiht%5C%22%2C%5C%22gttbjc%5C%22%2C%5C%2243er8k%5C%22%2C%5C%22vgujbz%5C%22%2C%5C%22conwso%5C%22%2C%5C%22qhneqr%5C%22%2C%5C%22it954g%5C%22%2C%5C%22qbn8x2%5C%22%2C%5C%224q9l40%5C%22%2C%5C%22tv5det%5C%22%2C%5C%22vk1dz4%5C%22%2C%5C%22d4cn15%5C%22%2C%5C%22bljygq%5C%22%2C%5C%22af72y8%5C%22%2C%5C%22k64pus%5C%22%2C%5C%22hwdnu5%5C%22%2C%5C%227l5hpv%5C%22%2C%5C%22nc3gmf%5C%22%2C%5C%22l75rcq%5C%22%2C%5C%22zb2uaz%5C%22%2C%5C%227gslcl%5C%22%2C%5C%22n9cueu%5C%22%2C%5C%225hne4v%5C%22%2C%5C%22bxkng5%5C%22%2C%5C%22w5k17a%5C%22%2C%5C%22aqiall%5C%22%2C%5C%22hxluky%5C%22%2C%5C%22f7mzgd%5C%22%2C%5C%22dyfdn5%5C%22%2C%5C%22bwm4qt%5C%22%2C%5C%22eozxiw%5C%22%2C%5C%22sa2a6q%5C%22%2C%5C%22uc42ar%5C%22%2C%5C%229ccyt3%5C%22%2C%5C%22ov8w1o%5C%22%2C%5C%22h9085v%5C%22%2C%5C%22obrjv3%5C%22%2C%5C%22pts58q%5C%22%2C%5C%22rctp90%5C%22%2C%5C%220wbf8p%5C%22%2C%5C%22ed1cs2%5C%22%2C%5C%22bnv2ns%5C%22%2C%5C%22c5q3iu%5C%22%2C%5C%22ja55rt%5C%22%2C%5C%22jbz0uy%5C%22%2C%5C%22asj0sy%5C%22%2C%5C%22kt39pq%5C%22%2C%5C%22bkeq24%5C%22%2C%5C%22sw5ax0%5C%22%2C%5C%22w7fngn%5C%22%2C%5C%22iyqcwg%5C%22%2C%5C%22d6vj30%5C%22%2C%5C%22y0uq31%5C%22%2C%5C%22neutur%5C%22%2C%5C%22hjkifi%5C%22%2C%5C%222ksxja%5C%22%2C%5C%222rd76e%5C%22%2C%5C%222mqr6r%5C%22%2C%5C%22bpwwxo%5C%22%2C%5C%22p4a4ly%5C%22%2C%5C%22lfyiyn%5C%22%2C%5C%22ywfh04%5C%22%2C%5C%22eh8z4m%5C%22%2C%5C%22vgfso9%5C%22%2C%5C%22yjij17%5C%22%2C%5C%22hfmoib%5C%22%2C%5C%22yk7it8%5C%22%2C%5C%2274j83y%5C%22%2C%5C%22fbyv8z%5C%22%2C%5C%22nev0g8%5C%22%2C%5C%22u47wjb%5C%22%2C%5C%22wehxiu%5C%22%2C%5C%22ynqy3n%5C%22%2C%5C%220506zk%5C%22%2C%5C%22knab8m%5C%22%2C%5C%22dvcr7s%5C%22%2C%5C%22tj4q1a%5C%22%2C%5C%22an500m%5C%22%2C%5C%22vvrdxj%5C%22%2C%5C%22flcs8h%5C%22%2C%5C%222l7tdf%5C%22%2C%5C%22j1i9fl%5C%22%2C%5C%22x55bmb%5C%22%2C%5C%22lzos1d%5C%22%2C%5C%22fug67i%5C%22%2C%5C%22sokx6p%5C%22%2C%5C%22lwi7bf%5C%22%2C%5C%222gg3s2%5C%22%2C%5C%224pg8m6%5C%22%2C%5C%22pia9xj%5C%22%2C%5C%22mgk0ro%5C%22%2C%5C%22nfeum5%5C%22%2C%5C%222wld0u%5C%22%2C%5C%223309mt%5C%22%2C%5C%229heuct%5C%22%2C%5C%22o846n4%5C%22%2C%5C%22v5ce81%5C%22%2C%5C%22pra5l9%5C%22%2C%5C%2287i435%5C%22%2C%5C%22rkm46w%5C%22%2C%5C%22lt711y%5C%22%2C%5C%22qha8vq%5C%22%2C%5C%22s9vc2i%5C%22%2C%5C%2271uqev%5C%22%2C%5C%22mc5s4b%5C%22%2C%5C%22irfizf%5C%22%2C%5C%226g4avl%5C%22%2C%5C%22665v5s%5C%22%2C%5C%222tnrli%5C%22%2C%5C%22gol71b%5C%22%2C%5C%22fgamlw%5C%22%2C%5C%22ctn2v9%5C%22%2C%5C%223tcfrp%5C%22%2C%5C%22crdfjq%5C%22%2C%5C%22t9pxn7%5C%22%2C%5C%22tax17a%5C%22%2C%5C%2219jhx7%5C%22%2C%5C%22y74olp%5C%22%2C%5C%22euncuq%5C%22%2C%5C%229btuwa%5C%22%2C%5C%22h1hqsi%5C%22%2C%5C%22x53oqt%5C%22%2C%5C%229wtdt4%5C%22%2C%5C%22yr8sh8%5C%22%2C%5C%22eki1kt%5C%22%2C%5C%228ny8va%5C%22%2C%5C%22cbyjc7%5C%22%2C%5C%22tochuq%5C%22%2C%5C%22amsjo9%5C%22%2C%5C%221ycpp4%5C%22%2C%5C%223x7pav%5C%22%2C%5C%22dg00hx%5C%22%2C%5C%223vmom9%5C%22%2C%5C%22kepfi8%5C%22%2C%5C%220mfk1q%5C%22%2C%5C%22uc4lhh%5C%22%2C%5C%22ph948p%5C%22%2C%5C%22mklkzb%5C%22%2C%5C%22bw3tql%5C%22%2C%5C%22fdsghl%5C%22%2C%5C%22co2nzy%5C%22%2C%5C%22qpz1y2%5C%22%2C%5C%220rhj0x%5C%22%2C%5C%22b49gxf%5C%22%2C%5C%228cre3h%5C%22%2C%5C%22l2u3je%5C%22%2C%5C%22ib7588%5C%22%2C%5C%22voprus%5C%22%2C%5C%228ll3ol%5C%22%2C%5C%22ck5rom%5C%22%2C%5C%22p7jq7m%5C%22%2C%5C%221tem5o%5C%22%2C%5C%22b5kt6r%5C%22%2C%5C%22w43m8k%5C%22%2C%5C%22y9enps%5C%22%2C%5C%22n4zdla%5C%22%2C%5C%22633gr8%5C%22%2C%5C%226h4i0r%5C%22%2C%5C%22us1vwl%5C%22%2C%5C%225xqg72%5C%22%2C%5C%2270j2tf%5C%22%2C%5C%22xfufl0%5C%22%2C%5C%22o80y56%5C%22%2C%5C%22ypf49l%5C%22%2C%5C%22ti102j%5C%22%2C%5C%22d3i8k1%5C%22%2C%5C%22mxujx7%5C%22%2C%5C%22eekt6e%5C%22%2C%5C%22z2nh2s%5C%22%2C%5C%22mf5oe4%5C%22%2C%5C%22v42v3u%5C%22%2C%5C%2276j3t0%5C%22%5D%2C%5C%22cspNonce%5C%22%3A%5C%22f69050f0-d0a1-4b51-bfeb-2888452bd279%5C%22%2C%5C%22referrerHeader%5C%22%3Anull%2C%5C%22lithiumBootstrap%5C%22%3A%7B%5C%22deprecatedPersistentId%5C%22%3A%5C%224ac80746.a0ea.45e6.a4b7.5fe08adcf88a.19B5198B473%5C%22%2C%5C%22version%5C%22%3A%5C%222385786954%5C%22%2C%5C%22debugTool%5C%22%3Afalse%2C%5C%22ssrLogs%5C%22%3A%5B%5D%2C%5C%22serviceCalls%5C%22%3A%5B%5D%2C%5C%22renderError%5C%22%3Anull%2C%5C%22stagingError%5C%22%3Anull%2C%5C%22staticServiceOverrides%5C%22%3A%7B%7D%2C%5C%22hmrEnabled%5C%22%3Afalse%7D%2C%5C%22clientEntrypoint%5C%22%3A%5C%22ib7588%5C%22%2C%5C%22urqlSsrData%5C%22%3A%7B%5C%22results%5C%22%3A%7B%5C%2265767282%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Internallinks_buildHreflangs%5C%5C%5C%22%3A%7B%5C%5C%5C%22hreflangs%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-GB%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.uk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-CA%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ca%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr-CA%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ffr.tripadvisor.ca%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22it%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.it%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.es%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22de%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.de%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.fr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ja%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.jp%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22zh-Hans%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcn.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-IN%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.in%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22sv%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.se%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22nl%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.nl%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22pt%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.br%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22tr%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.tr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22da%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.dk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-MX%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.mx%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-IE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ie%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ar%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Far.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ar-EG%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.eg%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22de-AT%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.at%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22el%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.gr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-AU%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.au%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-MY%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.my%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-NZ%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.nz%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-PH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.ph%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-SG%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.sg%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-ZA%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.za%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-AR%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.ar%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-CL%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.cl%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-CO%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-PE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.pe%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-VE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.ve%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22id%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.id%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22he%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.il%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ko%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.kr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22nb%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fno.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22pt-PT%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.pt%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ru%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ru%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22th%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fth.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22vi%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.vn%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22zh-Hant%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.tw%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22de-CH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ch%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr-CH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ffr.tripadvisor.ch%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22it-CH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fit.tripadvisor.ch%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-HK%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fen.tripadvisor.com.hk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr-BE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ffr.tripadvisor.be%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22nl-BE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.be%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22zh-hk%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.hk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%5D%7D%7D%5C%22%7D%2C%5C%22156332295%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22currency%5C%5C%5C%22%3A%7B%5C%5C%5C%22code%5C%5C%5C%22%3A%5C%5C%5C%22USD%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22U.S.%20Dollars%5C%5C%5C%22%2C%5C%5C%5C%22symbol%5C%5C%5C%22%3A%5C%5C%5C%22%24%5C%5C%5C%22%7D%7D%5C%22%7D%2C%5C%221190521020%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22No%20results%20were%20found%20in%20Tulum%2C%20but%20here%20are%20some%20options%20nearby.%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%222772378239%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22Top%20restaurants%20in%20Tulum%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%223291687264%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22detail%5C%5C%5C%22%3A%7B%5C%5C%5C%22info%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%2C%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.21242%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.463547%7D%7D%7D%7D%5C%22%7D%2C%5C%224059846075%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22response%5C%5C%5C%22%3A%7B%5C%5C%5C%22__typename%5C%5C%5C%22%3A%5C%5C%5C%22RestaurantPresentation_NarrowGeoResponse%5C%5C%5C%22%2C%5C%5C%5C%22contentTitle%5C%5C%5C%22%3A%5C%5C%5C%22Top%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22restaurants%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A24855625%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Alma%20Verde%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.203936%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45003%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.almaverde.com.mx%2Fcopia-de-copia-de-menu-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A348%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2Fef%2Fff%2Fc4%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1152%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20231%207473%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22M%202%20Av.%20Cob%C3%A1%20Sur%20Lote%203%20Plaza%20Hunab%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20ordered%20the%20%EF%BF%B9avocado%20toast%EF%BF%BB%20and%20the%20eggs%20Benedict%2C%20both%20were%20absolutely%20deli...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Amazing%20breakfast%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%22c6b48957-8af6-4397-a9c3-1ba4b7759b6e%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A25294307%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Xibak%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.200266%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46343%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fbeta.menuflow.com%2F900001450%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22VkowXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjQ4MzAzNjk2Jmdlbz0yNTI5NDMwNyZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4WTRTNWRnUzlsOEgyWFgxQUtNX1RhU0RDVHF6UEh6aTFUbjRhenUwV3dCSSZjcz0xMjRiZmY3NDlkN2Y0MTUxZmU0ZWNjNjVjMDQ4M2NkYmVfR3Q0%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPENING%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Opens%20in%204%20min%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A88%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F28%2F91%2F35%2Ff5%2Fxibak-tulum.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2691%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2953%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20310%204024%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Itzamna%20%23178%20Aldea%20Zama%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Nice%20Dinner%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d25294307-r992355448-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d25294307-r992355448-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Perfect%20Dining%20Experience%20in%20Aldea%20Zama!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d25294307-r986894307-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d25294307-r986894307-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%224c45f661-680d-4f91-b2d8-e9bf8bfafe5b%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A2286855%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Zebra%20Beach%20Restaurant%20And%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.14566%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45949%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flazebratulum.com%2Fdining-2%2F%23menu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22OEdSXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTcxODgyJmdlbz0yMjg2ODU1JmZyb209UmVzdGF1cmFudHMmYXJlYT1yZXNlcnZhdGlvbl9idXR0b24mc2xvdD0xJm1hdGNoSUQ9MSZvb3M9MCZjbnQ9MSZzaWxvPTQ1NTgyJmJ1Y2tldD05Nzk2NDEmbnJhbms9MSZjcmFuaz0xJmNsdD1SJnR0eXBlPVJlc3RhdXJhbnQmdG09MzQ2NTE3NzI5Jm1hbmFnZWQ9ZmFsc2UmY2FwcGVkPWZhbHNlJmdvc294PTlETXZpYkRUV2ROYUZlR1lMOFFUYWt5djFMQllaYUdwZUI0TUVIbi1odjJtWkJOMzc2RUhWQ0d1c1RJUnhtSjhVWUNVaWF0YUZIMXBQak9uY3doUUVqbkNTcE1EcTV2eE5BSDg0OWI0ZkVjJmNzPTFlMTZkNWEyNmYxZTE5MjkzMDYxYmViNDA0ZWNhYzZmYl9VZjI%3D%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A6445%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fac%2Ff1%2Fd5%2Frestaurant-interior-and.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2240%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3360%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20115%204726%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum%20A%20Boca%20Paila%20Km%208.2%20La%20Zebra%20Hotel%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Provides%20a%20variety%20of%20rooms%2C%20restaurants%2C%20and%20%EF%BF%B9bar%EF%BF%BB%20as%20well%20as%20beach%20area%20to%20en...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d2286855-r1039942673-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d2286855-r1039942673-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Edgar%2C%20%EF%BF%B9tacos%EF%BF%BB%2C%20and%20drinks%E2%80%A6%20amazing!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d2286855-r1023181036-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d2286855-r1023181036-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23483492%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Fit-a-licious%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.199438%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46239%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Flinktr.ee%2FFitaliciousmx%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A913%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F23%2Fd8%2F44%2Fa9%2Four-new-look-come-check.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1500%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20176%200165%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Aldea%20Zama%2026%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22%EF%BF%B9Arepas%EF%BF%BB%2C%20sough%20dough%20French%20toast%2C%20quesadillas%20and%20amazing%20Gluten%20free%20salmon...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23483492-r1009328024-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23483492-r1009328024-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Had%20the%20%EF%BF%B9guacamole%EF%BF%BB%20and%20smoked%20salmon%20rye...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23483492-r1040878747-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23483492-r1040878747-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A1144210%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mezzanine%20Thai%20Restaurant%20%26%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.197395%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.43733%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fmezzaninetulum.com%2Fmenus%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22dUdxXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTc5ODczJmdlbz0xMTQ0MjEwJmZyb209UmVzdGF1cmFudHMmYXJlYT1yZXNlcnZhdGlvbl9idXR0b24mc2xvdD0xJm1hdGNoSUQ9MSZvb3M9MCZjbnQ9MSZzaWxvPTQ1NTgyJmJ1Y2tldD05Nzk2NDEmbnJhbms9MSZjcmFuaz0xJmNsdD1SJnR0eXBlPVJlc3RhdXJhbnQmdG09MzQ2NTE3NzI5Jm1hbmFnZWQ9ZmFsc2UmY2FwcGVkPWZhbHNlJmdvc294PTlETXZpYkRUV2ROYUZlR1lMOFFUYWt5djFMQllaYUdwZUI0TUVIbi1odjJtWkJOMzc2RUhWQ0d1c1RJUnhtSjhKTVctZlhKdjlWRi10b0ZOaWNQbkZleFdjTXRJQWg5dVMxSG5SdGhOdGRNJmNzPTFmNjczZjEzMTE0ZDI5YmU4ODFhZWI0NDU1YTRiMDg3Yl9XQ0M%3D%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A4783%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F25%2F22%2F34%2F80%2Fphad-thai.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A628%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1200%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20115%204728%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum-Boca%20Paila%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22As%20was%20my%20tofu%20%EF%BF%B9green%20curry%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1144210-r1006758941-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1144210-r1006758941-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20the%20Tulum%20%EF%BF%B9salad%EF%BF%BB%2C%20tortillas%2C%20som%20tam...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1144210-r1027585849-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1144210-r1027585849-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A12091598%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurante%20Estrada%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.209045%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45844%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1566%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F30%2Ff9%2F26%2F4d%2Festrada.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1200%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1600%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20124%200011%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calle%20Centauro%20Sur%20Entre%20Gama%20Oriente%20y%20Asteroide%20Oriente%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Fantastic%20%EF%BF%B9seafood%EF%BF%BB%20restaurant%20on%20the%20way%20back%20from%20the%20cenotes%20-%20would%20highly...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d12091598-r980535510-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d12091598-r980535510-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20day%20was%20superb%20as%20was%20the%20%EF%BF%B9guacamole%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d12091598-r1041160852-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d12091598-r1041160852-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A27096664%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Sexy%20Pizza%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.212246%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.4588%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pizza%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pub%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beer%20restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A401%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2c%2F1a%2Ff1%2Fb0%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2096%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1179%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20177%207336%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277776%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22av%20Tulum%2016%20Av%20Principal%20de%20Tulum%2C%20Tulum%2077776%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Amazing!%20%20Friendly%20and%20yummy!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27096664-r1043153883-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27096664-r1043153883-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Marco%20was%20kind%20and%20Pizza%20delicious%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27096664-r1043039963-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27096664-r1043039963-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23597112%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mamazul%20Tulum%20Mezcaleria%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.199795%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46075%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fmamazul.short.gy%2F3PHxpA%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22RnRhXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjM3MjkwMDU2Jmdlbz0yMzU5NzExMiZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4NGp0MWRsRWE3Sk1KM2VYSGNrdDExNmVLMDNVUlFKcDY2TVdwSk53VWU1WSZjcz0xYWRkNjY5NWY1Yjk1YWQxNGRjNDgyMTMyMDY5NmRlZTdfNmVF%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A52%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.6%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F26%2Ff5%2F5a%2F6b%2Foctopus-with-mezcal-pairing.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A483%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A723%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20807%206724%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Aldea%20Zama%20Andador%20Kanbul%2C%20entre%20Andador%20Kaan%20y%20Calle%20Ixchel%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Best%20%EF%BF%B9tacos%EF%BF%BB%20ever%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23597112-r974966219-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23597112-r974966219-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Mezcal%20yes%20please%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23597112-r1038908163-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23597112-r1038908163-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%22755c7fbe-93b5-4e44-89c7-f2a48cb01121%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A27105003%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Lula's%20Kitchen%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.145996%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45929%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flulahoteltulum.com%2Feat-drink%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22NkRTXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU3MjEyMzk1Jmdlbz0yNzEwNTAwMyZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4aXR3NmhVNUsyczh4WUhPMW9oVVlpWWp6bkdHdDdJQU1qb2t0dmpfUmotcyZjcz0xYTdlYWEwOTU2ZDNkMTU5NzM2ZTkzODRiMGExZDJhM2FfVlZT%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A795%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2F22%2Ff4%2F4a%2Fterrace-view.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2199%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3298%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20210%203028%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20km%208.1%20Lula%20Seaside%20Boutique%20Hotel%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22I%20highly%20recommend%20the%20shrimp%20%EF%BF%B9tacos%EF%BF%BB!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27105003-r1036239399-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27105003-r1036239399-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9French%20toast%EF%BF%BB%2C%20bowls%2C%20and%20all%20of%20the...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27105003-r1007731489-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27105003-r1007731489-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23719230%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Distrito%20Panamera%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.212597%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.458786%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22b0xNXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMjAwNDkxJmdlbz0yMzcxOTIzMCZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4ZV9iaXphUnhqWXAtY3gwdnR1a0d0UHhxbElzdkNlaG52N1EwbGRrZDJwdyZjcz0xYzA2ZjA3NTA3YmE5ZDE4M2ZhZTE5MmM3NWFhOWYwM2ZfN2VV%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pizza%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pub%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A175%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2F4c%2F45%2F2a%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1536%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B529982277902%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%208.%205%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9room%EF%BF%BB%20was%20very%20comfortable%E2%80%94the%20bed%20was%20nice%20and%20firm%20and%20the%20air%20condition...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23719230-r1038375088-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23719230-r1038375088-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Distrito%20Awesome!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23719230-r1043395800-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23719230-r1043395800-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A20296875%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Negra%20Tomasa%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.211016%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.459915%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flanegratomasatulum.webnode.mx%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brew%20Pub%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beer%20restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1240%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1c%2F63%2F4d%2F59%2Fphoto0jpg.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2048%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20156%209555%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Andromeda%20Oriente%20%23%2014%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20surf-and-turf%20tostada%2C%20octopus%2C%20and%20fried%20%EF%BF%B9tuna%EF%BF%BB%20tacos%20were%20incredibly%20tend...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d20296875-r1041265282-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d20296875-r1041265282-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Beautiful%20place%20for%20family%20dinner%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d20296875-r1043402272-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d20296875-r1043402272-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10066419%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Popular%20-%20N%C3%B4made%20Temple%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.13606%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46304%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcutt.ly%2FgbJo5nL%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22bzVMXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjYzMTI4NDQwJmdlbz0xMDA2NjQxOSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4NHU3YWFsNnFvUjIwRjhZOWhCTTJlNF83bm5vRVI3cmdYdmg4cUwxVWkwTSZjcz0xMjFlZjcxZWZjOTY3MTA0ODI2ZDQwZTEwNjk5OGIzMTZfckNP%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A411%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fd8%2F44%2Fe1%2Fla-popular.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20745%200314%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277880%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%2010.5%20Tulum%20Beach%2C%20Tulum%2077880%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Excellent%20%EF%BF%B9seafood%EF%BF%BB%20by%20the%20sea%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10066419-r1001760708-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10066419-r1001760708-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9ceviche%EF%BF%BB%20was%20a%20highlight.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10066419-r1001883756-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10066419-r1001883756-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A21408449%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tantra%20Tulum%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.158932%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.455475%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tantrabeachclub.com%2Fen%2Fmenu%3Futm_source%3Dtripadvisor%26utm_medium%3Dorganic%26utm_campaign%3Dmenu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22c3FCXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU5MDk0MTI4Jmdlbz0yMTQwODQ0OSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4OTFaX0hXdFpBa2FNUzZCMjRZR1BXQjhjTXZBQjVIelNXMTVhUVNqM3VVOCZjcz0xMzhiNmQ2MjI4OTU0ZDFhM2I5YzQwYWI1NmY2YWUwZGVfWnE2%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1678%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2F28%2Fa4%2F11%2Ftantra-tulum.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1080%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1080%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200252%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20KM%207%20Tantra%20Bohemian%20Luxury%20Beach%20Club%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22%EF%BF%B9Anniversary%EF%BF%BB%20at%20tantra%20tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d21408449-r1023510138-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d21408449-r1023510138-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Tantra%20is%20a%20must%20visit!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d21408449-r1042412855-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d21408449-r1042412855-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10252068%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mi%20Amor%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.194103%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.43926%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tulumhotelmiamor.com%2Frestaurant-bar%2F%23menu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22RVNaXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTg5OTQ4Jmdlbz0xMDI1MjA2OCZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4Q3RvZWprNEdJVjFuWGtRV0dCdjR3NmF0c1hFNzBhN0VVYVpxY0g3ZUVrYyZjcz0xYzY1YWNiNWRhMmU2YTExYmEyMjdjMzRlNWIxOWJlYjZfVGpR%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A972%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F29%2Fc9%2Fd0%2Fad%2Frestaurant.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1492%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2880%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%201%20984%20188%204273%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km.%204.1%20Zona%20Hotelera%2C%20Parque%20Nacional%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Paradise%20best%20%EF%BF%B9guacamole%EF%BF%BB%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10252068-r876554607-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10252068-r876554607-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20best%20%EF%BF%B9cheesecake%EF%BF%BB%20dessert%20and%20amazing...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10252068-r1032238441-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10252068-r1032238441-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A24855625%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Alma%20Verde%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.203936%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45003%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.almaverde.com.mx%2Fcopia-de-copia-de-menu-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A348%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2Fef%2Fff%2Fc4%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1152%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20231%207473%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22M%202%20Av.%20Cob%C3%A1%20Sur%20Lote%203%20Plaza%20Hunab%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20ordered%20the%20%EF%BF%B9avocado%20toast%EF%BF%BB%20and%20the%20eggs%20Benedict%2C%20both%20were%20absolutely%20deli...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Amazing%20breakfast%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A15294346%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Casa%20Maria%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.195839%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.43832%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fqr.imenupro.com%2Fhod-32%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22b0pYXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTc1MzI5Jmdlbz0xNTI5NDM0NiZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4ZVNuQVpzTkFfTUs4d1JwSWcySnNvM2ZnRU1CQldqS0xjdEd4UHh4MVEtUSZjcz0xNDEyZDA5MWZkYTNkYzY4M2Q5Nzg4YWU2ZTliNmZkODJfNWlv%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1049%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2F6a%2Fea%2Fb8%2Ffish-tacos.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4024%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20876%202115%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Boca%20Paila%20Km%202.5%20Diamante%20Hotel%20%26%20Resturant%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20had%20grilled%20%EF%BF%B9octopus%EF%BF%BB%20and%20baja%20style%20fish%20tacos.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d15294346-r1010070146-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d15294346-r1010070146-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20and%20mixed%20%EF%BF%B9ceviche%EF%BF%BB%20Mexican%20style!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d15294346-r1013051733-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d15294346-r1013051733-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A27477578%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Piaggia%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.212048%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45109%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ftoptablegroup.com%2Fpiaggia-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22NG4zXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU3OTUzMzU4Jmdlbz0yNzQ3NzU3OCZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4b3FhYjdUaGtsRWN2cGRDQmpFTndUSUVoUWtUZzdJVnNuUzlJd1pZQjlvVSZjcz0xYzQ0OGU4NDg5MTAyNTFlNDQzNmJhNjU2YzY4NjVlMmZfUXRC%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Italian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A340%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2Fe0%2F3a%2F6c%2Fpiaggia-tulum-is-an-italo.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A3000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20310%207074%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calla%20Gamma%20Ote%20Mz%204%20Lote%201%2C%20Local%208a%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20had%20the%20a%C3%A7a%C3%AD%20bowl%20and%20the%20%EF%BF%B9French%20toast%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27477578-r1018922684-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27477578-r1018922684-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Best%20pizza%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27477578-r1042532282-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27477578-r1042532282-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10434099%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Fresco's%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.156944%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45616%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Fwww.frescostulum.com%2Fmenus%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22NnNLXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTg4MDI1Jmdlbz0xMDQzNDA5OSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4WlJzcFV6S3VBOXF4Z2JKQ1BPLU9aRXQ2MGZlVERqaFF2RmFDcU1GWVdLQSZjcz0xMTY2NWFjNWQ3OTVjNzIwNzE1YWI1ZmQyNmUwNWE3ZGRfSGl5%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1101%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2f%2F93%2F34%2F2d%2Ffresco-s-restaurant-entrnace.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1280%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1920%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20871%201141%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum-Boca%20Paila%20km%207%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22AMAZING!!%20Best%20breakfast%20in%20tulum!%20This%20place%20is%20so%20cute%20and%20the%20service%20is...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d10434099-r789410173-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d10434099-r789410173-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%2C%20fresh%20and%20DELICIOUS!%20Immaculate%20vibes%2C%20impeccable%20service%2C%20and%20SO%20SO%20SO%20FRESH!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d10434099-r1042265342-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d10434099-r1042265342-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23432753%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Parallel%2020%C2%B0%20by%20Kimpton%20Aluna%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.201447%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45102%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22emhLXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTkzNzQwJmdlbz0yMzQzMjc1MyZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4bmg1c2pIVHkybWNydl82dmV5aXpWa01nNkNyem5JbGJwVzF4a1ZzanBtTSZjcz0xYmE3MDQ5NzNiNTgwZThjYzQ0NTU1MDEwYTlhNGMyNjVfZW9I%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fusion%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20Bar%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A90%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1d%2F1f%2F0e%2F98%2Fparallel-20.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1972%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2732%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20689%200430%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Av%20Itzamna%20Poligono%200014%20Lote%201-11%2C%20Aldea%20Zama%20Hotel%20Kimpton%20Aluna%20Tulum%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22He%20even%20prepared%20something%20out%20of%20menu%20for%20me!%20Alex%20in%20the%20morning%20buffet%20is...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23432753-r791998099-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23432753-r791998099-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20I%20think%20said%20he%20was%20from%20%EF%BF%B9Mexico%20City%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23432753-r875671659-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23432753-r875671659-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%22e8346626-9505-453a-95ea-f101695a2223%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23652674%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Onyx%20Tulum%20Restaurant%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.211458%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.460495%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A728%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1e%2F5b%2Fdf%2F3c%2Fview-of-the-restaurant.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4032%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3024%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20209%201788%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calle%20Ori%C3%B3n%20Sur%20M4%20L-009%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9ceviche%EF%BF%BB%20and%20mushroom%20tacos%20especially%20were%20great%2C%20and%20the%20Onyx%20Mescalita...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23652674-r1011720733-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23652674-r1011720733-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20got%20the%20%EF%BF%B9guacamole%EF%BF%BB%20for%20an%20app%2C%20steak...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23652674-r1014804001-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23652674-r1014804001-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10169568%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22El%20Suspiro%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.214872%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45359%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A163%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F17%2F8f%2F84%2Fed%2Fnuestro-original-ceviche.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A3744%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A5616%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20279%201370%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Avenida%20Coba%2C%20Mz%2036%2C%20lote%204%20y%205%20Mz%2036%2C%20Lote%204%20y%205%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Alejandro%20was%20our%20Server%20with%20immediate%20attention%20and%20very%20considerable.!!!%20Our...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10169568-r769759628-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10169568-r769759628-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20pescado%20a%20%EF%BF%B9la%20parrilla%EF%BF%BB%20y%20el%20ceviche%3A...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10169568-r1031485099-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10169568-r1031485099-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A26265692%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Golden%20Cham%20Thai%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.209072%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46888%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soups%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20today%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A321%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2F02%2Fff%2Fdb%2Fit-s-a-great-place-to.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A960%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1280%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20120%203206%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Av.%20Tulum%20Entre%20Saturno%20y%20Luna%20Sur%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20had%20shrimp%20steamed%20%EF%BF%B9dumplings%EF%BF%BB%20%26%20Thai...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26265692-r1024085297-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26265692-r1024085297-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Fabulous%20in%20all%20ways!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26265692-r1033578153-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26265692-r1033578153-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23180563%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22%C3%81tico%20Rooftop%20Bar%20%26%20Lounge%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.205343%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.450386%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gastropub%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A158%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1c%2Fd9%2Ff1%2F51%2Finfinity-pool-at-atico.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1333%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20689%200506%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Mza%2005%20Av.%20Coba%20Lot%2001%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20restaurant%20and%20rooftop%20staff%20were%20also%20fabulous%20and%20great%20to%20chat%20with...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23180563-r792599062-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23180563-r792599062-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Great%20Service%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23180563-r892064524-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23180563-r892064524-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A13201059%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Loco%20Tulum%20Mediterranean%20Kitchen%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.137993%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46335%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Flocotulum.com%2Fmenu%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22UEt4Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTgzNjc3Jmdlbz0xMzIwMTA1OSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4Z3JQSHkyTXRKRVluX3REaVlHeVJjUVpiMDh3VW55YjFGWWJPdEJPb3ZqSSZjcz0xNjU0NDg2ODgxMDY1YjI0ZTNjMmRmYTY0MTFkM2QxZmJfaWpi%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Middle%20Eastern%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A960%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1a%2Fb3%2F9e%2Ff9%2Flocos-at-loco-tulum-km.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1400%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1400%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20309%200149%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum%20Boca%20Paila%20Km%209.3%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20enjoyed%20the%20pita%20%26%20%EF%BF%B9hummus%EF%BF%BB%2C%20tuna%20tataki%2C%20shrimp%20tacos%2C%20and%20Caesar%20salad%E2%80%94eve...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d13201059-r1003566357-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d13201059-r1003566357-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22I%E2%80%99m%20very%20speechless%20how%20was%20the%20service...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d13201059-r805223610-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d13201059-r805223610-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23780639%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Rosanegra%20Beach%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.151724%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45831%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Frosanegrabeachclub.com%2Fen%2Fbest-lunch-in-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22Vmp0Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTgxMzAzJmdlbz0yMzc4MDYzOSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4UUh4UjZsWTJpRkpkb3BSU2J6WXIzZVBYdGFaY2hjaVlxRmt6MmpFb0c3USZjcz0xY2I3ZTFlNTgwOTZhZDNkMTJhYmExNWQxMTU2YzlkNmVfWndw%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20today%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A238%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F22%2F0c%2F1b%2F38%2Fbest-beach-club-in-town.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A3283%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A4925%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200283%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-boca%20Paila%205-km%207%2C%20Tulum%20Beach%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22A%20perfect%20day%20at%20the%20%EF%BF%B9tulum%20beach%EF%BF%BB%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d23780639-r972205014-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d23780639-r972205014-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22GREAT!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d23780639-r1014112900-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d23780639-r1014112900-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23459171%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Cervecer%C3%ADa%20Tulum%20Brewery%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.232265%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.42345%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ftulumcerveceriaartesanal.com%2Fmenu-tulum%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22ZmlxXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTgzNzI3Jmdlbz0yMzQ1OTE3MSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4SHU3OTQ1TzdLc3FZaEptZE1aY0FhcFhRN01wT09jMnB0LXRjdTVHRDNSbyZjcz0xYTZmMjIzODJkZmQ5MDA5MGE5YTUwYWU3M2JlMzQ2YmFfZXNI%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1493%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2F6c%2Fee%2F98%2Fla-mejor-cocina-mexicana.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1400%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1500%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20218%200506%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277789%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Canc%C3%BAn%20-%20Chetumal%20Km.%20230%20Parque%20Nacional%20Del%20Jaguar%20De%20Tulum%2C%20Tulum%2077789%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20lager%20is%20delicious%20and%20the%20%EF%BF%B9tuna%EF%BF%BB%20tiradito%20is%20killer.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23459171-r988921293-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23459171-r988921293-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Good%20dj%2C%20nice%20ambiance%20and%20decoration...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23459171-r800587562-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23459171-r800587562-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23524261%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Bio-natural%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.204794%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45029%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fbio-natural.mx%2Fen%2Fmenu%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soups%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fruit%20parlours%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Medicinal%20foods%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A305%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F23%2Fde%2F72%2Fc3%2Fbio-natural-tulum.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4032%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3024%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20168%206138%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Av%20Coba%20Plaza%20Mayan%20Monkey%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Vegan%20friendly!%20Thanks%20to%20Harry%20and%20Antonio%20great%20people%20and%20very%20friendly%20%E2%98%BA%EF%B8%8F%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23524261-r800511136-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23524261-r800511136-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%225%20star%20for%20Bio%20natural%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23524261-r901154099-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23524261-r901154099-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23280755%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Confessions%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.159094%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.4552%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.confessions.com.mx%2Fmixologia-bares-en-cancun-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22bnFyXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU5MDk0MDk4Jmdlbz0yMzI4MDc1NSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4OC04ampyR3A2V0dRdVFGRW9EWW5HZVR1aHJGcTVTN0dzRFZvMEhwTnh6ZyZjcz0xODVhZjA5MTVhYTY3M2MwODhkZWNmNGY0Mjk4MGI1ZjFfS1Bl%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A11776%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bars%20%26%20Pubs%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20today%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A997%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F23%2F3f%2Fc7%2Fe1%2Fconfessions.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1080%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1080%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200275%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum-%20Boca%20Paila%20Zona%20Hotelera%2C%20Km.%207%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Hands%20down%20best%20night%20scene%20in%20tulum%20ask%20for%20them!!%20Confessions%20is%20an%20amazing...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23280755-r794483951-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23280755-r794483951-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Long%20and%20large%20%EF%BF%B9bar%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23280755-r996936626-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23280755-r996936626-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A19458762%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Curry%20Thai%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.202757%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.47367%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A732%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2F8b%2F3c%2F58%2Fstaff.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1200%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1600%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20239%203265%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22C.%207%20Sur%20La%20Veleta%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Great%20%EF%BF%B9Thai%EF%BF%BB%20food%20in%20Tulum%20and%20Fermin%20was%20a%20fantastic%20waiter%2C%20super%20attentive%2C...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19458762-r923747319-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19458762-r923747319-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%E2%80%99ve%20stayed%20in%20tulum%20now%20for%20a%20few...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19458762-r783269823-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19458762-r783269823-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A26128325%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Copal%20Gastro%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.20358%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46014%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcopalgastrobar.com%2Fwp-content%2Fuploads%2F2023%2F09%2FSPRECIO-COPAL-MENU-INGLES.pdf%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22dGt4Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU0NzA4MjQ3Jmdlbz0yNjEyODMyNSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjUxNzcyOSZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4NmdsaHY3a29YVS11N2Z1OTJiMm9FMWhJbWVoVlVKTGZfOTNhQU9teXNLUSZjcz0xNGRlOTUxOGRlNzJmYzM0YmFmZGZkNGU3ODAwYjMzMmVfbklr%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A663%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F29%2F7c%2F05%2F1c%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20181%206970%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calle%20Ixchel%20esquina%20Av.%20Juanek%2C%20Aldea%20Zama%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Fantastic%20fish%20%EF%BF%B9tacos%EF%BF%BB%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26128325-r954595371-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26128325-r954595371-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Must%20Visit%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26128325-r1043408151-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26128325-r1043408151-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A1775007%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Ziggy%20Beach%20Club%20%26%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.156145%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45642%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.ziggybeachtulum.com%2Fmenu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22cXdsXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTcwMDIyJmdlbz0xNzc1MDA3JmZyb209UmVzdGF1cmFudHMmYXJlYT1yZXNlcnZhdGlvbl9idXR0b24mc2xvdD0xJm1hdGNoSUQ9MSZvb3M9MCZjbnQ9MSZzaWxvPTQ1NTgyJmJ1Y2tldD05Nzk2NDEmbnJhbms9MSZjcmFuaz0xJmNsdD1SJnR0eXBlPVJlc3RhdXJhbnQmdG09MzQ2NTE3NzI5Jm1hbmFnZWQ9ZmFsc2UmY2FwcGVkPWZhbHNlJmdvc294PTlETXZpYkRUV2ROYUZlR1lMOFFUYWt5djFMQllaYUdwZUI0TUVIbi1odjJtWkJOMzc2RUhWQ0d1c1RJUnhtSjhhWWlZaWZHOGExLWhsaEt6MTNuc1NSN3BLVk1UT3FIREdMUXJuaVhERWtrJmNzPTFiNTI3YmRjYzM4OTkwM2VmMWE5ZjhjYjc1MDA2N2FmMF8wUE8%3D%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A2186%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2F1e%2F3d%2F74%2Fbeach-club.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2250%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A4000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20871%201145%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20km%207.5%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Stayed%20at%20the%20beach%20tulum%20and%20ate%20at%20ziggy%E2%80%99s%20for%20lunch%20and%20dinner%20many%20times...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1775007-r785116688-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1775007-r785116688-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22For%20the%20main%20dish%20had%20the%20%EF%BF%B9octopus%EF%BF%BB%2C%20grea...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1775007-r962472267-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1775007-r962472267-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23259804%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mamazzita%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.15775%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45616%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Fbit.ly%2F2PV0Uln%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fusion%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A11776%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bars%20%26%20Pubs%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1184%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F22%2F33%2Fb4%2F1c%2Fmain-entrance-to-mamazzita.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A720%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1280%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200274%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum%20Boca%20Paila%2C%20Km.%207%20Zona%20Hotelera%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Great%20service%2C%20Carlos%20was%20our%20server.%20The%20ambiance%20and%20atmosphere%20is%20really...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23259804-r790949867-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23259804-r790949867-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20wavy%20%EF%BF%B9meatballs%EF%BF%BB%2C%20prime%20rib%20tacos%20%26%20s...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23259804-r954878154-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23259804-r954878154-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A19855557%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Las%20Hijas%20De%20La%20Tostada%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.211218%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.461555%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flas-hijas-de-la-tostada-4.ola.click%2Fproducts%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Food%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A178%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fa4%2F5d%2F94%2Fshrimp-burger.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A820%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1200%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20235%200690%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Centro%2C%20Col%20Huracanes%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Not%20too%20sweet%20and%20quality%20ingredients.%20I%20had%20a%20hibiscus%2Fmescal%20drink%20and%20a...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19855557-r737833923-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19855557-r737833923-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20and%20salmon%20%EF%BF%B9tostada%EF%BF%BB%20and%20both%20were%20su...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19855557-r1036251845-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19855557-r1036251845-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A21248033%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Funky%20Geisha%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.149506%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.4594%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.funkygeisha.com.mx%2Fen%2Fmenu-comida-asiatica-thai-japonesa%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Japanese%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1457%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1c%2F61%2F87%2Fd9%2Flounge.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A755%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A986%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200255%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%207.8%20Tulum%20Beach%20-%20Hotel%20Zone%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20the%20Panang%20curry%20and%20all%20of%20our%20%EF%BF%B9sushi%EF%BF%BB%20(order%20the%20kimchi%20%EF%BF%B9sushi%EF%BF%BB!)%20was%20delic...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d21248033-r984875421-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d21248033-r984875421-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22A%20great%20restaurant%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d21248033-r1038637138-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d21248033-r1038637138-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%5D%2C%5C%5C%5C%22facets%5C%5C%5C%22%3A%7B%5C%5C%5C%22onlineOptions%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Online%20options%20%26%20offers%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220693%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A531%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Online%20Reservations%20%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20693%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Cuisines%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225110%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A225%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5110%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210643%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A123%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10643%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210640%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A73%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10640%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210679%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A68%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10679%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210648%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A64%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10648%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210639%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A59%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10639%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210642%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A51%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10642%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%224617%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A47%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Italian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A4617%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210649%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A36%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10649%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210622%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10622%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210641%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pizza%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10641%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210671%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A29%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fusion%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10671%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210670%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A28%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pub%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10670%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210668%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A26%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Grill%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10668%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210655%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A25%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Spanish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10655%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229908%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A25%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22American%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9908%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210669%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A20%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10669%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210749%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22South%20American%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10749%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210651%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A17%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Barbecue%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10651%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210659%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10659%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210345%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Steakhouse%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10345%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210760%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Central%20American%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10760%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225473%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A14%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Japanese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5473%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210653%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10653%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210698%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A12%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Argentinean%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10698%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210682%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10682%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225086%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22French%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5086%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210646%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fast%20Food%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10646%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210654%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22European%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10654%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221355%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beer%20restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21355%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210660%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10660%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221353%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dining%20bars%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21353%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210621%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A6%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brew%20Pub%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10621%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210687%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A6%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Middle%20Eastern%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10687%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221343%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A5%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fruit%20parlours%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21343%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210631%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Peruvian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10631%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210683%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gastropub%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10683%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221367%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Japanese%20Fusion%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21367%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210346%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Indian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10346%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210700%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soups%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10700%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220076%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Southern-Italian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20076%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210663%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Turkish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10663%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210699%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Canadian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10699%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210772%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hawaiian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10772%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220062%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Neapolitan%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20062%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220067%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Romana%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20067%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220068%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lazio%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20068%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220069%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sicilian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20069%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220075%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Central-Italian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20075%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221350%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Medicinal%20foods%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21350%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210348%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brazilian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10348%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210633%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Moroccan%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10633%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210666%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Deli%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10666%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210675%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vietnamese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10675%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210676%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Diner%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10676%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210680%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Portuguese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10680%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210686%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Food%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10686%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210696%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Taiwanese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10696%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211739%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Central%20Asian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11739%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220064%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Campania%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20064%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220077%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Catalan%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20077%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225379%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chinese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5379%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22diets%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Dietary%20restrictions%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210665%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A160%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A285%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vegetarian%20friendly%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10665%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210697%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A138%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A285%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vegan%20options%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10697%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210992%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A94%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A285%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gluten%20free%20options%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10992%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22diningOptions%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Features%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210852%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A308%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seating%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10852%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216547%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A305%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Table%20Service%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16547%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210602%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A250%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Reservations%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10602%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210862%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A248%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Serves%20Alcohol%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10862%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210603%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A231%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Outdoor%20Seating%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10603%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211780%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A211%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Accepts%20Credit%20Cards%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11780%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210870%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A210%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Free%20Wifi%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10870%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210863%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A191%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Full%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10863%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210601%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A160%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Takeout%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10601%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210864%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A129%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20and%20Beer%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10864%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210854%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A101%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Parking%20Available%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10854%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210861%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A99%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wheelchair%20Accessible%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10861%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210867%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A98%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Visa%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10867%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210866%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A94%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mastercard%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10866%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220996%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A93%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dog%20Friendly%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20996%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210855%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A82%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10855%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210860%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A80%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Highchairs%20Available%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10860%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220989%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A76%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Live%20Music%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20989%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210865%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A73%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22American%20Express%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10865%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221271%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A69%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Family%20style%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21271%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210600%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A62%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Delivery%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10600%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210868%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A43%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Digital%20Payments%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10868%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210856%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A40%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Validated%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10856%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210858%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A40%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Free%20off-street%20parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10858%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220990%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A37%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Waterfront%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20990%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210859%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A36%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Television%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10859%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220993%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beach%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20993%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210702%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A24%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Private%20Dining%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10702%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210869%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cash%20Only%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10869%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210871%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A21%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Discover%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10871%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221379%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Non-smoking%20restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21379%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221907%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gift%20Cards%20Available%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21907%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220992%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A14%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Drive%20Thru%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20992%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210857%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Valet%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10857%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220995%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Playgrounds%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20995%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221381%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sports%20bars%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21381%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220994%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A5%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22BYOB%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20994%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210612%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Buffet%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10612%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220991%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Jazz%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20991%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22dishes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Dishes%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220515%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A197%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cocido%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20515%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2219954%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A121%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tacos%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A19954%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222444%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A115%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beef%20Tacos%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22444%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210937%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A96%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Shrimp%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10937%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221324%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A96%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21324%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223001%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A96%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Jumbo%20Shrimp%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23001%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216554%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A89%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Salad%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16554%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210883%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A83%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Ceviche%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10883%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223145%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A71%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Guacamole%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23145%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222950%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A65%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Desserts%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22950%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221354%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A63%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20dishes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21354%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220336%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A54%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Steak%20Frites%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20336%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220711%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A52%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Octopus%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20711%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222620%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A50%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mixed%20Seafood%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22620%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222390%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A49%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Burger%20and%20Fries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22390%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221239%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A48%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pesto%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21239%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222870%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A48%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fresh%20Pasta%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22870%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210907%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A47%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Burger%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10907%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220552%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A45%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tuna%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20552%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210678%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A44%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pasta%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10678%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222992%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A39%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mole%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22992%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216553%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A38%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fish%20Taco%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16553%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220487%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cecina%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20487%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223109%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cake%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23109%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220752%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A34%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beef%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20752%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223203%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A34%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fried%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23203%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222568%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soup%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22568%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229911%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Juice%20%26%20Smoothies%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9911%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210915%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A32%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lobster%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10915%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222388%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A32%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Breads%20and%20Pastries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22388%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222293%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20Quesadilla%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22293%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223103%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Quesadilla%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23103%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223285%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Quesadillas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23285%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220031%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A29%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Crawfish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20031%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220041%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A28%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Nachos%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20041%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222299%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A28%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Rice%20dishes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22299%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210944%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A27%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tortillas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10944%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210685%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A25%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20Wings%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10685%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220034%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A24%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fajitas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20034%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222879%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A23%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tostadas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22879%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222901%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A23%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tostada%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22901%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220730%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Toasts%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20730%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221326%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pork%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21326%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222890%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Toasties%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22890%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210647%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sandwiches%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10647%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210878%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Burrito%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10878%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220175%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Veggie%20Burrito%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20175%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220547%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Salmon%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20547%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210932%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Ribs%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10932%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220697%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Foie%20gras%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20697%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223357%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Potato%20Wedges%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23357%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220148%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A17%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hoagie%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20148%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222521%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mango%20dishes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22521%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223331%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beef%20Ribs%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23331%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222305%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Enchiladas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22305%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222628%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20Enchiladas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22628%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221275%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A14%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cakes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21275%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221174%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lamb%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21174%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222438%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22White%20Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22438%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222573%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Coconut%20Shrimp%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22573%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222584%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Avocado%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22584%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210902%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A12%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22French%20Toast%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10902%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220029%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chili%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20029%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221338%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Garlic%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21338%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222280%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chili%20Con%20Carne%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22280%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222572%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%20Rolls%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22572%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222651%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Snapper%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22651%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222775%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Red%20Snapper%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22775%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223089%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chilaquiles%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23089%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210921%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Omelette%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10921%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216555%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pancakes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16555%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220027%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Calamari%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20027%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222386%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Grilled%20Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22386%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222867%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Patties%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22867%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223260%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fresh%20Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23260%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223368%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fish%20Fillet%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23368%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210924%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Paella%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10924%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220181%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Curry%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20181%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220312%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Risotto%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20312%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220475%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Paella%20Valenciana%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20475%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220754%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hamburgers%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20754%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221234%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Milanese%20risotto%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21234%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222275%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Avocado%20Toast%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22275%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210923%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pad%20Thai%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10923%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210939%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Steak%20Tartare%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10939%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220699%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Prawns%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20699%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221278%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chocolates%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21278%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221301%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Parfait%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21301%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222743%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Empanadas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22743%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223091%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Empanada%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23091%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223125%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vegetable%20Curry%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23125%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210893%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Crab%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10893%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220347%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fettuccine%20Alfredo%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20347%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220483%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Eggplant%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20483%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221037%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hanger%20steak%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21037%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221175%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Antipasti%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21175%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222319%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mixed%20Grill%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22319%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222431%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tartare%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22431%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223311%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pastries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23311%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2226531%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Huevos%20Rancheros%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A26531%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Establishment%20type%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210591%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A379%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10591%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211776%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A12%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bars%20%26%20Pubs%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11776%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229900%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Coffee%20%26%20Tea%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9900%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229909%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A6%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dessert%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9909%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216556%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Quick%20Bites%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16556%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229901%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bakeries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9901%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221908%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Delivery%20Only%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21908%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22mealTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Meal%20type%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210597%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A181%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Breakfast%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10597%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210606%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A122%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brunch%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10606%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210598%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A306%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lunch%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10598%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210599%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A308%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dinner%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10599%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22michelinAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22MICHELIN%20Guide%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221964%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A645%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22MICHELIN%20Guide%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21964%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221969%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A645%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22MICHELIN%20Bib%20Gourmand%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21969%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22minimumTravelerRating%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Traveler%20rating%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELER_RATING_LOW%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%223%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A380%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELER_RATING_MEDIUM%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%224%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A338%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELER_RATING_HIGH%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%225%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A64%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%5D%7D%2C%5C%5C%5C%22restaurantOwnerTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Diversity%20attributes%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22neighbourhoods%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Neighborhood%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22waypoints%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Airports%20and%20theme%20parks%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22travelersChoice%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Awards%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22coe%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22Travelers%E2%80%99%20Choice%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22openNow%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22OPEN_NOW%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Price%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210953%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A197%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A240%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cheap%20Eats%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10953%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210955%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A397%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A240%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mid-range%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10955%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210954%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%24%24%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A80%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A240%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fine%20Dining%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10954%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%5D%7D%2C%5C%5C%5C%22styles%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Great%20for%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210604%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A186%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Families%20with%20children%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10604%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210614%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A132%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Romantic%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10614%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211777%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A117%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Kids%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11777%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210607%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A82%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Special%20occasions%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10607%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210609%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A81%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Large%20groups%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10609%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210613%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A79%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Local%20cuisine%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10613%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210608%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A72%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%20scene%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10608%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210605%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A46%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Business%20meetings%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10605%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210610%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A45%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Scenic%20view%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10610%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216550%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A27%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hidden%20Gems%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16550%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2212504%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A23%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hot%20New%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A12504%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%7D%2C%5C%5C%5C%22banner%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22metadata%5C%5C%5C%22%3A%7B%5C%5C%5C%22nextCursor%5C%5C%5C%22%3A%5C%5C%5C%22eyJzdGFydF9hZnRlciI6WyI0LjQ0MDgyMSIsIjQuNzEyMTIzMzcyIiwiZjM0MzVlNWItOTJlZS00ZjQzLWI2MzYtNjU5ZTJmOTQ1M2VlIl0sInJhY19vZmZzZXQiOm51bGwsIm1heF9yYWNfb2Zmc2V0IjpudWxsLCJvZmZzZXQiOjAsImxpbWl0IjozMH0%5C%5C%5C%22%2C%5C%5C%5C%22total_hits%5C%5C%5C%22%3A397%2C%5C%5C%5C%22offset%5C%5C%5C%22%3A0%7D%2C%5C%5C%5C%22racMetadata%5C%5C%5C%22%3A%7B%5C%5C%5C%22requestToken%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22defaultDatetime%5C%5C%5C%22%3A%5C%5C%5C%222025-12-24T19%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22defaultPartySize%5C%5C%5C%22%3A2%2C%5C%5C%5C%22currentDatetime%5C%5C%5C%22%3A%5C%5C%5C%222025-12-24T14%3A55%3A29.535783%5C%5C%5C%22%2C%5C%5C%5C%22enabled%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22sorts%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22POPULARITY%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Highest%20Rating%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Highest%20rated%20restaurants%20on%20Tripadvisor%2C%20based%20on%20reviews.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22RELEVANCE%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Relevance%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20ranked%20by%20how%20well%20they%20match%20your%20selections%20and%20traveler%20reviews.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22AVAILABILITY%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Availability%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20available%20for%20your%20date%20appear%20first%2C%20ranked%20by%20ratings%20and%20reviews.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22FEATURED%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Featured%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20ranked%20according%20to%20page%20views%2C%20reviews%20and%20individual%20attributes%20such%20as%20price%20range%2C%20cuisine%20and%20location%2C%20as%20well%20as%20aggregated%20Tripadvisor%20data%20comparing%20user%20interactions%20with%20all%20the%20other%20restaurant%20pages%20for%20this%20locale.%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22searchCenter%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22geoBroadenStatus%5C%5C%5C%22%3A%7B%5C%5C%5C%22is_broadened%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22cta_message%5C%5C%5C%22%3A%7B%5C%5C%5C%22desktopText%5C%5C%5C%22%3A%5C%5C%5C%22Looking%20to%20expand%20your%20search%20outside%20of%20Tulum%3F%20We%20have%20suggestions.%5C%5C%5C%22%2C%5C%5C%5C%22mobileText%5C%5C%5C%22%3A%5C%5C%5C%22We%20found%20great%20results%20outside%20of%20Tulum.%5C%5C%5C%22%7D%2C%5C%5C%5C%22action_message%5C%5C%5C%22%3A%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Expand%20your%20search.%5C%5C%5C%22%7D%7D%7D%7D%5C%22%7D%2C%5C%224252927822%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22breadcrumbsData%5C%5C%5C%22%3A%7B%5C%5C%5C%22breadcrumbs%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150768-Mexico-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Yucatan%20Peninsula%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150805-Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Quintana%20Roo%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g1575485-Quintana_Roo_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Riviera%20Maya%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g659488-Riviera_Maya_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%5D%2C%5C%5C%5C%22usePageHeader%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22additionalTrails%5C%5C%5C%22%3A%5B%5D%7D%7D%5C%22%7D%2C%5C%22100483659892248%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Theme_getTheme%5C%5C%5C%22%3Anull%7D%5C%22%7D%2C%5C%2216178272766%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Opf_getOnPageFactorsForLocale%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22errMessage%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22factors%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22META_DESCRIPTION%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Moderately%20Priced%20Restaurants%20in%20Tulum%2C%20Riviera%20Maya%3A%20Find%20Tripadvisor%20traveler%20reviews%20of%20THE%20BEST%20Tulum%20Mid%20Range%20Restaurants%20and%20search%20by%20price%2C%20location%2C%20and%20more.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22MASTHEAD_H1%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Moderately%20Priced%20Restaurants%20in%20Tulum%2C%20Riviera%20Maya%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22TITLE%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22THE%2010%20BEST%20Moderately-Priced%20Restaurants%20in%20Tulum%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22MAIN_H1%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Mid%20Range%20Restaurants%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22IS_INDEXABLE%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22true%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-2754358323%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AdMissionControl_getAdSlotsListForPageAndPlatform%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22slots%5C%5C%5C%22%3A%5B%5C%5C%5C%22InlineNova%5C%5C%5C%22%2C%5C%5C%5C%22footer%5C%5C%5C%22%2C%5C%5C%5C%22rail3%5C%5C%5C%22%2C%5C%5C%5C%22rail2%5C%5C%5C%22%2C%5C%5C%5C%22horizon%5C%5C%5C%22%2C%5C%5C%5C%22inline1%5C%5C%5C%22%2C%5C%5C%5C%22inline2%5C%5C%5C%22%2C%5C%5C%5C%22inline3%5C%5C%5C%22%2C%5C%5C%5C%22rail1%5C%5C%5C%22%2C%5C%5C%5C%22inline4%5C%5C%5C%22%2C%5C%5C%5C%22inline5%5C%5C%5C%22%2C%5C%5C%5C%22inline6%5C%5C%5C%22%2C%5C%5C%5C%22inline7%5C%5C%5C%22%2C%5C%5C%5C%22inline8%5C%5C%5C%22%5D%7D%5D%7D%5C%22%7D%2C%5C%227470346746%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AdMissionControl_getPageSlotSettings%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22slots%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22rail1%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22rail2%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline1%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22rail3%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22footer%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline7%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline6%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22InlineNova%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline8%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline3%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline2%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline5%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline4%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22horizon%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-2154554899%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22locations%5C%5C%5C%22%3A%7B%5C%5C%5C%22detail%5C%5C%5C%22%3A%7B%5C%5C%5C%22info%5C%5C%5C%22%3A%7B%5C%5C%5C%22countryId%5C%5C%5C%22%3A150768%7D%7D%7D%7D%5C%22%7D%2C%5C%2211562210216%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22gptInfo%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22adBase%5C%5C%5C%22%3A%5C%5C%5C%22%2F5349%2Fta.ta.br.s%2Fna.mex.yucatan_peninsula.tulum%5C%5C%5C%22%2C%5C%5C%5C%22ppid%5C%5C%5C%22%3A%5C%5C%5C%22RISDC7I7N743UT7HYSHPZCMEOJKOBGPEDJ6OA7DI57RIISBSXX4Q%5C%5C%5C%22%2C%5C%5C%5C%22pageLevelTargeting%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22aud_id%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%2217403%5C%5C%5C%22%2C%5C%5C%5C%2217874%5C%5C%5C%22%2C%5C%5C%5C%2217736%5C%5C%5C%22%2C%5C%5C%5C%2217384%5C%5C%5C%22%2C%5C%5C%5C%2217875%5C%5C%5C%22%2C%5C%5C%5C%2215434%5C%5C%5C%22%2C%5C%5C%5C%2217765%5C%5C%5C%22%2C%5C%5C%5C%2216856%5C%5C%5C%22%2C%5C%5C%5C%2215440%5C%5C%5C%22%2C%5C%5C%5C%2217323%5C%5C%5C%22%2C%5C%5C%5C%2216860%5C%5C%5C%22%2C%5C%5C%5C%2217307%5C%5C%5C%22%2C%5C%5C%5C%2220048%5C%5C%5C%22%2C%5C%5C%5C%2250011%5C%5C%5C%22%2C%5C%5C%5C%2216062%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22browser%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22chrome%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22country%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22150768%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22dest%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22beach%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22detail%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%220%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22geo%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22150813%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22geo_ctx%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%2211903%5C%5C%5C%22%2C%5C%5C%5C%2211912%5C%5C%5C%22%2C%5C%5C%5C%2212024%5C%5C%5C%22%2C%5C%5C%5C%2212028%5C%5C%5C%22%2C%5C%5C%5C%2212035%5C%5C%5C%22%2C%5C%5C%5C%2212067%5C%5C%5C%22%2C%5C%5C%5C%2212077%5C%5C%5C%22%2C%5C%5C%5C%2221827%5C%5C%5C%22%2C%5C%5C%5C%2221828%5C%5C%5C%22%2C%5C%5C%5C%2221840%5C%5C%5C%22%2C%5C%5C%5C%2221852%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22loctype%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22restaurants%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22PageType%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22Restaurants%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22platform%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22desktop%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22premium_category%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22modalhorizon%3Apremium%2Fhorizon%3Apremium%2Frail3%3Apremium%2Frail1%3Apremium%2Finline8%3Apremium%2Finline6%3Asufficient%2Finline7%3Asufficient%2Finlinenova%3Asufficient%2Finline1%3Asufficient%2Finline2%3Asufficient%2Ffooter%3Asufficient%2Finline3%3Apremium%2Finline4%3Apremium%2Finline5%3Apremium%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22rd%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22com%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22region%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22150813%5C%5C%5C%22%2C%5C%5C%5C%22659488%5C%5C%5C%22%2C%5C%5C%5C%221575485%5C%5C%5C%22%2C%5C%5C%5C%22150805%5C%5C%5C%22%5D%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-12449012372%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22ad_refresh_tweak_1677267844%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22control%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-9948974379%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22trips_rebrand_on_web_1753200458%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3Anull%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-2632168235%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22standalone_tips_1744116322%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22global_nav_cta_simplification_1764622874%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-612125067%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22typeahead___nested_results__icons___hard_code_list__1723050108%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3Anull%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-22033722027%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22core_combined_login_modern_ui_poc_modal_1720186558%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-1389843205%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22detail%5C%5C%5C%22%3A%7B%5C%5C%5C%22info%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22rateLimiting%5C%5C%5C%22%3A%7B%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22OK%5C%5C%5C%22%7D%7D%7D%7D%5C%22%7D%2C%5C%22-18230052403%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22routes%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Tourism%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22a2k4Xy9Ub3VyaXNtLWcxNTA4MTMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEtVmFjYXRpb25zLmh0bWxfSFRU%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22HotelsFusion%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22YmZPXy9Ib3RlbHMtZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS1Ib3RlbHMuaHRtbF9wT3Q%3D%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22AttractionsFusion%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22MENKXy9BdHRyYWN0aW9ucy1nMTUwODEzLUFjdGl2aXRpZXMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEuaHRtbF9PUzY%3D%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22cXFaXy9SZXN0YXVyYW50cy1nMTUwODEzLVR1bHVtX1l1Y2F0YW5fUGVuaW5zdWxhLmh0bWxfODBI%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Cruises%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22VGxZXy9DcnVpc2VzLWcxNTA4MTMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEtQ3J1aXNlc194Tks%3D%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22ShowForum%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22WTJmXy9TaG93Rm9ydW0tZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS5odG1sXzMzVA%3D%3D%5C%5C%5C%22%7D%5D%7D%5C%22%7D%2C%5C%228843354577%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22region%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22route%5C%5C%5C%22%3Anull%7D%5D%7D%5C%22%7D%2C%5C%2212102622365%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22GNavlinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22Root%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Global%20Nav%20Root%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Tourism%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Tourism%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Hotels%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_bnb%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Bed%20and%20Breakfast%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-c2-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Attractions%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Things%20to%20Do%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAttractions-g150813-Activities-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Forum%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Travel%20Forum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Photos%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Photos%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FLocationPhotos-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Maps%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Map%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FLocalMaps-g150813-Tulum-Area.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_Hotels%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22All%20Tulum%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_SD%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotel%20Deals%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FSmartDeals-g150813-Tulum_Yucatan_Peninsula-Hotel-Deals.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_LM%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Last%20Minute%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FLastMinute-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_hotels_near_me%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20Near%20Me%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNearMe%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22By%20Hotel%20Type%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_motels_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Motels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfd2-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_hostels_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hostels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-c3-zff26-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_hostels_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Campgrounds%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-c3-zff29-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Glamping%20Stays%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff105-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff10-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Business%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff7-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Family%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff4-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Green%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff24-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff8-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Luxury%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff12-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Romantic%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff3-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Casinos%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff14-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Spa%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff13-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22By%20Hotel%20Class%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%225-Star%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfc5-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%223-Star%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfc3-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%224-Star%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfc4-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_brand_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22By%20Hotel%20Brand%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Ahau%20Collection%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb22174-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22OYO%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb11779-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Luxury%20Bahia%20Principe%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb11763-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Oasis%20Hotels%20%26%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb16281-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22InterContinental%20(IHG)%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb9205-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Marriott%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb9208-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Colibri%20Boutique%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb11845-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hilton%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb9204-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_amenity_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Amenities%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_amenities_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Pet%20Friendly%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfa9-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_amenities_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotels%20with%20Pools%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfa3-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_amenities_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotels%20with%20Free%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfa7-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Tulum%20Categories%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Boutique%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Boutique-Hotels-zfp1512.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Discount%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Cheap-Hotels-zfp11040.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Honeymoon%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Honeymoon-Resorts-zfp13996.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wedding%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Wedding-Hotels-zfp16067.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22All%20Inclusive%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-All-Inclusive-Hotels-zfp41115.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22All%20Inclusive%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-All-Inclusive-Resorts-zfp1189394.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Adults%20Only%20and%20Adult%20Friendly%20Resorts%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Adults-Only-And-Adult-Friendly-Resorts-zfp28929004.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Golf%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Golf-Hotels-zfp28929053.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Singles%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Singles-Resorts-zfp28929102.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Spring%20Break%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Spring-Break-Resorts-zfp28929151.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Landmarks%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Tulum%20Archaeological%20Site%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Akumal%20Beach%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Cenotes%20Dos%20Ojos%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d1478385-Cenotes_Dos_Ojos-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Playa%20Paraiso%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d3321137-Playa_Paraiso-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Gran%20Cenote%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d669578-Gran_Cenote-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Yal-ku%20Lagoon%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d153288-Yal_ku_Lagoon-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Aktun%20Chen%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d153913-Aktun_Chen-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Akumal%20Monkey%20Sanctuary%20%26%20Rescued%20Animals%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d10441459-Akumal_Monkey_Sanctuary_Rescued_Animals-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Cenotes%20Sac%20Actun%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d3387166-Cenotes_Sac_Actun-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20The%20Yacht%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Airports%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Cancun%20Airport%20(CUN)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150807-qCUN-Cancun_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Felipe%20Carrillo%20Puerto%20International%20Airport%20(TQO)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-qTQO-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Aerodromo%20de%20Playa%20Del%20Carmen%20(PCM)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150812-qPCM-Playa_del_Carmen_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_motels_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Motels%20near%20Aerodromo%20de%20Playa%20Del%20Carmen%20(PCM)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150812-qPCM-zfd2-Playa_del_Carmen_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_categories_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Hotel%20Categories%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Ryokan%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9435-Ryokan.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Family%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft6216-Family_Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Family%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft6216%2C9672-Family_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Ski%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9672%2C19075-Ski_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Singles%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9672%2C20284-Singles_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20All-Inclusive%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9172%2C9189-All_Inclusive_Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Honeymoon%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9624%2C9672-Honeymoon_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20All-Inclusive%20Family%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft6216%2C9172%2C9672-All_Inclusive_Family_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Adults%20Only%20All-Inclusive%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9172%2C9672%2C18884-Adults_Only_All_Inclusive_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Casino%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9171-Casino_Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22ATTRACTIONS_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Things%20to%20Do%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAttractions-g150813-Activities-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22All%20Tulum%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_popular_types_of_food_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Types%20of%20Food%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Argentinian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10698-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Food%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10686-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Peruvian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10631-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hawaiian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10772-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10679-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Latin%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10639-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Fast%20Food%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10646-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dietary_restriction_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Gluten%20Free%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfz10992-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dietary_restriction_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Vegetarian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfz10665-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dietary_restriction_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Vegan%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfz10697-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dining_option_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Spanish%20Restaurants%20with%20Outdoor%20Seating%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c36-zfp6-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dining_option_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Late%20Night%20Steakhouses%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c37-zfp8-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dining_option_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%20Restaurants%20for%20Lunch%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c38-zfp30-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_popular_dishes_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Dishes%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Donuts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10758-Tulum_Yucatan_Peninsula-Donuts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20French%20Toast%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10902-Tulum_Yucatan_Peninsula-French_Toast.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Chow%20Mein%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10890-Tulum_Yucatan_Peninsula-Chow_Mein.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Unagi%20%26%20Dojo%20(Eel%20%26%20Soil)%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd11724-Tulum_Yucatan_Peninsula-Unagi_Dojo_Eel_Soil.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Cuban%20Sandwich%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd19960-Tulum_Yucatan_Peninsula-Cuban_Sandwich.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Mac%20and%20cheese%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20040-Tulum_Yucatan_Peninsula-Mac_and_cheese.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Steak%20Frites%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20336-Tulum_Yucatan_Peninsula-Steak_Frites.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Burritos%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10878-Tulum_Yucatan_Peninsula-Burrito.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Gyros%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10905-Tulum_Yucatan_Peninsula-Gyros.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Crab%20Cakes%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10894-Tulum_Yucatan_Peninsula-Crab_Cakes.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Red%20Curry%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20183-Tulum_Yucatan_Peninsula-Red_Curry.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Veggie%20Burger%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20156-Tulum_Yucatan_Peninsula-Veggie_Burger.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Tres%20Leches%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20137-Tulum_Yucatan_Peninsula-Tres_Leches.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Cucumber%20Salad%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20161-Tulum_Yucatan_Peninsula-Cucumber_Salad.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Lettuce%20Wraps%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20189-Tulum_Yucatan_Peninsula-Lettuce_Wraps.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_popular_categories_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Restaurant%20Categories%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links_breakfast%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Breakfast%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp2-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links_lunch%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Lunch%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp30-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links_dinner%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Dinner%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp58-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20with%20a%20View%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp43-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20for%20Group%20Dining%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp9-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Buffet%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp49-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Late%20Night%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp8-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20with%20Outdoor%20Seating%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp6-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Food%20Delivery%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp19-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Kid%20Friendly%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp5-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_establishments_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Bakeries%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfg9901-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_establishments_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Coffee%20%26%20Tea%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfg9900-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_establishments_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Desserts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfg9909-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Lula%20Seaside%20Boutique%20Hotel%20%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d7223463-Lula_Seaside_Boutique_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Wakax%20Hacienda%20-%20Cenote%20%26%20Boutique%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23825874-Wakax_Hacienda_Cenote_Boutique_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Hotel%20Ma'xanab%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d19006698-Hotel_Ma_xanab_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Mi%20Amor%20Colibri%20Boutique%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d7183451-Mi_Amor_Colibri_Boutique_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Mezzanine%20Tulum%2C%20a%20Small%20Luxury%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d557443-Mezzanine_Tulum_a_Small_Luxury_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Kanan%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d17495992-Kanan_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20La%20Zebra%2C%20an%20SLH%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g23240074-d296667-La_Zebra_an_SLH_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Secrets%20Akumal%20Riviera%20Maya%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g499445-d8285376-Secrets_Akumal_Riviera_Maya-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Suenos%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g23240074-d557040-Suenos_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20The%20Beach%20Tulum%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g23240074-d3979959-The_Beach_Tulum_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Landmarks%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Tulum%20Archaeological%20Site%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Akumal%20Beach%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20The%20Yacht%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20My%20Quest%20Concierge%20Private%20Tours%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23065077-My_Quest_Concierge_Private_Tours-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Dlux%20Transfer%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23852094-Dlux_Transfer-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Marine%20Life%20Akumal%20MX%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g499445-d23928640-Marine_Life_Akumal_MX-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Mayikal%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23285262-Mayikal_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Parque%20Ecoturistico%20Cenote%20Aktun%20Bej%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d26344863-Parque_Ecoturistico_Cenote_Aktun_Bej-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Tulum%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d26394539-Tulum_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Playacht%20-%20Private%20Yacht%20Rentals%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d20967421-Playacht_Private_Yacht_Rentals-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22ARTICLES_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Travel%20Stories%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FArticles%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22CRUISES_nm_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Cruises%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FCruises%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22tours%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tours%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAttractions-g150813-Activities-c42-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22ADD_LISTING_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Add%20a%20Place%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAddListing%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TRAVEL_FORUMS_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Travel%20Forum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22AIRLINES_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Airlines%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAirlines%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELERS_CHOICE_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Travelers'%20Choice%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTravelersChoice%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HELP_CENTER_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Help%20Center%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisorsupport.com%2Fhc%2Fen-us%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TRIPS_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Trip%20Planner%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTrips%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22250489043532424%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22currentLocale%5C%5C%5C%22%3A%7B%5C%5C%5C%22pointOfSaleName%5C%5C%5C%22%3A%5C%5C%5C%22United%20States%5C%5C%5C%22%2C%5C%5C%5C%22ietfLocale%5C%5C%5C%22%3A%5C%5C%5C%22en%5C%5C%5C%22%2C%5C%5C%5C%22languageName%5C%5C%5C%22%3A%5C%5C%5C%22English%5C%5C%5C%22%2C%5C%5C%5C%22country%5C%5C%5C%22%3A%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A191%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22United%20States%5C%5C%5C%22%7D%7D%7D%5C%22%7D%2C%5C%22232167697255007%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22data%5C%5C%5C%22%3A%5B%5C%5C%5C%227149%5C%5C%5C%22%2Cnull%2C%5C%5C%5C%226977%5C%5C%5C%22%2C%5C%5C%5C%226940%5C%5C%5C%22%2C%5C%5C%5C%226890%5C%5C%5C%22%2C%5C%5C%5C%227133%5C%5C%5C%22%2C%5C%5C%5C%225108%5C%5C%5C%22%2Cnull%2C%5C%5C%5C%225388%5C%5C%5C%22%2C%5C%5C%5C%227342%5C%5C%5C%22%2C%5C%5C%5C%225386%5C%5C%5C%22%2C%5C%5C%5C%226879%5C%5C%5C%22%2C%5C%5C%5C%227192%5C%5C%5C%22%2C%5C%5C%5C%227195%5C%5C%5C%22%2C%5C%5C%5C%226971%5C%5C%5C%22%2C%5C%5C%5C%226877%5C%5C%5C%22%2C%5C%5C%5C%225071%5C%5C%5C%22%2Cnull%2C%5C%5C%5C%226893%5C%5C%5C%22%2C%5C%5C%5C%226875%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%22-15724433%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22menu_links_on__r_1718132076%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3Anull%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-5481293365%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22ai_travel_assistant_fab__web__1744635547%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-16161885676%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22ollie__ai_assistant__dynamic_prompts__conversation_starters_1753882029%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-6191652010%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Routing_canonicalize%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%5D%7D%5C%22%7D%2C%5C%22-7144532626%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22Best%20dining%20in%20Tulum%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%22-8501101610%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22No%20restaurants%20were%20found%20near%20Tulum.%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%226865839759%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22alertSection%5C%5C%5C%22%3A%7B%5C%5C%5C%22alerts%5C%5C%5C%22%3A%5B%5D%7D%7D%5C%22%7D%2C%5C%22-9100016233%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22Tulum%20restaurants%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%22-12694294617%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22paginationRoutes%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22params%5C%5C%5C%22%3A%7B%5C%5C%5C%22offset%5C%5C%5C%22%3A%5C%5C%5C%220%5C%5C%5C%22%2C%5C%5C%5C%22geoId%5C%5C%5C%22%3A150813%2C%5C%5C%5C%22diningOptionId%5C%5C%5C%22%3A10955%7D%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa0-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa0-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22params%5C%5C%5C%22%3A%7B%5C%5C%5C%22offset%5C%5C%5C%22%3A%5C%5C%5C%2230%5C%5C%5C%22%2C%5C%5C%5C%22geoId%5C%5C%5C%22%3A150813%2C%5C%5C%5C%22diningOptionId%5C%5C%5C%22%3A10955%7D%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%5D%7D%5C%22%7D%7D%7D%7D%22)))%3B"></script></body></html>
"""

# Creando un objeto BeautifulSoup a partir del HTML completo
soup_completo = BeautifulSoup(html_completo_ejemplos, 'html.parser')

# Encontrar todos los bloques de restaurante
# La clase 'tbrcR' combinada con otras parece ser un buen selector para cada "tarjeta" de restaurante.
restaurant_blocks = soup_completo.find_all('div', {'data-automation': 'restaurantCard'})

print("Iniciando la extracción detallada de cada restaurante:")
restaurantes_detalhados = []  # Lista para almacenar los datos estructurados

for block in restaurant_blocks:
    # --- Selectores para la información DENTRO de cada bloque de restaurante ---

    # 1. Nombre del Restaurante
    # Pistas: El nombre está dentro de una <a> que tiene las clases 'BMQDV' y 'ukgoS'.
    # El texto real del nombre está dentro de una <div> que es hija de esa <a>.
    # Ejemplo HTML: <a class="BMQDV ... ukgoS"><div class="biGQs ...">1. Nombre del Restaurante</div></a>
    nombre_link_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'ukgoS' in c)
    nombre = "Nombre no encontrado"
    if nombre_link_tag:
        # La div específica que contiene el texto del nombre
        nombre_div_tag = nombre_link_tag.find('div', class_=lambda c: c and 'biGQs' in c and 'OgHoE' in c)
        if nombre_div_tag:
            nombre = nombre_div_tag.get_text(strip=True)

    # 2. Calificación (ej., 4,9)
    # Pistas: Encontrar la div con data-automation="bubbleRatingValue" y tomar el texto del span interno.
    calificacion_tag = block.find('div', {'data-automation': 'bubbleRatingValue'})
    calificacion = calificacion_tag.find('span').get_text(strip=True) if calificacion_tag else "Calificación no encontrada"

    # 3. Cantidad de Reseñas
    # Pistas: Está en una <div> con el atributo 'data-automation="bubbleReviewCount"'.
    # El texto real de la cantidad está dentro de un <span> que es hijo de esa <div>.
    # Usamos regex para extraer solo los números.
    review_count_tag = block.find('div', {'data-automation': 'bubbleReviewCount'})
    cantidad_resenas = "Cantidad de reseñas no encontrada"
    if review_count_tag:
        text_content = review_count_tag.get_text(strip=True)
        # Extraer solo los números de la cadena (ej: "(2.529 reseñas)" -> "2.529")
        match = re.search(r'\d[\d.]*', text_content)
        if match:
            cantidad_resenas = match.group(0).replace('.', '')  # Elimina el punto para convertir a int, si es necesario

    # 4. Tipo de Cocina (ej: Mexicana, Bar)
    # Pistas: Es un <span> dentro de un <div> con clases 'ZvrsW N G biqBm'.
    # El texto puede estar directamente en el <span> o en un <span> anidado.
    # Intentaremos tomar el primer span que no contenga un SVG o un enlace.
    cuisine_tag = block.find('div', class_=lambda c: c and 'ZvrsW' in c and 'biqBm' in c)
    tipo_cozinha = []
    if cuisine_tag:
        # Encontrar todos los spans que son hijos directos o casi directos y que contienen texto relevante
        spans = cuisine_tag.find_all('span', class_=lambda c: c and 'biGQs' in c and 'ZNjnF' in c)
        for span in spans:
            # Excluir spans que contienen SVGs (íconos) o que parecen ser contadores de reseñas
            # y tomar solo el texto que no sea de cantidad de reseñas ni "Cerrado ahora" ni "Menú"
            if not span.find('svg') and 'reseñas' not in span.get_text() and 'Cerrado ahora' not in span.get_text() and 'Menú' not in span.get_text():
                cuisine_text = span.get_text(strip=True)
                if cuisine_text:
                    tipo_cozinha.append(cuisine_text)
                # else: tipo_cozinha.append("Tipo de cocina no encontrado")
    # tipo_cocina = tipo_cocina[0]
    tipo_cocina = ", ".join(tipo_cozinha) if tipo_cozinha else "Tipo de cocina no encontrado"
    tipo_cocina = tipo_cocina.split(',')
    tipo_cocina = tipo_cocina[0]
   

    # 5. Rango de Precios (ej: $$ - $$$)
    # Pistas: Está en un <span> con las mismas clases del tipo de cocina, pero es el que no tiene SVG
    # y que corresponde al patrón de precio.
    rango_precio = "Rango de precios no encontrado"
    if cuisine_tag:
        for span in cuisine_tag.find_all('span', class_=lambda c: c and 'biGQs' in c and 'ZNjnF' in c):
            text_content = span.get_text(strip=True)
            if re.match(r'\$+', text_content):  # Verifica si el texto es uno o más símbolos de dólar
                rango_precio = text_content
                break

    # 6. Estado de Funcionamiento (Abierto/Cerrado)
    # Pistas: Está dentro de una div con clase 'Pwlnc f lFNDg', y el texto real en el span interno.
    status_funcionamiento_tag = block.find('div', class_=lambda c: c and 'Pwlnc' in c and 'lFNDg' in c)
    status_funcionamiento = status_funcionamiento_tag.find('span').get_text(strip=True) if status_funcionamiento_tag else "Estado de funcionamiento no encontrado"

    # 7. URL del Restaurante
    # Pistas: La etiqueta <a> principal que envuelve todo el bloque del restaurante.
    # El atributo 'href' contiene la ruta relativa. Necesitamos concatenarla con la base de Tripadvisor.
    url_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'PaRlG' in c)
    url_base = "https://www.tripadvisor.com.mx"
    url_restaurante = url_base + url_tag['href'] if url_tag and 'href' in url_tag.attrs else "URL no encontrada"

    # 8. URL de la Imagen Principal
    # Pistas: La primera etiqueta <picture> dentro del carrusel, y la etiqueta <img> dentro de ella.
    # El atributo 'src' contiene la URL de la imagen.
    image_tag = block.find('div', class_='IdURT').find('img') if block.find('div', class_='IdURT') else None
    url_imagen = image_tag['src'] if image_tag and 'src' in image_tag.attrs else "URL de la imagen no encontrada"

    # Almacenar los datos en un diccionario
    restaurante_info = {
        "Nombre": nombre,
        "Calificacion": calificacion,
        "Cantidad_Resenas": cantidad_resenas,
        "Tipo_Cocina": tipo_cocina,
        "Rango_Precio": rango_precio,
        "Estado_Funcionamiento": status_funcionamiento,
        "URL_Restaurante": url_restaurante,
        "URL_Imagen_Principal": url_imagen
    }
    restaurantes_detalhados.append(restaurante_info)

# Imprimir los resultados para verificación
for i, restaurante in enumerate(restaurantes_detalhados):
    print(f"\n--- Restaurante {i+1} ---")
    for key, value in restaurante.items():
        print(f"{key}: {value}")

Iniciando la extracción detallada de cada restaurante:

--- Restaurante 1 ---
Nombre: Alma Verde Tulum
Calificacion: 4.9
Cantidad_Resenas: 348
Tipo_Cocina: Mexican
Rango_Precio: $$ - $$$
Estado_Funcionamiento: Open now
URL_Restaurante: https://www.tripadvisor.com.mx/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html
URL_Imagen_Principal: https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&h=400&s=1

--- Restaurante 2 ---
Nombre: Xibak Tulum
Calificacion: 4.8
Cantidad_Resenas: 88
Tipo_Cocina: International
Rango_Precio: $$ - $$$
Estado_Funcionamiento: Opens in 4 min
URL_Restaurante: https://www.tripadvisor.com.mx/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html
URL_Imagen_Principal: https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&h=-1&s=1

--- Restaurante 3 ---
Nombre: 1. La Zebra Beach Restaurant And Bar
Calificacion: 4.8
Cantidad_Resena

In [28]:
from bs4 import BeautifulSoup

html_do_tripadvisor = """
<!DOCTYPE html><html lang="en-US"><head><link rel="icon" id="favicon" type="image/x-icon" href="https://static.tacdn.com/assets/s/6b5096f1.ico"/><link rel="icon" type="image/svg+xml" href="https://static.tacdn.com/assets/s/3ba4aeda.svg"/><link rel="apple-touch-icon" sizes="180x180" href="https://static.tacdn.com/assets/s/12951248.png"/><link rel="mask-icon" sizes="any" color="#00210c" href="https://static.tacdn.com/assets/s/4f134e39.svg"/><meta name="theme-color" content="#00eb5b"/><meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"/><meta name="format-detection" content="telephone=no"/><meta property="al:ios:app_name" content="TripAdvisor"/><meta property="al:ios:app_store_id" content="284876795"/><meta property="twitter:app:id:ipad" name="twitter:app:id:ipad" content="284876795"/><meta property="twitter:app:id:iphone" name="twitter:app:id:iphone" content="284876795"/><meta property="al:ios:url" content="tripadvisor://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html?m=33762"/><meta property="twitter:app:url:ipad" name="twitter:app:url:ipad" content="tripadvisor://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html?m=33762"/><meta property="twitter:app:url:iphone" name="twitter:app:url:iphone" content="tripadvisor://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html?m=33762"/><meta name="apple-itunes-app" content="app-id=-1"/><script type="application/ld+json">{"@context":"https:\u002F\u002Fschema.org","@graph":[{"@type":"Organization","name":"Tripadvisor","url":"https:\u002F\u002Fwww.tripadvisor.com\u002F","logo":"https:\u002F\u002Fstatic.tacdn.com\u002Fimg2\u002Fbrand_refresh\u002FTripadvisor_logoset_solid_green.svg","sameAs":["https:\u002F\u002Fwww.facebook.com\u002FTripadvisor","https:\u002F\u002Ftwitter.com\u002FTripadvisor","https:\u002F\u002Finstagram.com\u002Ftripadvisor\u002F","https:\u002F\u002Fwww.linkedin.com\u002Fcompany\u002Ftripadvisor"]},{"@type":"WebSite","name":"Tripadvisor","url":"https:\u002F\u002Fwww.tripadvisor.com\u002F","potentialAction":{"@type":"SearchAction","target":{"@type":"EntryPoint","urlTemplate":"https:\u002F\u002Fwww.tripadvisor.com\u002FSearch?q={search_term_string}"},"query":"required name=search_term_string"}}]}</script><meta property="fb:pages" content="5863091683"/><meta property="fb:pages" content="329182043776593"/><meta name="twitter:widgets:csp" content="on"/><script nonce="884ba8d9-c881-4482-b7ef-5b279351627a">window.performance&&performance.mark&&performance.getEntriesByType&&0===performance.getEntriesByType("visibility-state").length&&(performance.mark("visible"===document.visibilityState?"visible":"hidden"),document.addEventListener&&document.addEventListener("visibilitychange",function(){performance.mark("visible"===document.visibilityState?"visible":"hidden")},!1));</script><title>THE 10 BEST Moderately-Priced Restaurants in Tulum</title><meta name="viewport" content="width=device-width, initial-scale=1.0, viewport-fit=cover"/><meta name="description" content="Best Moderately Priced Restaurants in Tulum, Riviera Maya: Find Tripadvisor traveler reviews of THE BEST Tulum Mid Range Restaurants and search by price, location, and more."/><link rel="canonical" href="https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en" href="https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-GB" href="https://www.tripadvisor.co.uk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-CA" href="https://www.tripadvisor.ca/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr-CA" href="https://fr.tripadvisor.ca/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="it" href="https://www.tripadvisor.it/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es" href="https://www.tripadvisor.es/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="de" href="https://www.tripadvisor.de/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr" href="https://www.tripadvisor.fr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ja" href="https://www.tripadvisor.jp/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="zh-Hans" href="https://cn.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-IN" href="https://www.tripadvisor.in/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="sv" href="https://www.tripadvisor.se/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="nl" href="https://www.tripadvisor.nl/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="pt" href="https://www.tripadvisor.com.br/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="tr" href="https://www.tripadvisor.com.tr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="da" href="https://www.tripadvisor.dk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-MX" href="https://www.tripadvisor.com.mx/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-IE" href="https://www.tripadvisor.ie/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ar" href="https://ar.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ar-EG" href="https://www.tripadvisor.com.eg/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="de-AT" href="https://www.tripadvisor.at/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="el" href="https://www.tripadvisor.com.gr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-AU" href="https://www.tripadvisor.com.au/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-MY" href="https://www.tripadvisor.com.my/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-NZ" href="https://www.tripadvisor.co.nz/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-PH" href="https://www.tripadvisor.com.ph/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-SG" href="https://www.tripadvisor.com.sg/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-ZA" href="https://www.tripadvisor.co.za/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-AR" href="https://www.tripadvisor.com.ar/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-CL" href="https://www.tripadvisor.cl/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-CO" href="https://www.tripadvisor.co/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-PE" href="https://www.tripadvisor.com.pe/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="es-VE" href="https://www.tripadvisor.com.ve/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="id" href="https://www.tripadvisor.co.id/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="he" href="https://www.tripadvisor.co.il/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ko" href="https://www.tripadvisor.co.kr/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="nb" href="https://no.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="pt-PT" href="https://www.tripadvisor.pt/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="ru" href="https://www.tripadvisor.ru/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="th" href="https://th.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="vi" href="https://www.tripadvisor.com.vn/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="zh-Hant" href="https://www.tripadvisor.com.tw/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="de-CH" href="https://www.tripadvisor.ch/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr-CH" href="https://fr.tripadvisor.ch/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="it-CH" href="https://it.tripadvisor.ch/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="en-HK" href="https://en.tripadvisor.com.hk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="fr-BE" href="https://fr.tripadvisor.be/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="nl-BE" href="https://www.tripadvisor.be/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><link rel="alternate" hrefLang="zh-hk" href="https://www.tripadvisor.com.hk/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><meta property="og:url" content="https://www.tripadvisor.com/Restaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html"/><meta property="og:site_name" content="Tripadvisor"/><meta property="og:type" content="website"/><meta property="og:locale" content="en-US"/><meta property="og:title" content="THE 10 BEST Moderately-Priced Restaurants in Tulum"/><meta property="og:description" content="Best Moderately Priced Restaurants in Tulum, Riviera Maya: Find Tripadvisor traveler reviews of THE BEST Tulum Mid Range Restaurants and search by price, location, and more."/><meta property="og:image" content="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=600&amp;h=600&amp;s=1"/><meta property="og:image:width" content="600"/><meta property="og:image:height" content="600"/><meta property="twitter:image" content="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=600&amp;h=600&amp;s=1"/><link rel="stylesheet" href="https://static.tacdn.com/assets/35pr0yuitw1c-c.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/p7jq7m.MNync6r.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/1tem5o.hHmbYjL.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/b5kt6r.n2F6kJa.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/633gr8.DQuSDwJ.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/6h4i0r.GoM2Ynr.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/us1vwl.EXnWuEi.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/70j2tf.BwhsA2m.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/eekt6e.u6AkCOt.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/z2nh2s.8hQa4LG.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/mf5oe4.a8E9pkW.css" crossorigin=""/><link rel="stylesheet" href="https://static.tacdn.com/assets/76j3t0.GC8HV7Y.css" crossorigin=""/><script src="https://static.tacdn.com/assets/0ci59y4imqm7-c.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/37os3oto1kwr-c.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/im/f77fb48474eac182.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/gg6fk5.tkO6FtA.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/mqwhqn.ZtVGKN4.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/8cre3h.gxuh19I.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/p7jq7m.MNync6r.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/1tem5o.hHmbYjL.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/b5kt6r.n2F6kJa.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/w43m8k.bS66dJh.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/y9enps.BdLa8A8.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/n4zdla._ZcRb0A.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/633gr8.DQuSDwJ.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/6h4i0r.GoM2Ynr.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/us1vwl.EXnWuEi.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/5xqg72.tPBR6UO.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/70j2tf.BwhsA2m.en-US.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/xfufl0.-vjAwhJ.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/o80y56.qGwLjD9.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/ypf49l.F13Hn5S.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/ti102j.ns_prO6.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/d3i8k1.6Xxde1x.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/mxujx7.UvrWd0b.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/eekt6e.u6AkCOt.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/z2nh2s.8hQa4LG.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/mf5oe4.a8E9pkW.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/v42v3u.g2VkMHb.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script><script src="https://static.tacdn.com/assets/76j3t0.GC8HV7Y.js" async="" crossorigin="" fetchpriority="low" nonce="884ba8d9-c881-4482-b7ef-5b279351627a"></script></head><body><div id="lithium-root"><!--$--><div id="ad-banner" aria-hidden="true" class="adsbygoogle ad-slot ad-banner adsbox ad" style="position:absolute;top:0px;bottom:-20px;width:10px;height:10px;background-color:transparent"><img src="data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACwAAAAAAQABAAACAkQBADs=" alt="Advertisement" style="display:block;width:1px;height:1px"/></div><!--$--><!--/$--><div class="CvDIt _Q f e Gi Ra _s P6"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Skip to main content</span></button></div><div class="ad SxZAK f _h u e j Gi"><div class="ktyWU f _h u e j w Gi"><div id="slot:5x1-728x90:horizon" class="WMKsc w xbgrb o"></div></div></div><span><div></div><header data-component="ta-brand-header" class="aVAoX t z"><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><nav class="MGToN u"><div class="oqeTs u JpEOb"><a class="f _Z wSSLS" href="/"><picture><img class="XpHHt _Z" src="https://static.tacdn.com/img2/brand_refresh_2025/logos/wordmark.svg" alt="Tripadvisor"/></picture></a></div><div class="OzBsv"><div class="HREWK t l s _U S ijEOu"></div><div class="gqnca"><div class="ZjkxF f e Wh z _f oPzxw" data-test-attribute="typeahead-SINGLE_SEARCH_NAV"><form role="search" action="/Search"><input type="hidden" name="searchSessionId" value="00128845d6928bb2.ssid"/><input type="hidden" name="searchNearby" value="false"/><input type="hidden" name="geo" value="150813"/><div class="oKjvH f u Ma Nk MA NM"><div class="F1 f u Q2"><button type="submit" formaction="/Search" class="LhcRH _G _H B- G_ _S t u j H0" title="Search" aria-label="Search" tabindex="-1"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M10.12 4.14a5.99 5.99 0 1 0 0 11.98 5.99 5.99 0 0 0 0-11.98m-7.49 5.99a7.49 7.49 0 1 1 13.299 4.728L21.37 20.3l-1.06 1.061-5.44-5.44A7.49 7.49 0 0 1 2.63 10.13"></path></svg></button><div class="w"><div class="biGQs _P AWdfh"><input type="search" autoComplete="off" autoCorrect="off" autoCapitalize="none" spellcheck="false" required="" name="q" disabled="" class="hUpcN _G G_ B- z F1 w R0 OavzT _O" placeholder="Search" title="Search" role="searchbox" aria-label="Search" aria-autocomplete="list" value=""/></div></div></div></div></form></div></div></div><div class="inHiW JpEOb"><div class="eByLG QA c"><div class="AuQZR f"><div class="f u" data-automation="topNav_rewards"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" href="/Rewards"><span class="biGQs _P ezezH">Rewards</span></a></div><div class="f u" data-automation="topNav_discover"><div class="JLKop w"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH">Discover</span></button></div></div><div class="f u" data-automation="topNav_trips"><div class="JLKop w"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH">Trips</span></button></div></div><div class="f u" data-automation="topNav_review"><div class="JLKop w"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH">Review</span></button></div></div></div></div></div><div class="lGhNq QA c JpEOb" data-automation="desktop-cart-and-profile"><button class="rmyCe _G B- z _S c Wc wSSLS jWkoZ InwKl" type="button"><span class="biGQs _P ezezH"><span class="q"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M9.31 9.82h4.178c-.069-1.591-.356-2.993-.766-4.017-.237-.593-.501-1.023-.756-1.293-.211-.223-.38-.3-.5-.32h-.133c-.12.02-.289.097-.5.32-.255.27-.519.7-.756 1.293-.41 1.024-.697 2.426-.767 4.017m-.374-5.14q-.135.272-.252.566C8.194 6.472 7.88 8.07 7.81 9.82H5.055a6.39 6.39 0 0 1 3.88-5.14m2.301-1.989a7.883 7.883 0 0 0-7.726 7.88 7.88 7.88 0 0 0 7.884 7.885c.584 0 .871-.014 1.11-.074.124-.031.172-.049.213-.064.058-.02.099-.036.312-.073l-.26-1.477a4 4 0 0 0-.628.159c-.031.007-.132.029-.743.029-.121 0-.313-.06-.566-.327-.255-.27-.519-.699-.756-1.292-.41-1.025-.697-2.426-.767-4.017h4.203a4.7 4.7 0 0 1-.113.843 6 6 0 0 1-.112.413l-.012.04-.004.016c-.034.114-.089.298-.1.478l1.498.088v.006s.007-.035.044-.161l.012-.04c.038-.126.09-.298.136-.505.072-.313.134-.698.152-1.178h2.734a4.2 4.2 0 0 1-.195.949c-.055.16-.11.28-.166.403l-.004.01-.01.02c-.046.101-.132.288-.161.486l1.484.219-.005.026s.012-.032.057-.13l.008-.018c.056-.123.137-.3.216-.529.168-.49.31-1.168.31-2.186a7.88 7.88 0 0 0-7.72-7.879 2 2 0 0 0-.325 0m2.626 1.99q.134.271.252.565c.49 1.226.805 2.824.875 4.574h2.75a6.38 6.38 0 0 0-3.877-5.14M8.94 16.466a8 8 0 0 1-.256-.573c-.49-1.227-.805-2.824-.875-4.574H5.055a6.39 6.39 0 0 0 3.885 5.147"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M20.489 12.343h-8.75v6.648h5.255l3.495 2.325zm-1.5 1.5v4.674l-1.542-1.026H13.24v-3.648z"></path></svg><div class="TzXjz d Vb"></div>USD</span></span></button><a class="rmyCe _G B- z _S c Wc wSSLS w jWkoZ sOtnj" href="/RegistrationController?flow=sign_up_and_save&amp;returnTo=%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html&amp;fullscreen=true&amp;flowOrigin=login&amp;hideNavigation=true&amp;isLithium=true"><span class="biGQs _P ezezH"><div class="eoYut">Sign in</div></span></a></div></nav></div><span><div class="o"><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="uNcsI _T o" data-test-target="global-nav"><div class="fSuCF w q czCLw"><div class="_T w"><div class="OykfZ f _e _h"><div class="KjnXN"><div role="tablist" data-automation="tab-list"><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_Tourism"><span class="biGQs _P OgHoE"><a class="JBbKZ Q1 nadlY" href="/ClientLink?value=c0xNXy9Ub3VyaXNtLWcxNTA4MTMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEtVmFjYXRpb25zLmh0bWxfRkdx"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.256 7.032A2.441 2.441 0 1 0 9.746 8.9a2.441 2.441 0 0 0 4.51-1.868m-2.615.065a.941.941 0 1 1 .72 1.738.941.941 0 0 1-.72-1.738"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M7.26 5.882 2.748 6.91v14.1l6.418-1.951 5.64 1.714 6.446-1.47V5.708l-4.185.954a5.34 5.34 0 0 0-5.066-3.67 5.34 5.34 0 0 0-4.741 2.89M6.73 7.54a5 5 0 0 0-.053.745c0 1.265.506 2.476 1.163 3.536l-3.592.955v-4.67zm10.592.6q.003.073.002.145c0 1.573-.782 3.063-1.663 4.284l-.077.105v.353l4.168-.95v-4.49zm-3.288 6.368c-.363.386-.706.73-.996 1.019l-.057.057-.002.002-.987.98-.974-.998-.03-.03a44 44 0 0 1-1.072-1.093v3.273l4.168 1.266v-4.46zm.41-2.817c-.778 1.077-1.72 2.028-2.437 2.745-.72-.716-1.676-1.675-2.462-2.765-.814-1.128-1.368-2.294-1.368-3.386 0-2.082 1.717-3.795 3.824-3.795s3.823 1.713 3.823 3.795c0 1.098-.56 2.27-1.38 3.406m5.308 1.924-4.168.95v4.491l4.168-.95zm-15.504.713 4.168-1.108v4.498l-4.168 1.266z"></path></svg>Tulum</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_Hotels"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=VjNBXy9Ib3RlbHMtZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS1Ib3RlbHMuaHRtbF9NR2g=">Hotels</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_ThingsToDo"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=WWI5Xy9BdHRyYWN0aW9ucy1nMTUwODEzLUFjdGl2aXRpZXMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEuaHRtbF9ZakE=">Things to Do</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS BmgDU" role="tab" type="button" aria-selected="true" data-automation="navHeader_Restaurants"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=OXZ1Xy9SZXN0YXVyYW50cy1nMTUwODEzLVR1bHVtX1l1Y2F0YW5fUGVuaW5zdWxhLmh0bWxfYjBB">Restaurants</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_Cruises"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/Cruises">Cruises</a></span></button><button class="PFswe d Gv B- _S Mi Nj MH NL wSSLS" role="tab" type="button" aria-selected="false" tabindex="-1" data-automation="navHeader_ViewTravelForum"><span class="biGQs _P ezezH"><a class="JBbKZ Q1" href="/ClientLink?value=OWZJXy9TaG93Rm9ydW0tZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS5odG1sX2xNbQ==">Forums</a></span></button></div></div></div><div class="ADnRg _Q t l _U s S Z1"><div class="oeBcu f u _Q t s Y yfScb"><div class="FEPIv"><button class="BrOJk u j z _F _S wSSLS tIqAi Vonfv" type="button" aria-hidden="true" tabindex="-1" aria-label="Previous"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="m16.6 5.65-6.4 6.4 6.4 6.3-1.4 1.4-7.8-7.7 7.8-7.8z"></path></svg></button></div></div><div class="oeBcu f u _Q t s Y MTngI"><div class="FEPIv"><button class="BrOJk u j z _F _S wSSLS tIqAi Vonfv" type="button" aria-hidden="true" tabindex="-1" aria-label="Next"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></button></div></div></div></div></div></div><div class="hNYnL _T"></div></div></div></span></header></span><main><!--$--><div data-test-target="restaurants-list"><div class="SIFnQ D Za" style="bottom:50px;right:100px"><!--$--><!--/$--></div><!--$--><!--/$--><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="f k O Cj Pj PN Pw PA"><div data-automation="breadcrumbs" class="Nm"><div class="buhZJ _T Cj"><a href="/Tourism-g150768-Mexico-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Mexico</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g150805-Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Yucatan Peninsula</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g1575485-Quintana_Roo_Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Quintana Roo</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g659488-Riviera_Maya_Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Riviera Maya</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Tourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Tulum</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ EabUM FGwzt ukgoS"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Tulum Restaurants</span></span></a><span class="XhBgo y S0 c"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="m7.4 5.65 6.4 6.4-6.4 6.3 1.4 1.4 7.8-7.7-7.8-7.8z"></path></svg></span></div><div class="buhZJ _T Cj"><span class="mzjtg n q"><span class="biGQs _P avBIb navcl">Tulum Mid Range Restaurants</span></span></div><script type="application/ld+json">{"@context":"https:\u002F\u002Fschema.org","@type":"BreadcrumbList","itemListElement":[{"@type":"ListItem","position":1,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g150768-Mexico-Vacations.html","name":"Mexico"},{"@type":"ListItem","position":2,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g150805-Yucatan_Peninsula-Vacations.html","name":"Yucatan Peninsula"},{"@type":"ListItem","position":3,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g1575485-Quintana_Roo_Yucatan_Peninsula-Vacations.html","name":"Quintana Roo"},{"@type":"ListItem","position":4,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g659488-Riviera_Maya_Yucatan_Peninsula-Vacations.html","name":"Riviera Maya"},{"@type":"ListItem","position":5,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html","name":"Tulum"},{"@type":"ListItem","position":6,"item":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurants-g150813-Tulum_Yucatan_Peninsula.html","name":"Tulum Restaurants"},{"@type":"ListItem","position":7,"name":"Tulum Mid Range Restaurants"}]}</script></div><h1 data-automation="masthead_h1" class="Wi _m H0"><div class="biGQs _P VImYz avBIb navcl">Best Moderately Priced Restaurants in Tulum, Riviera Maya</div></h1></div></div><div></div><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="u f k mowmC"><div data-test-target="main_h1"><h2 class="biGQs _P SrOyz">Tulum Mid Range Restaurants</h2></div><div class="Mjzwz u f K"><span class="I"><button class="Datwj z W _S Gn B1 Z BF Rd _M j u Nfawr wSSLS" type="button" data-automation="reserveTable"><div class="SXRwk j u"><span class="Ni"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M8.007 2.75a.75.75 0 0 1 .748.752l-.002.748h2.512V3.5a.75.75 0 0 1 1.5 0v.75h2.472l-.002-.748a.75.75 0 1 1 1.5-.004l.002.752h4.018v17H3.245v-17h4.008l.002-.752a.75.75 0 0 1 .752-.748m-.758 3H4.745v14h14.51v-14h-2.513l.006 1.747-1.5.005-.006-1.752h-2.477V7.5h-1.5V5.75H8.749l-.004 1.752-1.5-.004zm-.004 6h5.5v5.5h-5.5zm1.5 1.5v2.5h2.5v-2.5z"></path></svg></span><span class="biGQs _P FvqAl FGgoA AWdfh">Reserve a table</span></div></button></span><span data-automation="mapButton"><button class="EAeYu z Xa q W _S Gy Rd Cr _M pGzPy wSSLS" type="button" aria-label="View map"><span class="sZLBj u k Q1"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg><span class="biGQs _P AWdfh">Map</span></span></button></span></div></div><div class="Mjzwz u f K tyUdl"><div class="u f"><button class="Datwj z W _S Gn B1 Z BF Rd _M j u Nfawr wSSLS XuzGl" type="button"><div class="SXRwk j u"><span class="biGQs _P FvqAl FGgoA AWdfh"><span data-automation="appliedBandFilter2">Mid-range</span></span><span class="NK"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M10.94 12 4.47 5.53l1.06-1.06L12 10.94l6.47-6.47 1.06 1.06L13.06 12l6.47 6.47-1.06 1.06L12 13.06l-6.47 6.47-1.06-1.06z"></path></svg></span></div></button></div><button class="xyffN _G B- _S _W u _T c G_ wSSLS eeawa" type="button" data-automation="clearAllfilters"><span class="biGQs _P ezezH">Clear all filters</span></button></div><div class="isRBk"><hr class="RMdFe tyUdl"/><div class="fYwEb izCpD"><div class="f u Q2"><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path d="m16.6 5.65-6.4 6.4 6.4 6.3-1.4 1.4-7.8-7.7 7.8-7.8z"></path></svg><span class="biGQs _P VImYz AWdfh">Tulum restaurants</span></a></div><div class="u f Q2"><div class="biGQs _P VImYz AWdfh" data-automation="resultsTotal">397<!-- --> results</div><div class="NB"><span><div class="f u"><div class="biGQs _P VImYz egaXP kSNRl ZNjnF">Sort:<!-- --> </div><div class="lOvTR"><button class="xyffN _G B- _S _W u _T c G_ wSSLS eeawa zMkft" type="button"><span class="biGQs _P ezezH">Relevance</span><div class="YQrIS _W PH"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div><div class="DgJpu"><button type="button" class="ypcsE _S wSSLS" aria-labelledby=":lithium-Rfjb8s9f9bvlq:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb rihVH UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-Rfjb8s9f9bvlq:" class="KrpDA o">Restaurants ranked by how well they match your selections and traveler reviews.</span></div></div></span></div></div></div><div class="seiBA _T wCxWg" data-automation="LeftRailContainer"><div class="wjLQL _T Ft" data-automation="LeftRail"><div class="NbqPV e iESum"><div class="WVPPt y S5 Gi f e"><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Establishment type</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R2pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R2pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.10591" aria-labelledby="establishmentTypes.10591_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.10591" value="10591"/></div><div class="hTbJa f H o"><label id="establishmentTypes.10591_label" for="establishmentTypes.10591" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Restaurants</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.11776" aria-labelledby="establishmentTypes.11776_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.11776" value="11776"/></div><div class="hTbJa f H o"><label id="establishmentTypes.11776_label" for="establishmentTypes.11776" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Bars &amp; Pubs</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.9900" aria-labelledby="establishmentTypes.9900_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.9900" value="9900"/></div><div class="hTbJa f H o"><label id="establishmentTypes.9900_label" for="establishmentTypes.9900" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Coffee &amp; Tea</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="establishmentTypes.9909" aria-labelledby="establishmentTypes.9909_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="establishmentTypes.9909" value="9909"/></div><div class="hTbJa f H o"><label id="establishmentTypes.9909_label" for="establishmentTypes.9909" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Dessert</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button" aria-expanded="false" aria-controls="filter-expando-establishmentTypes"><span class="biGQs _P ezezH">Show more<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Meal type</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R4pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R4pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10597" aria-labelledby="mealTypes.10597_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10597" value="10597"/></div><div class="hTbJa f H o"><label id="mealTypes.10597_label" for="mealTypes.10597" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Breakfast</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10606" aria-labelledby="mealTypes.10606_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10606" value="10606"/></div><div class="hTbJa f H o"><label id="mealTypes.10606_label" for="mealTypes.10606" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Brunch</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10598" aria-labelledby="mealTypes.10598_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10598" value="10598"/></div><div class="hTbJa f H o"><label id="mealTypes.10598_label" for="mealTypes.10598" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Lunch</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="mealTypes.10599" aria-labelledby="mealTypes.10599_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="mealTypes.10599" value="10599"/></div><div class="hTbJa f H o"><label id="mealTypes.10599_label" for="mealTypes.10599" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Dinner</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Cuisines</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R6pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R6pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.5110" aria-labelledby="cuisines.5110_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.5110" value="5110"/></div><div class="hTbJa f H o"><label id="cuisines.5110_label" for="cuisines.5110" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Mexican</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.10643" aria-labelledby="cuisines.10643_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.10643" value="10643"/></div><div class="hTbJa f H o"><label id="cuisines.10643_label" for="cuisines.10643" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Seafood</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.10640" aria-labelledby="cuisines.10640_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.10640" value="10640"/></div><div class="hTbJa f H o"><label id="cuisines.10640_label" for="cuisines.10640" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Bar</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="cuisines.10679" aria-labelledby="cuisines.10679_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="cuisines.10679" value="10679"/></div><div class="hTbJa f H o"><label id="cuisines.10679_label" for="cuisines.10679" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Healthy</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Show all</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Dishes</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-R8pdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-R8pdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.20515" aria-labelledby="dishes.20515_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.20515" value="20515"/></div><div class="hTbJa f H o"><label id="dishes.20515_label" for="dishes.20515" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Cocido</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.19954" aria-labelledby="dishes.19954_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.19954" value="19954"/></div><div class="hTbJa f H o"><label id="dishes.19954_label" for="dishes.19954" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Tacos</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.22444" aria-labelledby="dishes.22444_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.22444" value="22444"/></div><div class="hTbJa f H o"><label id="dishes.22444_label" for="dishes.22444" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Beef Tacos</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="dishes.10937" aria-labelledby="dishes.10937_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="dishes.10937" value="10937"/></div><div class="hTbJa f H o"><label id="dishes.10937_label" for="dishes.10937" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Shrimp</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Show all</span></button></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH"><div class="f">Awards<div class="TtrFj"><div class="biGQs _P ezezH"><span class="NK"><div class="DgJpu"><button type="button" class="ypcsE _S wSSLS" aria-labelledby=":lithium-R4bapdb8s9f9bvlq:"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path d="M11 10v7h2v-7zm-.034-2.952a1.25 1.25 0 0 0-.216.692A1.24 1.24 0 0 0 12 9a1.25 1.25 0 1 0-1.034-1.952"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M12 2C6.477 2 2 6.477 2 12s4.477 10 10 10 10-4.477 10-10S17.523 2 12 2M4 12a8 8 0 1 1 16 0 8 8 0 0 1-16 0"></path></svg></button><span id=":lithium-R4bapdb8s9f9bvlq:" class="KrpDA o">Travelers’ Choice Awards winners (including the “Best of the Best” title) are among the top 10% of listings on Tripadvisor, according to the reviews and opinions of travelers across the globe.</span></div></span></div></div></div></h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Rapdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rapdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="travelersChoice.coe" aria-labelledby="travelersChoice.coe_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="travelersChoice.coe" value="coe"/></div><div class="hTbJa f H o"><label id="travelersChoice.coe_label" for="travelersChoice.coe" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="f Q1 u"><svg width="24" height="24" viewBox="0 0 24 24" fill="none" class="iezEl I" aria-hidden="true"><mask id=":lithium-R2ulapdb8s9f9bvlq:" fill="#fff"><path d="M0 12C0 5.373 5.373 0 12 0s12 5.373 12 12-5.373 12-12 12S0 18.627 0 12"></path></mask><path d="M0 12C0 5.373 5.373 0 12 0s12 5.373 12 12-5.373 12-12 12S0 18.627 0 12" fill="#00EB5B"></path><path d="M12 24v-1C5.925 23 1 18.075 1 12h-2c0 7.18 5.82 13 13 13zm12-12h-1c0 6.075-4.925 11-11 11v2c7.18 0 13-5.82 13-13zM12 0v1c6.075 0 11 4.925 11 11h2c0-7.18-5.82-13-13-13zm0 0v-1C4.82-1-1 4.82-1 12h2C1 5.925 5.925 1 12 1z" fill="#002B11" mask="url(#:lithium-R2ulapdb8s9f9bvlq:)"></path><path d="M12 5.294a6.202 6.202 0 1 0 .001 12.404 6.202 6.202 0 0 0 0-12.404m0 11.71a5.508 5.508 0 1 1 0-11.016 5.508 5.508 0 0 1 0 11.015m-1.38-5.088a.759.759 0 1 1-1.518 0 .759.759 0 0 1 1.518 0m4.277 0a.759.759 0 1 1-1.517 0 .759.759 0 0 1 1.517 0m.683-1.58.7-.762H14.73a4.8 4.8 0 0 0-2.73-.84 4.8 4.8 0 0 0-2.725.84H7.72l.7.761a2.14 2.14 0 1 0 2.896 3.15l.685.747.686-.746a2.14 2.14 0 1 0 2.895-3.15m-5.72 3.027a1.448 1.448 0 1 1 0-2.896 1.448 1.448 0 0 1 0 2.896M12 11.874c0-.952-.693-1.77-1.607-2.12a4.2 4.2 0 0 1 1.606-.32c.57 0 1.113.114 1.607.32-.913.35-1.606 1.168-1.606 2.12m2.139 1.49a1.448 1.448 0 1 1 0-2.897 1.448 1.448 0 0 1 0 2.896m-7.046 3.099s-.314-.788-1.163-1.585c-.85-.798-1.92-.943-1.92-.943s.381.85 1.285 1.699c.904.848 1.798.83 1.798.83m2.141 1.174s-.437-.425-1.233-.735-1.551-.16-1.551-.16.499.45 1.346.78 1.438.115 1.438.115m2.077.421s-.454-.187-1.13-.163a2.3 2.3 0 0 0-1.18.392s.507.185 1.227.16c.722-.023 1.083-.389 1.083-.389m-5.154-3.502s-.146-.84-.812-1.795a3.84 3.84 0 0 0-1.679-1.315s.198.915.907 1.932 1.584 1.178 1.584 1.178m-.837-2.559s.163-.851-.11-1.994A3.85 3.85 0 0 0 4.13 8.16s-.141.94.15 2.157c.292 1.217 1.04 1.679 1.04 1.679" fill="#002B11"></path><path d="M16.906 16.462s.314-.788 1.164-1.585c.849-.798 1.919-.943 1.919-.943s-.381.85-1.285 1.699c-.904.848-1.798.83-1.798.83m-2.139 1.174s.436-.425 1.232-.735 1.551-.16 1.551-.16-.498.45-1.346.78c-.847.33-1.437.115-1.437.115m-2.077.421s.453-.187 1.13-.163 1.18.392 1.18.392-.507.185-1.228.16c-.722-.023-1.083-.389-1.083-.389m5.154-3.502s.146-.84.812-1.795a3.84 3.84 0 0 1 1.679-1.315s-.198.915-.907 1.932-1.584 1.178-1.584 1.178m.837-2.559s-.163-.851.11-1.994c.274-1.143 1.078-1.842 1.078-1.842s.141.94-.15 2.157-1.039 1.679-1.039 1.679M12 18.706a.345.345 0 1 0 0-.689.345.345 0 0 0 0 .689" fill="#002B11"></path></svg>Travelers’ Choice</div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Price</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Rcpdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rcpdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="priceTypes.10953" aria-labelledby="priceTypes.10953_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="priceTypes.10953" value="10953"/></div><div class="hTbJa f H o"><label id="priceTypes.10953_label" for="priceTypes.10953" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Cheap Eats</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="priceTypes.10955" aria-labelledby="priceTypes.10955_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="true" name="priceTypes.10955" checked="" value="10955"/></div><div class="hTbJa f H o"><label id="priceTypes.10955_label" for="priceTypes.10955" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Mid-range</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="priceTypes.10954" aria-labelledby="priceTypes.10954_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="priceTypes.10954" value="10954"/></div><div class="hTbJa f H o"><label id="priceTypes.10954_label" for="priceTypes.10954" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Fine Dining</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Online options &amp; offers</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Repdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Repdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="onlineOptions.20693" aria-labelledby="onlineOptions.20693_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="onlineOptions.20693" value="20693"/></div><div class="hTbJa f H o"><label id="onlineOptions.20693_label" for="onlineOptions.20693" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Online Reservations </span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Traveler rating</h3> <button type="button" tabindex="0" aria-expanded="false" aria-controls=":lithium-Rgpdb8s9f9bvlq:" class="FSCFM _S B- Cq G- fjLuk"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rgpdb8s9f9bvlq:" class="qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="minimumTravelerRating.TRAVELER_RATING_LOW" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_LOW_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_LOW" value="TRAVELER_RATING_LOW"/></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_LOW_label" for="minimumTravelerRating.TRAVELER_RATING_LOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-R5qlgpdb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R5qlgpdb8s9f9bvlq:">3 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P VImYz biKBZ AWdfh">&amp; up</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="minimumTravelerRating.TRAVELER_RATING_MEDIUM" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_MEDIUM" value="TRAVELER_RATING_MEDIUM"/></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_MEDIUM_label" for="minimumTravelerRating.TRAVELER_RATING_MEDIUM" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-R5slgpdb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R5slgpdb8s9f9bvlq:">4 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg></div></div></div><span class="biGQs _P VImYz biKBZ AWdfh">&amp; up</span></span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="minimumTravelerRating.TRAVELER_RATING_HIGH" aria-labelledby="minimumTravelerRating.TRAVELER_RATING_HIGH_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="minimumTravelerRating.TRAVELER_RATING_HIGH" value="TRAVELER_RATING_HIGH"/></div><div class="hTbJa f H o"><label id="minimumTravelerRating.TRAVELER_RATING_HIGH_label" for="minimumTravelerRating.TRAVELER_RATING_HIGH" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh"><div class="VVbkp"><div class="MyMKp u"><div class="nKWJn u"><svg class="evwcZ" viewBox="0 0 128 24" width="88" height="16" aria-labelledby=":lithium-R5ulgpdb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R5ulgpdb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg></div></div></div></span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Open now</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Ripdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Ripdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="openNow.OPEN_NOW" aria-labelledby="openNow.OPEN_NOW_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="openNow.OPEN_NOW" value="OPEN_NOW"/></div><div class="hTbJa f H o"><label id="openNow.OPEN_NOW_label" for="openNow.OPEN_NOW" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Open now</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Dietary restrictions</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Rkpdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rkpdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diets.10665" aria-labelledby="diets.10665_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diets.10665" value="10665"/></div><div class="hTbJa f H o"><label id="diets.10665_label" for="diets.10665" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Vegetarian friendly</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diets.10697" aria-labelledby="diets.10697_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diets.10697" value="10697"/></div><div class="hTbJa f H o"><label id="diets.10697_label" for="diets.10697" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Vegan options</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diets.10992" aria-labelledby="diets.10992_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diets.10992" value="10992"/></div><div class="hTbJa f H o"><label id="diets.10992_label" for="diets.10992" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Gluten free options</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">MICHELIN Guide</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Rmpdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rmpdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="michelinAwards.21964" aria-labelledby="michelinAwards.21964_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="michelinAwards.21964" data-automation="michelinGuideFilter" value="21964"/></div><div class="hTbJa f H o"><label id="michelinAwards.21964_label" for="michelinAwards.21964" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">MICHELIN Guide</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="michelinAwards.21969" aria-labelledby="michelinAwards.21969_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="michelinAwards.21969" value="21969"/></div><div class="hTbJa f H o"><label id="michelinAwards.21969_label" for="michelinAwards.21969" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">MICHELIN Bib Gourmand</span></span></span></label></div></span></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Great for</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Ropdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Ropdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.10604" aria-labelledby="styles.10604_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.10604" value="10604"/></div><div class="hTbJa f H o"><label id="styles.10604_label" for="styles.10604" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Families with children</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.10614" aria-labelledby="styles.10614_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.10614" value="10614"/></div><div class="hTbJa f H o"><label id="styles.10614_label" for="styles.10614" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Romantic</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.11777" aria-labelledby="styles.11777_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.11777" value="11777"/></div><div class="hTbJa f H o"><label id="styles.11777_label" for="styles.11777" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Kids</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="styles.10607" aria-labelledby="styles.10607_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="styles.10607" value="10607"/></div><div class="hTbJa f H o"><label id="styles.10607_label" for="styles.10607" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Special occasions</span></span></span></label></div></span></div><div class="nDgtO A _c"><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button" aria-expanded="false" aria-controls="filter-expando-styles"><span class="biGQs _P ezezH">Show more<div class="biGQs _P avBIb ezezH"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></span></button></div></div></div></div></div></div></span><span><div class="CHHoy"><div role="group" class="eTICN Q2 _c"><div class="ZFYPe Q2 Fh u"><h3 class="biGQs _P ezezH">Features</h3> <button type="button" tabindex="0" aria-expanded="true" aria-controls=":lithium-Rqpdb8s9f9bvlq:" class="FSCFM _S B- Cq G-"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div id=":lithium-Rqpdb8s9f9bvlq:" class="rEDCr qRPyf"><div class="cvXsD _c"><div role="menu" class="cvXsD _c"><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.10852" aria-labelledby="diningOptions.10852_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.10852" value="10852"/></div><div class="hTbJa f H o"><label id="diningOptions.10852_label" for="diningOptions.10852" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Seating</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.16547" aria-labelledby="diningOptions.16547_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.16547" value="16547"/></div><div class="hTbJa f H o"><label id="diningOptions.16547_label" for="diningOptions.16547" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Table Service</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.10602" aria-labelledby="diningOptions.10602_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.10602" data-automation="reservationsFilter" value="10602"/></div><div class="hTbJa f H o"><label id="diningOptions.10602_label" for="diningOptions.10602" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Reservations</span></span></span></label></div></span><span class="VWnYD f V _T jMjBP"><div class="KKZFd P"><input id="diningOptions.10862" aria-labelledby="diningOptions.10862_label" type="checkbox" class="PMWyE _G _S Gn R2 B2 Z BF Vm _T wSSLS" aria-checked="false" name="diningOptions.10862" value="10862"/></div><div class="hTbJa f H o"><label id="diningOptions.10862_label" for="diningOptions.10862" class="vfpRs f _S"><span class="Rwgng"><span class="biGQs _P AWdfh"><span class="biGQs _P VImYz AWdfh">Serves Alcohol</span></span></span></label></div></span></div><div class="Mj"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Show all</span></button></div></div></div></div></div></span></div></div><div class="oqGNy" data-automation="LeftRailSlot"><div class="uGSLz f e"><div class="IUxgK"><div style="top:24px" class="CsVUk E"><div class="YcOBw mFLzV"><div class="Eufjb j u ChFsW Xd o S" id="slot:300x600:rail1"></div></div></div></div><div class="IUxgK"><div style="top:24px" class="CsVUk E"><div class="YcOBw mFLzV"><div class="Eufjb j u ChFsW Xd o S" id="slot:300x600:rail3"></div></div></div></div></div></div></div><div class="LvzMf _T" data-automation="LeftRailMain"><div class="klkOK"><!--$--><!--/$--><div class="rfqBq f e"><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-24855625"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Alma Verde Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556g6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556g6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->348<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Cafe</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.almaverde.com.mx/copia-de-copia-de-menu-tulum" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We ordered the <b>avocado toast</b> and the eggs Benedict, both were absolutely deli...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amazing breakfast</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10066419"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">La Popular - Nômade Temple Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gaalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->412<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://cutt.ly/gbJo5nL" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excellent <b>seafood</b> by the sea</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>ceviche</b> was a highlight.</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/ac/f1/d5/restaurant-interior-and.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-2286855"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">1. La Zebra Beach Restaurant And Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gealb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->6,446<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>4.6<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://lazebratulum.com/dining-2/#menu" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Provides a variety of rooms, restaurants, and <b>bar</b> as well as beach area to en...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Edgar, <b>tacos</b>, and drinks… amazing!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline1" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/d8/44/a9/our-new-look-come-check.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23483492"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">2. Fit-a-licious</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gialb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->914<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, International</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl"><b>Arepas</b>, sough dough French toast, quesadillas and amazing Gluten free salmon...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Had the <b>guacamole</b> and smoked salmon rye...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/22/34/80/phad-thai.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-1144210"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">3. Mezzanine Thai Restaurant &amp; Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gmalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gmalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->4,783<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>2<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Asian, Thai</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://mezzaninetulum.com/menus" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">As was my tofu <b>green curry</b>.</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... the Tulum <b>salad</b>, tortillas, som tam...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/30/f9/26/4d/estrada.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-12091598"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">4. Restaurante Estrada</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gqalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,566<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Caribbean, Latin</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Fantastic <b>seafood</b> restaurant on the way back from the cenotes - would highly...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... day was superb as was the <b>guacamole</b>.</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/1a/f1/b0/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27096664"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">5. Sexy Pizza Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556gualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556gualb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->406<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bar, Pizza</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Sexy Pizza with Great Variants</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Wonderful</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline2" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/28/91/35/f5/xibak-tulum.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="365" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-25294307"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Xibak Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556h2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556h2alb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->88<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">International</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://beta.menuflow.com/900001450" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Nice Dinner</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Perfect Dining Experience in Aldea Zama!</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/22/f4/4a/terrace-view.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27105003"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">6. Lula&#x27;s Kitchen</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556h6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556h6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->795<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, Mediterranean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://lulahoteltulum.com/eat-drink/" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">I highly recommend the shrimp <b>tacos</b>!</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>French toast</b>, bowls, and all of the...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/4c/45/2a/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23719230"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">7. Distrito Panamera</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556haalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556haalb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->175<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bar, Pizza</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>room</b> was very comfortable—the bed was nice and firm and the air condition...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Distrito Awesome!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/63/4d/59/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-20296875"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">8. La Negra Tomasa</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556healb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556healb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,240<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Brew Pub</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://lanegratomasatulum.webnode.mx/" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The surf-and-turf tostada, octopus, and fried <b>tuna</b> tacos were incredibly tend...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Beautiful place for family dinner</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline3" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/d8/44/e1/la-popular.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10066419"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">9. La Popular - Nômade Temple Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hialb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->412<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://cutt.ly/gbJo5nL" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excellent <b>seafood</b> by the sea</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>ceviche</b> was a highlight.</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/c9/d0/ad/restaurant.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10252068"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">10. Mi Amor Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hmalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hmalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->973<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">International, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.tulumhotelmiamor.com/restaurant-bar/#menu" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Paradise best <b>guacamole</b> in Tulum</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The best <b>cheesecake</b> dessert and amazing...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/28/a4/11/tantra-tulum.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-21408449"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">11. Tantra Tulum Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hqalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,678<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>3.7<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, International</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl"><b>Anniversary</b> at tantra tulum</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Christmas in Tulum</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/ef/ff/c4/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-24855625"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">12. Alma Verde Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556hualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556hualb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->348<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Cafe</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.almaverde.com.mx/copia-de-copia-de-menu-tulum" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We ordered the <b>avocado toast</b> and the eggs Benedict, both were absolutely deli...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amazing breakfast</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline4" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/6a/ea/b8/fish-tacos.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-15294346"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">13. Casa Maria</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556i2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556i2alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,049<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://qr.imenupro.com/hod-32" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We had grilled <b>octopus</b> and baja style fish tacos.</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... and mixed <b>ceviche</b> Mexican style!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/e0/3a/6c/piaggia-tulum-is-an-italo.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27477578"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">14. Piaggia Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556i6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556i6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->340<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Italian, Mediterranean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://toptablegroup.com/piaggia-tulum" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We had the açaí bowl and the <b>French toast</b>.</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Best pizza in Tulum</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/93/34/2d/fresco-s-restaurant-entrnace.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10434099"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">15. Fresco&#x27;s</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iaalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,101<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>3.9<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Latin</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="http://www.frescostulum.com/menus" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">AMAZING!! Best breakfast in tulum! This place is so cute and the service is...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Healthy, fresh and DELICIOUS! Immaculate vibes, impeccable service, and SO SO SO FRESH!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1d/1f/0e/98/parallel-20.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23432753"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Parallel 20° by Kimpton Aluna Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iealb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->90<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">International, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">He even prepared something out of menu for me! Alex in the morning buffet is...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... I think said he was from <b>Mexico City</b>.</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1e/5b/df/3c/view-of-the-restaurant.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23652674"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">16. Onyx Tulum Restaurant Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iialb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->728<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The <b>ceviche</b> and mushroom tacos especially were great, and the Onyx Mescalita...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We got the <b>guacamole</b> for an app, steak...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/17/8f/84/ed/nuestro-original-ceviche.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10169568"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">17. El Suspiro</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556imalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556imalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->163<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Latin</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Alejandro was our Server with immediate attention and very considerable.!!! Our...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... pescado a <b>la parrilla</b> y el ceviche:...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/02/ff/db/it-s-a-great-place-to.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-26265692"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">18. Golden Cham Thai</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iqalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->321<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, Asian</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We had shrimp steamed <b>dumplings</b> & Thai...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Fabulous in all ways!</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/d9/f1/51/infinity-pool-at-atico.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23180563"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">19. Ático Rooftop Bar &amp; Lounge</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556iualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556iualb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->158<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The restaurant and rooftop staff were also fabulous and great to chat with...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Great Service</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline5" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/0c/1b/38/best-beach-club-in-town.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23780639"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">20. Rosanegra Beach Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556j2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556j2alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->238<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>4.2<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Latin, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">A perfect day at the <b>tulum beach</b></span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">GREAT!</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/6c/ee/98/la-mejor-cocina-mexicana.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="374" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23459171"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">21. Cervecería Tulum Brewery Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556j6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556j6alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,493<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">The lager is delicious and the <b>tuna</b> tiradito is killer.</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Good dj, nice ambiance and decoration...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/de/72/c3/bio-natural-tulum.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23524261"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">22. Bio-natural Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jaalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->305<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Contemporary</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://bio-natural.mx/en/menu/" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vegan friendly! Thanks to Harry and Antonio great people and very friendly ☺️</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">5 star for Bio natural</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/23/3f/c7/e1/confessions.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23280755"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">23. Confessions Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jealb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->997<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bars &amp; Pubs, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Hands down best night scene in tulum ask for them!! Confessions is an amazing...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Long and large <b>bar</b>.</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline6" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/7c/05/1c/caption.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-26128325"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">24. Copal Gastro Bar</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5.0</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jialb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jialb8s9f9bvlq:">5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->665<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://copalgastrobar.com/wp-content/uploads/2023/09/SPRECIO-COPAL-MENU-INGLES.pdf" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Fantastic fish <b>tacos</b></span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Amazing</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/8b/3c/58/staff.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-19458762"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">25. Curry Thai Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jmalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jmalb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->732<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Asian, Thai</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Great <b>Thai</b> food in Tulum and Fermin was a fantastic waiter, super attentive,...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We’ve stayed in tulum now for a few...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/1e/3d/74/beach-club.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-1775007"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">26. Ziggy Beach Club &amp; Restaurant</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.5</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jqalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jqalb8s9f9bvlq:">4.5 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->2,186<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>3.9<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Caribbean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.ziggybeachtulum.com/menu" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Stayed at the beach tulum and ate at ziggy’s for lunch and dinner many times...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">For the main dish had the <b>octopus</b>, grea...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/33/b4/1c/main-entrance-to-mamazzita.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23259804"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">27. Mamazzita Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556jualb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556jualb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,184<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Bars &amp; Pubs, Mexican</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Closed now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="http://bit.ly/2PV0Uln" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Great service, Carlos was our server. The ambiance and atmosphere is really...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... wavy <b>meatballs</b>, prime rib tacos & s...</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline7" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=700&amp;h=-1&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/b3/9e/f9/locos-at-loco-tulum-km.jpg?w=400&amp;h=-1&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-13201059"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">28. Loco Tulum Mediterranean Kitchen</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.9</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556k2alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556k2alb8s9f9bvlq:">4.9 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->961<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div><div class="ryXnC f e Wi NM"><div class="biGQs _P VImYz ZNjnF"><span class="MwRQW f u q"><span class="PV"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M4.25 9.799c0-4.247 3.488-7.707 7.75-7.707s7.75 3.46 7.75 7.707c0 2.28-1.138 4.477-2.471 6.323-1.31 1.813-2.883 3.388-3.977 4.483l-.083.083-.002.002-1.225 1.218-1.213-1.243-.03-.03-.012-.013c-1.1-1.092-2.705-2.687-4.035-4.53-1.324-1.838-2.452-4.024-2.452-6.293M12 3.592c-3.442 0-6.25 2.797-6.25 6.207 0 1.796.907 3.665 2.17 5.415 1.252 1.736 2.778 3.256 3.886 4.357l.043.042.16.164.148-.149.002-.002.061-.06c1.103-1.105 2.605-2.608 3.843-4.322 1.271-1.76 2.187-3.64 2.187-5.445 0-3.41-2.808-6.207-6.25-6.207m1.699 5.013a1.838 1.838 0 1 0-3.397 1.407A1.838 1.838 0 0 0 13.7 8.605m-2.976-2.38a3.338 3.338 0 1 1 2.555 6.168 3.338 3.338 0 0 1-2.555-6.169"></path></svg></span>5.1<!-- --> mi</span></div><div class="biGQs _P VImYz ZNjnF">Tulum Beach</div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Seafood, Mediterranean</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">We enjoyed the pita & <b>hummus</b>, tuna tataki, shrimp tacos, and Caesar salad—eve...</span></div></a><a target="_blank" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">I’m very speechless how was the service...</span></div></a></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/a4/5d/94/shrimp-burger.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-19855557"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">29. Las Hijas De La Tostada Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556k6alb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556k6alb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->178<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican, Seafood</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://las-hijas-de-la-tostada-4.ola.click/products" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Not too sweet and quality ingredients. I had a hibiscus/mescal drink and a...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... and salmon <b>tostada</b> and both were su...</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/61/87/d9/lounge.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-21248033"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div><div class="cnQjJ z fBZpu"><div class="" role="img" aria-label="Travelers&#x27; Choice 2025 Winner" data-automation="travelersChoice-2025"><img class="TPuys GTDUB EAoGl" aria-hidden="true" src="https://static.tacdn.com/img2/brand_refresh_2025/travelers_choice/badges/2025/2025_TC_Badge_Large.svg"/></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">30. Funky Geisha Tulum</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.8</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556kaalb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556kaalb8s9f9bvlq:">4.8 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->1,457<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Japanese, Sushi</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://www.funkygeisha.com.mx/en/menu-comida-asiatica-thai-japonesa" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">... the Panang curry and all of our <b>sushi</b> (order the kimchi <b>sushi</b>!) was delic...</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">A great restaurant in Tulum</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline8" class="Eufjb j u ChFsW Xd o S nFcpI"></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><div></div><a href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R afQPz eXZKw"><source media="(max-width: 1023px)" srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=700&amp;h=700&amp;s=1 2x"/><img srcSet="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/26/f5/5a/6b/octopus-with-mezcal-pairing.jpg?w=400&amp;h=400&amp;s=1" width="400" height="400" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li><li class="CyFNY _A"><picture class="NhWcC _R afQPz eXZKw"><img src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" width="100%" height="100%" style="object-fit:fill" alt="" loading="lazy"/></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Previous Photo" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Next Photo" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" style="width:126px" data-clicksource="IndicatorDot" data-automation="indicatorDots-0"><div class="gqkHJ u f HfKGv" style="transform:translate3d(0px, 0, 0) scale3d(0, 0, 1)"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform:translate3d(20px, 0, 0) scale3d(1.3, 1.3, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(40px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(60px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(80px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(100px, 0, 0) scale3d(1, 1, 1)"></div><div class="gqkHJ u f HfKGv" style="transform:translate3d(120px, 0, 0) scale3d(0, 0, 1)"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Save to a trip" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-23597112"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">Mamazul Tulum Mezcaleria</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4.6</span></div></span><div class="nKWJn u qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby=":lithium-R556kealb8s9f9bvlq:" data-automation="bubbleRatingImage"><title id=":lithium-R556kealb8s9f9bvlq:">4.6 of 5 bubbles</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(<!-- -->52<!-- --> reviews<!-- -->) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Mexican</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Open now</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Menu</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Best <b>tacos</b> ever</span></div></a><a target="_blank" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Mezcal yes please</span></div></a></div></div><div class="lZsUu"><div class="hJhgY cpoJM"><span class="biGQs _P navcl">Sponsored</span></div></div><div class="bGLbH"><div class=""><div class="WKHYu"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ MHYEA" href="/Restaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html" target="_blank" data-automation="orderButton"><span class="biGQs _P ezezH">Reserve</span></a></div></div></div></div></div></div></div></div><div class="qAZoU c"><div class="GMYGA"><div class="mkNRT j"><div class="HXmEd"><div class="IGLCo"><a class="BrOJk u j z _F _S wSSLS tIqAi unMkR" data-smoke-attr="pagination-next-arrow" aria-label="Next page" href="/Restaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></a></div></div></div><div class="LaMdn j"><div class="aMnVW"><div class="Yzhnw P"><button class="BrOJk u j z _F _S wSSLS tIqAi iNBVo SSqtP" type="button" disabled="" aria-label="1"><span class="kLqdM"><span class="biGQs _P ezezH">1</span></span></button></div><div class="Yzhnw P"><a class="BrOJk u j z _F _S wSSLS tIqAi iNBVo" aria-label="2" href="/Restaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html"><span class="kLqdM"><span class="biGQs _P ezezH">2</span></span></a></div><div class="Yzhnw P"><div class="biGQs _P ezezH"><div class="sUgyD">…</div></div></div></div></div></div></div><div class="qAZoU c"><div class="biGQs _P VImYz ZNjnF"><div class="Ci">Showing results <!-- -->1<!-- -->-<!-- -->30<!-- --> of <!-- -->397</div></div></div></div></div></div></div></div></div><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="Pk PY Px PK"><div class="oTksw EFdEd"><div id="slot:970x250-728x90:footer" class="Eufjb j u ChFsW Xd o S"></div></div></div></div></div><div data-automation="restaurant-list-jsonld"><script type="application/ld+json">{"@context":"https:\u002F\u002Fschema.org","@type":"ItemList","itemListOrder":"https:\u002F\u002Fschema.org\u002FItemListOrderAscending","itemListElement":[{"@type":"ListItem","position":1,"item":{"@type":"Restaurant","name":"La Zebra Beach Restaurant And Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":6446},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002Fac\u002Ff1\u002Fd5\u002Frestaurant-interior-and.jpg"],"telephone":"+52 984 115 4726","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum A Boca Paila Km 8.2 La Zebra Hotel, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.14566,"longitude":-87.45949}}},{"@type":"ListItem","position":2,"item":{"@type":"Restaurant","name":"Fit-a-licious","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":914},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F23\u002Fd8\u002F44\u002Fa9\u002Four-new-look-come-check.jpg"],"telephone":"+52 984 176 0165","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Aldea Zama 26, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.199438,"longitude":-87.46239}}},{"@type":"ListItem","position":3,"item":{"@type":"Restaurant","name":"Mezzanine Thai Restaurant & Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":4783},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F25\u002F22\u002F34\u002F80\u002Fphad-thai.jpg"],"telephone":"+52 984 115 4728","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum-Boca Paila Zona Hotelera, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.197395,"longitude":-87.43733}}},{"@type":"ListItem","position":4,"item":{"@type":"Restaurant","name":"Restaurante Estrada","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1566},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F30\u002Ff9\u002F26\u002F4d\u002Festrada.jpg"],"telephone":"+52 984 124 0011","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Calle Centauro Sur Entre Gama Oriente y Asteroide Oriente, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.209045,"longitude":-87.45844}}},{"@type":"ListItem","position":5,"item":{"@type":"Restaurant","name":"Sexy Pizza Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":406},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2c\u002F1a\u002Ff1\u002Fb0\u002Fcaption.jpg"],"telephone":"+52 984 177 7336","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77776","streetAddress":"av Tulum 16 Av Principal de Tulum, Tulum 77776 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.212246,"longitude":-87.4588}}},{"@type":"ListItem","position":6,"item":{"@type":"Restaurant","name":"Lula's Kitchen","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":795},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002F22\u002Ff4\u002F4a\u002Fterrace-view.jpg"],"telephone":"+52 984 210 3028","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Carr. Tulum-Boca Paila km 8.1 Lula Seaside Boutique Hotel, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.145996,"longitude":-87.45929}}},{"@type":"ListItem","position":7,"item":{"@type":"Restaurant","name":"Distrito Panamera","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":175},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002F4c\u002F45\u002F2a\u002Fcaption.jpg"],"telephone":"+529982277902","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila Km 8. 5, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.212597,"longitude":-87.458786}}},{"@type":"ListItem","position":8,"item":{"@type":"Restaurant","name":"La Negra Tomasa","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1240},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1c\u002F63\u002F4d\u002F59\u002Fphoto0jpg.jpg"],"telephone":"+52 984 156 9555","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Andromeda Oriente # 14, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.211016,"longitude":-87.459915}}},{"@type":"ListItem","position":9,"item":{"@type":"Restaurant","name":"La Popular - N\u00F4made Temple Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":412},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002Fd8\u002F44\u002Fe1\u002Fla-popular.jpg"],"telephone":"+52 984 745 0314","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77880","streetAddress":"Carr. Tulum-Boca Paila Km 10.5 Tulum Beach, Tulum 77880 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.13606,"longitude":-87.46304}}},{"@type":"ListItem","position":10,"item":{"@type":"Restaurant","name":"Mi Amor Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":973},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F29\u002Fc9\u002Fd0\u002Fad\u002Frestaurant.jpg"],"telephone":"+52 1 984 188 4273","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila Km. 4.1 Zona Hotelera, Parque Nacional, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.194103,"longitude":-87.43926}}},{"@type":"ListItem","position":11,"item":{"@type":"Restaurant","name":"Tantra Tulum Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1678},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2a\u002F28\u002Fa4\u002F11\u002Ftantra-tulum.jpg"],"telephone":"+52 998 690 0252","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila KM 7 Tantra Bohemian Luxury Beach Club, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.158932,"longitude":-87.455475}}},{"@type":"ListItem","position":12,"item":{"@type":"Restaurant","name":"Alma Verde Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":348},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2a\u002Fef\u002Fff\u002Fc4\u002Fcaption.jpg"],"telephone":"+52 984 231 7473","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"M 2 Av. Cob\u00E1 Sur Lote 3 Plaza Hunab, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.203936,"longitude":-87.45003}}},{"@type":"ListItem","position":13,"item":{"@type":"Restaurant","name":"Casa Maria","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1049},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002F6a\u002Fea\u002Fb8\u002Ffish-tacos.jpg"],"telephone":"+52 984 876 2115","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Boca Paila Km 2.5 Diamante Hotel & Resturant, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.195839,"longitude":-87.43832}}},{"@type":"ListItem","position":14,"item":{"@type":"Restaurant","name":"Piaggia Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":340},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002Fe0\u002F3a\u002F6c\u002Fpiaggia-tulum-is-an-italo.jpg"],"telephone":"+52 984 310 7074","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Calla Gamma Ote Mz 4 Lote 1, Local 8a, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.212048,"longitude":-87.45109}}},{"@type":"ListItem","position":15,"item":{"@type":"Restaurant","name":"Fresco's","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1101},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2f\u002F93\u002F34\u002F2d\u002Ffresco-s-restaurant-entrnace.jpg"],"telephone":"+52 984 871 1141","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum-Boca Paila km 7 Zona Hotelera, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.156944,"longitude":-87.45616}}},{"@type":"ListItem","position":16,"item":{"@type":"Restaurant","name":"Onyx Tulum Restaurant Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":728},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1e\u002F5b\u002Fdf\u002F3c\u002Fview-of-the-restaurant.jpg"],"telephone":"+52 984 209 1788","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Calle Ori\u00F3n Sur M4 L-009, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.211458,"longitude":-87.460495}}},{"@type":"ListItem","position":17,"item":{"@type":"Restaurant","name":"El Suspiro","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":163},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F17\u002F8f\u002F84\u002Fed\u002Fnuestro-original-ceviche.jpg"],"telephone":"+52 998 279 1370","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Avenida Coba, Mz 36, lote 4 y 5 Mz 36, Lote 4 y 5, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.214872,"longitude":-87.45359}}},{"@type":"ListItem","position":18,"item":{"@type":"Restaurant","name":"Golden Cham Thai","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":321},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2a\u002F02\u002Fff\u002Fdb\u002Fit-s-a-great-place-to.jpg"],"telephone":"+52 984 120 3206","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Av. Tulum Entre Saturno y Luna Sur, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.209072,"longitude":-87.46888}}},{"@type":"ListItem","position":19,"item":{"@type":"Restaurant","name":"\u00C1tico Rooftop Bar & Lounge","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":158},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1c\u002Fd9\u002Ff1\u002F51\u002Finfinity-pool-at-atico.jpg"],"telephone":"+52 984 689 0506","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Mza 05 Av. Coba Lot 01, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.205343,"longitude":-87.450386}}},{"@type":"ListItem","position":20,"item":{"@type":"Restaurant","name":"Rosanegra Beach Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":238},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F22\u002F0c\u002F1b\u002F38\u002Fbest-beach-club-in-town.jpg"],"telephone":"+52 998 690 0283","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-boca Paila 5-km 7, Tulum Beach Zona Hotelera, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.151724,"longitude":-87.45831}}},{"@type":"ListItem","position":21,"item":{"@type":"Restaurant","name":"Cervecer\u00EDa Tulum Brewery Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1493},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002F6c\u002Fee\u002F98\u002Fla-mejor-cocina-mexicana.jpg"],"telephone":"+52 984 218 0506","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77789","streetAddress":"Carretera Canc\u00FAn - Chetumal Km. 230 Parque Nacional Del Jaguar De Tulum, Tulum 77789 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.232265,"longitude":-87.42345}}},{"@type":"ListItem","position":22,"item":{"@type":"Restaurant","name":"Bio-natural Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":305},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F23\u002Fde\u002F72\u002Fc3\u002Fbio-natural-tulum.jpg"],"telephone":"+52 984 168 6138","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Av Coba Plaza Mayan Monkey, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.204794,"longitude":-87.45029}}},{"@type":"ListItem","position":23,"item":{"@type":"Restaurant","name":"Confessions Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":997},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F23\u002F3f\u002Fc7\u002Fe1\u002Fconfessions.jpg"],"telephone":"+52 998 690 0275","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum- Boca Paila Zona Hotelera, Km. 7, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.159094,"longitude":-87.4552}}},{"@type":"ListItem","position":24,"item":{"@type":"Restaurant","name":"Copal Gastro Bar","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"5","reviewCount":665},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F29\u002F7c\u002F05\u002F1c\u002Fcaption.jpg"],"telephone":"+52 984 181 6970","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Calle Ixchel esquina Av. Juanek, Aldea Zama, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.20358,"longitude":-87.46014}}},{"@type":"ListItem","position":25,"item":{"@type":"Restaurant","name":"Curry Thai Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":732},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002F8b\u002F3c\u002F58\u002Fstaff.jpg"],"telephone":"+52 984 239 3265","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"C. 7 Sur La Veleta, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.202757,"longitude":-87.47367}}},{"@type":"ListItem","position":26,"item":{"@type":"Restaurant","name":"Ziggy Beach Club & Restaurant","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.5","reviewCount":2186},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2e\u002F1e\u002F3d\u002F74\u002Fbeach-club.jpg"],"telephone":"+52 984 871 1145","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Carr. Tulum-Boca Paila km 7.5 Zona Hotelera, Tulum Beach, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.156145,"longitude":-87.45642}}},{"@type":"ListItem","position":27,"item":{"@type":"Restaurant","name":"Mamazzita Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":1184},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F22\u002F33\u002Fb4\u002F1c\u002Fmain-entrance-to-mamazzita.jpg"],"telephone":"+52 998 690 0274","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum Boca Paila, Km. 7 Zona Hotelera, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.15775,"longitude":-87.45616}}},{"@type":"ListItem","position":28,"item":{"@type":"Restaurant","name":"Loco Tulum Mediterranean Kitchen","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.9","reviewCount":961},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1a\u002Fb3\u002F9e\u002Ff9\u002Flocos-at-loco-tulum-km.jpg"],"telephone":"+52 984 309 0149","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carretera Tulum Boca Paila Km 9.3, Tulum Beach, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.137993,"longitude":-87.46335}}},{"@type":"ListItem","position":29,"item":{"@type":"Restaurant","name":"Las Hijas De La Tostada Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":178},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F2b\u002Fa4\u002F5d\u002F94\u002Fshrimp-burger.jpg"],"telephone":"+52 984 235 0690","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77760","streetAddress":"Tulum Centro, Col Huracanes, Tulum 77760 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.211218,"longitude":-87.461555}}},{"@type":"ListItem","position":30,"item":{"@type":"Restaurant","name":"Funky Geisha Tulum","description":"","url":"https:\u002F\u002Fwww.tripadvisor.com\u002FRestaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html?m=69411","aggregateRating":{"@type":"AggregateRating","ratingValue":"4.8","reviewCount":1457},"priceRange":"$$ - $$$","image":["https:\u002F\u002Fdynamic-media-cdn.tripadvisor.com\u002Fmedia\u002Fphoto-o\u002F1c\u002F61\u002F87\u002Fd9\u002Flounge.jpg"],"telephone":"+52 998 690 0255","address":{"@type":"PostalAddress","addressCountry":"Mexico","addressLocality":"","addressRegion":"","postalCode":"77780","streetAddress":"Carr. Tulum-Boca Paila Km 7.8 Tulum Beach - Hotel Zone, Tulum 77780 Mexico"},"geo":{"@type":"GeoCoordinates","latitude":20.149506,"longitude":-87.4594}}}]}</script></div><!--/$--></main><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="Kfkql Pg PN Pt PA Cj"><div class="Pg" data-test-target="global-nav-stacked"><div class=""><button aria-controls=":lithium-Rb9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-Rb9j9bvlq:" class="Duuud"><a href="/Tourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Tourism</span></a><a href="/Hotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotels</span></a><a href="/Hotels-g150813-c2-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Bed and Breakfast</span></a><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Restaurants</span></a><a href="/Attractions-g150813-Activities-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Things to Do in Tulum</span></a><a href="/ShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Travel Forum</span></a><a href="/LocationPhotos-g150813-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Photos</span></a><a href="/LocalMaps-g150813-Tulum-Area.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Map</span></a></div><button aria-controls=":lithium-Rj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-Rj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Tulum Hotels</span></a><a href="/SmartDeals-g150813-Tulum_Yucatan_Peninsula-Hotel-Deals.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotel Deals</span></a><a href="/LastMinute-g150813-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Last Minute Hotels in Tulum</span></a><a href="/HotelsNearMe" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels Near Me</span></a><button aria-controls=":lithium-R2qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">By Hotel Type</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R2qj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfd2-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Motels</span></a><a href="/Hotels-g150813-c3-zff26-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hostels</span></a><a href="/Hotels-g150813-c3-zff29-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Campgrounds</span></a><a href="/Hotels-g150813-zff105-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Glamping Stays in Tulum</span></a><a href="/Hotels-g150813-zff10-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Beach Hotels</span></a><a href="/Hotels-g150813-zff7-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Business Hotels</span></a><a href="/Hotels-g150813-zff4-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Family Hotels</span></a><a href="/Hotels-g150813-zff24-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Green Hotels</span></a><a href="/Hotels-g150813-zff8-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Resorts</span></a><a href="/Hotels-g150813-zff12-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Luxury Hotels</span></a><a href="/Hotels-g150813-zff3-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Romantic Hotels in Tulum</span></a><a href="/Hotels-g150813-zff14-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Casinos</span></a><a href="/Hotels-g150813-zff13-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Spa Resorts</span></a></div><button aria-controls=":lithium-R3aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">By Hotel Class</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R3aj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfc5-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">5-Star Hotels in Tulum</span></a><a href="/Hotels-g150813-zfc3-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">3-Star Hotels in Tulum</span></a><a href="/Hotels-g150813-zfc4-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">4-Star Hotels in Tulum</span></a></div><button aria-controls=":lithium-R3qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">By Hotel Brand</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R3qj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfb22174-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Ahau Collection Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb11779-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">OYO Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb11763-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Luxury Bahia Principe Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb16281-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Oasis Hotels &amp; Resorts in Tulum</span></a><a href="/Hotels-g150813-zfb9205-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">InterContinental (IHG) Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb9208-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Marriott Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb11845-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Colibri Boutique Hotels in Tulum</span></a><a href="/Hotels-g150813-zfb9204-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hilton Hotels in Tulum</span></a></div><button aria-controls=":lithium-R4aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Amenities</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R4aj9j9bvlq:" class="Duuud"><a href="/Hotels-g150813-zfa9-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Pet Friendly Hotels in Tulum</span></a><a href="/Hotels-g150813-zfa3-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotels with Pools</span></a><a href="/Hotels-g150813-zfa7-Tulum_Yucatan_Peninsula-Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Hotels with Free Parking</span></a></div><button aria-controls=":lithium-R4qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Tulum Categories</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R4qj9j9bvlq:" class="Duuud"><a href="/HotelsList-Tulum-Boutique-Hotels-zfp1512.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Boutique Hotels in Tulum</span></a><a href="/HotelsList-Tulum-Cheap-Hotels-zfp11040.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Discount Hotels in Tulum</span></a><a href="/HotelsList-Tulum-Honeymoon-Resorts-zfp13996.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Honeymoon Resorts in Tulum</span></a><a href="/HotelsList-Tulum-Wedding-Hotels-zfp16067.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Wedding Hotels in Tulum</span></a><a href="/HotelsList-Tulum-All-Inclusive-Hotels-zfp41115.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Inclusive Hotels in Tulum</span></a><a href="/HotelsList-Tulum-All-Inclusive-Resorts-zfp1189394.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Inclusive Resorts in Tulum</span></a><a href="/HotelsList-Tulum-Adults-Only-And-Adult-Friendly-Resorts-zfp28929004.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Adults Only and Adult Friendly Resorts Tulum</span></a><a href="/HotelsList-Tulum-Golf-Hotels-zfp28929053.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Golf Hotels</span></a><a href="/HotelsList-Tulum-Singles-Resorts-zfp28929102.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tulum Singles Resorts</span></a><a href="/HotelsList-Tulum-Spring-Break-Resorts-zfp28929151.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Spring Break Resorts in Tulum</span></a></div><button aria-controls=":lithium-R5aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Landmarks</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R5aj9j9bvlq:" class="Duuud"><a href="/HotelsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Tulum Archaeological Site</span></a><a href="/HotelsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Akumal Beach</span></a><a href="/HotelsNear-g150813-d1478385-Cenotes_Dos_Ojos-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Cenotes Dos Ojos</span></a><a href="/HotelsNear-g150813-d3321137-Playa_Paraiso-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Playa Paraiso</span></a><a href="/HotelsNear-g150813-d669578-Gran_Cenote-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Gran Cenote</span></a><a href="/HotelsNear-g499445-d153288-Yal_ku_Lagoon-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Yal-ku Lagoon</span></a><a href="/HotelsNear-g499445-d153913-Aktun_Chen-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Aktun Chen</span></a><a href="/HotelsNear-g499445-d10441459-Akumal_Monkey_Sanctuary_Rescued_Animals-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Akumal Monkey Sanctuary &amp; Rescued Animals</span></a><a href="/HotelsNear-g150813-d3387166-Cenotes_Sac_Actun-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Cenotes Sac Actun</span></a><a href="/HotelsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near The Yacht Experiences</span></a></div><button aria-controls=":lithium-R5qj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Airports</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R5qj9j9bvlq:" class="Duuud"><a href="/HotelsNear-g150807-qCUN-Cancun_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Cancun Airport (CUN)</span></a><a href="/HotelsNear-g150813-qTQO-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Felipe Carrillo Puerto International Airport (TQO)</span></a><a href="/HotelsNear-g150812-qPCM-Playa_del_Carmen_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hotels near Aerodromo de Playa Del Carmen (PCM)</span></a><a href="/HotelsNear-g150812-qPCM-zfd2-Playa_del_Carmen_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Motels near Aerodromo de Playa Del Carmen (PCM)</span></a></div><button aria-controls=":lithium-R6aj9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Hotel Categories</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R6aj9j9bvlq:" class="Duuud"><a href="/HotelCategory-zft9435-Ryokan.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Ryokan</span></a><a href="/HotelCategory-zft6216-Family_Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Family Hotels</span></a><a href="/HotelCategory-zft6216,9672-Family_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Family Resorts</span></a><a href="/HotelCategory-zft9672,19075-Ski_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Ski Resorts</span></a><a href="/HotelCategory-zft9672,20284-Singles_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Singles Resorts</span></a><a href="/HotelCategory-zft9172,9189-All_Inclusive_Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular All-Inclusive Hotels</span></a><a href="/HotelCategory-zft9624,9672-Honeymoon_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Honeymoon Resorts</span></a><a href="/HotelCategory-zft6216,9172,9672-All_Inclusive_Family_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular All-Inclusive Family Resorts</span></a><a href="/HotelCategory-zft9172,9672,18884-Adults_Only_All_Inclusive_Resorts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Adults Only All-Inclusive Resorts</span></a><a href="/HotelCategory-zft9171-Casino_Hotels.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Casino Hotels</span></a></div></div><a href="/Attractions-g150813-Activities-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Things to Do</span></a><button aria-controls=":lithium-R139j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R139j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">All Tulum Restaurants</span></a><button aria-controls=":lithium-R1b39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Types of Food</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R1b39j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-c10698-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Argentinian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10686-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Street Food Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10631-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Peruvian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10772-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Hawaiian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10679-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Healthy Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10639-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Latin Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c10646-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Fast Food Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfz10992-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Gluten Free Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfz10665-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Vegetarian Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfz10697-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Vegan Restaurants in Tulum</span></a><a href="/Restaurants-g150813-c36-zfp6-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Spanish Restaurants with Outdoor Seating in Tulum</span></a><a href="/Restaurants-g150813-c37-zfp8-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Late Night Steakhouses in Tulum</span></a><a href="/Restaurants-g150813-c38-zfp30-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Sushi Restaurants for Lunch in Tulum</span></a></div><button aria-controls=":lithium-R1r39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Dishes</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R1r39j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-zfd10758-Tulum_Yucatan_Peninsula-Donuts.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Donuts in Tulum</span></a><a href="/Restaurants-g150813-zfd10902-Tulum_Yucatan_Peninsula-French_Toast.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best French Toast in Tulum</span></a><a href="/Restaurants-g150813-zfd10890-Tulum_Yucatan_Peninsula-Chow_Mein.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Chow Mein in Tulum</span></a><a href="/Restaurants-g150813-zfd11724-Tulum_Yucatan_Peninsula-Unagi_Dojo_Eel_Soil.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Unagi &amp; Dojo (Eel &amp; Soil) in Tulum</span></a><a href="/Restaurants-g150813-zfd19960-Tulum_Yucatan_Peninsula-Cuban_Sandwich.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Cuban Sandwich in Tulum</span></a><a href="/Restaurants-g150813-zfd20040-Tulum_Yucatan_Peninsula-Mac_and_cheese.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Mac and cheese in Tulum</span></a><a href="/Restaurants-g150813-zfd20336-Tulum_Yucatan_Peninsula-Steak_Frites.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Steak Frites in Tulum</span></a><a href="/Restaurants-g150813-zfd10878-Tulum_Yucatan_Peninsula-Burrito.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Burritos in Tulum</span></a><a href="/Restaurants-g150813-zfd10905-Tulum_Yucatan_Peninsula-Gyros.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Gyros in Tulum</span></a><a href="/Restaurants-g150813-zfd10894-Tulum_Yucatan_Peninsula-Crab_Cakes.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Crab Cakes in Tulum</span></a><a href="/Restaurants-g150813-zfd20183-Tulum_Yucatan_Peninsula-Red_Curry.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Red Curry in Tulum</span></a><a href="/Restaurants-g150813-zfd20156-Tulum_Yucatan_Peninsula-Veggie_Burger.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Veggie Burger in Tulum</span></a><a href="/Restaurants-g150813-zfd20137-Tulum_Yucatan_Peninsula-Tres_Leches.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Tres Leches in Tulum</span></a><a href="/Restaurants-g150813-zfd20161-Tulum_Yucatan_Peninsula-Cucumber_Salad.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Cucumber Salad in Tulum</span></a><a href="/Restaurants-g150813-zfd20189-Tulum_Yucatan_Peninsula-Lettuce_Wraps.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Best Lettuce Wraps in Tulum</span></a></div><button aria-controls=":lithium-R2b39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Popular Restaurant Categories</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R2b39j9bvlq:" class="Duuud"><a href="/Restaurants-g150813-zfp2-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Breakfast Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp30-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Lunch Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp58-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Dinner Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp43-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants with a View in Tulum</span></a><a href="/Restaurants-g150813-zfp9-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants for Group Dining in Tulum</span></a><a href="/Restaurants-g150813-zfp49-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Buffet Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp8-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Late Night Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp6-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants with Outdoor Seating in Tulum</span></a><a href="/Restaurants-g150813-zfp19-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Food Delivery Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfp5-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Kid Friendly Restaurants in Tulum</span></a><a href="/Restaurants-g150813-zfg9901-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Bakeries in Tulum</span></a><a href="/Restaurants-g150813-zfg9900-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Coffee &amp; Tea in Tulum</span></a><a href="/Restaurants-g150813-zfg9909-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Desserts in Tulum</span></a></div><button aria-controls=":lithium-R2r39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Hotels</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R2r39j9bvlq:" class="Duuud"><a href="/RestaurantsNear-g150813-d7223463-Lula_Seaside_Boutique_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Lula Seaside Boutique Hotel </span></a><a href="/RestaurantsNear-g150813-d23825874-Wakax_Hacienda_Cenote_Boutique_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Wakax Hacienda - Cenote &amp; Boutique Hotel</span></a><a href="/RestaurantsNear-g150813-d19006698-Hotel_Ma_xanab_Tulum-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Hotel Ma&#x27;xanab Tulum</span></a><a href="/RestaurantsNear-g150813-d7183451-Mi_Amor_Colibri_Boutique_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Mi Amor Colibri Boutique Hotel</span></a><a href="/RestaurantsNear-g150813-d557443-Mezzanine_Tulum_a_Small_Luxury_Hotel-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Mezzanine Tulum, a Small Luxury Hotel</span></a><a href="/RestaurantsNear-g150813-d17495992-Kanan_Tulum-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Kanan Tulum</span></a><a href="/RestaurantsNear-g23240074-d296667-La_Zebra_an_SLH_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near La Zebra, an SLH Hotel</span></a><a href="/RestaurantsNear-g499445-d8285376-Secrets_Akumal_Riviera_Maya-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Secrets Akumal Riviera Maya</span></a><a href="/RestaurantsNear-g23240074-d557040-Suenos_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Suenos Tulum</span></a><a href="/RestaurantsNear-g23240074-d3979959-The_Beach_Tulum_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near The Beach Tulum Hotel</span></a></div><button aria-controls=":lithium-R3b39j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Near Landmarks</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R3b39j9bvlq:" class="Duuud"><a href="/RestaurantsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Tulum Archaeological Site</span></a><a href="/RestaurantsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Akumal Beach</span></a><a href="/RestaurantsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near The Yacht Experiences</span></a><a href="/RestaurantsNear-g150813-d23065077-My_Quest_Concierge_Private_Tours-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near My Quest Concierge Private Tours</span></a><a href="/RestaurantsNear-g150813-d23852094-Dlux_Transfer-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Dlux Transfer</span></a><a href="/RestaurantsNear-g499445-d23928640-Marine_Life_Akumal_MX-Akumal_Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Marine Life Akumal MX</span></a><a href="/RestaurantsNear-g150813-d23285262-Mayikal_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Mayikal Experiences</span></a><a href="/RestaurantsNear-g150813-d26344863-Parque_Ecoturistico_Cenote_Aktun_Bej-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Parque Ecoturistico Cenote Aktun Bej</span></a><a href="/RestaurantsNear-g150813-d26394539-Tulum_Experiences-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Tulum Experiences</span></a><a href="/RestaurantsNear-g150813-d20967421-Playacht_Private_Yacht_Rentals-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Restaurants near Playacht - Private Yacht Rentals</span></a></div></div><a href="/Articles" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Travel Stories</span></a><a href="/Cruises" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Cruises</span></a><button aria-controls=":lithium-R1r9j9bvlq:" aria-expanded="false" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">More</span><span class="EuuxD u ND S4 H0"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></span></button><div id=":lithium-R1r9j9bvlq:" class="Duuud"><a href="/Attractions-g150813-Activities-c42-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Tours</span></a><a href="/AddListing" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Add a Place</span></a><a href="/ShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Travel Forum</span></a><a href="/Airlines" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Airlines</span></a><a href="/TravelersChoice" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Travelers&#x27; Choice</span></a><a href="https://www.tripadvisorsupport.com/hc/en-us" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Help Center</span></a><a href="/Trips" class="ZcAuY Cj _T _S G_ B-"><span class="biGQs _P ZNjnF">Trip Planner</span></a></div></div></div></div></div><footer data-component="ta-brand-footer" class="Gz Cj Pl PN Py PA"><div class="IDaDx mvTrV cyIij fluiI SMjpI Iwmxp"><div class="dLJNv"><div class="MfOiD"><div class="MQpxt f _Y"><picture><img class="HAOGn F0 Nf" src="https://static.tacdn.com/img2/brand_refresh_2025/logos/logo.svg" alt="Tripadvisor"/></picture><div><div class="CDuLn"><div class="biGQs _P VImYz navcl">© <!-- -->2025<!-- --> Tripadvisor LLC<!-- --> <!-- -->All rights reserved.</div></div><div class="VPSQM f K"><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="https://tripadvisor.mediaroom.com/us-terms-of-use" rel="noopener" target="_self"><span class="biGQs _P ezezH">Terms of Use</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="https://tripadvisor.mediaroom.com/us-privacy-policy" rel="noopener" target="_self"><span class="biGQs _P ezezH">Privacy and Cookies Statement</span></a><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" type="button"><span class="biGQs _P ezezH">Cookie consent</span></button><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="/SiteIndex" rel="noopener" target="_self"><span class="biGQs _P ezezH">Site Map</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="/pages/serviceEN.html" rel="noopener" target="_self"><span class="biGQs _P ezezH">How the site works</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="https://tripadvisor.mediaroom.com/US-contact-us" rel="noopener" target="_blank"><span class="biGQs _P ezezH">Contact us</span></a><a class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG raEkE" href="/Trust-lgF5hKLTqw3U.html" rel="noopener" target="_blank"><span class="biGQs _P ezezH">Accessibility Statement</span></a></div></div></div><div class="_T FKffI bmUTE"><div class="fIrGe _T bgMZj" style="-webkit-line-clamp:2;line-break:normal;cursor:auto"><div class="biGQs _P VImYz ZNjnF"><span class="JguWG"><p>This is the version of our website addressed to speakers of <!-- -->English<!-- --> in <!-- -->the United States<!-- -->. If you are a resident of another country or region, please select the appropriate version of Tripadvisor for your country or region in the drop-down menu.</p><br/><br/><p>Tripadvisor  LLC makes no guarantees for availability of prices advertised on our sites and applications. Listed prices may require a stay of a particular length or have blackout dates, qualifications or other applicable restrictions. Tripadvisor LLC is not responsible for any content on external web sites that are not owned or operated by Tripadvisor .<!-- --> <!-- -->Indicative hotel prices displayed on our “Explore” pages are estimates extrapolated from historic pricing data.</p><br/><br/><p>Tripadvisor LLC is not a booking agent or tour operator. When you book with one of our partners, please be sure to check their site for a full disclosure of all applicable fees.</p><br/><br/></span></div></div><div class="lszDU"><button class="UikNM _G B- _S _W u _T c G_ wSSLS wnNQG" type="button"><span class="biGQs _P ezezH">Read more</span><div class="CECjK _W"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></div></button></div></div></div><div class="sVVKe"><div class="JLKop w"><button type="button" class="EVXMG _G f k Q2 u _u w Gn Z B2 BF Cq Ra _S wSSLS" aria-haspopup="listbox" aria-label="Currency: $ USD"><span class="biGQs _P ezezH">$ USD</span><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div><div class="JLKop w"><button type="button" class="EVXMG _G f k Q2 u _u w Gn Z B2 BF Cq Ra _S wSSLS" aria-haspopup="menu" aria-label="Region: United States"><span class="biGQs _P ezezH">United States</span><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path d="M18.4 7.4 12 13.7 5.6 7.4 4.2 8.8l7.8 7.8 7.8-7.8z"></path></svg></button></div></div></div></div></footer><!--$--><!--/$--><!--$--><!--/$--><!--/$--></div><script nonce="884ba8d9-c881-4482-b7ef-5b279351627a" async src="data:text/javascript,(this.%24WP%3Dthis.%24WP%7C%7C%5B%5D).push(function(e)%7Bvar%20t%3De.clientEntrypoint%3Breturn%20delete%20e.clientEntrypoint%2C%5B%22bootstrap%22%2Cfunction(t%2Cn)%7Bvar%20r%3Breturn%5Bfunction()%7Br(document.getElementById(%22lithium-root%22)%2Ce%2Cnull%3D%3D%3De.lithiumBootstrap.renderError)%2C%22serviceWorker%22in%20navigator%26%26navigator.serviceWorker.getRegistrations().then(function(e)%7Be.forEach(function(e)%7Be.unregister()%7D)%7D)%2Cwindow.indexedDB%26%26indexedDB.deleteDatabase%26%26indexedDB.deleteDatabase(%22keyval-store%22)%7D%2C%5Bfunction(e)%7Br%3De.default%7D%5D%5D%7D%2C%5Bt%5D%5D%7D(JSON.parse(%22%7B%5C%22userAgent%5C%22%3A%7B%5C%22userAgentCategory%5C%22%3A%5C%22DESKTOP%5C%22%2C%5C%22os%5C%22%3A%7B%5C%22family%5C%22%3A%5C%22WINDOWS%5C%22%2C%5C%22majorVersion%5C%22%3A-1%7D%2C%5C%22browser%5C%22%3A%7B%5C%22family%5C%22%3A%5C%22CHROME%5C%22%2C%5C%22majorVersion%5C%22%3A143%7D%2C%5C%22isWebview%5C%22%3Afalse%2C%5C%22crawlerCategory%5C%22%3Anull%2C%5C%22nativeVersion%5C%22%3Anull%7D%2C%5C%22domainName%5C%22%3A%5C%22www.tripadvisor.com%5C%22%2C%5C%22cdnPrefix%5C%22%3A%5C%22https%3A%2F%2Fstatic.tacdn.com%5C%22%2C%5C%22locale%5C%22%3A%5C%22en-US%5C%22%2C%5C%22userId%5C%22%3Anull%2C%5C%22sessionId%5C%22%3A%5C%222E720B5647256721025BB38391F1F4FC%5C%22%2C%5C%22route%5C%22%3A%5B%7B%5C%22page%5C%22%3A%5C%22Restaurants%5C%22%2C%5C%22params%5C%22%3A%7B%5C%22geoId%5C%22%3A150813%2C%5C%22diningOptionId%5C%22%3A10955%7D%2C%5C%22path%5C%22%3A%5C%22%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%22%2C%5C%22fragment%5C%22%3A%5C%22%5C%22%7D%2C%7B%5C%22uid%5C%22%3A%5C%22d98b22c8-c925-417f-963b-ea8d282aea40%5C%22%7D%5D%2C%5C%22cookies%5C%22%3A%7B%7D%2C%5C%22webviewCurrencyOverride%5C%22%3Anull%2C%5C%22serverSideCanonicalized%5C%22%3Atrue%2C%5C%22timeOrigin%5C%22%3A1766760733954%2C%5C%22documentInstalledChunkIds%5C%22%3A%5B%5C%22h2v4d7%5C%22%2C%5C%22iucilr%5C%22%2C%5C%22mg6599%5C%22%2C%5C%22pwxlu9%5C%22%2C%5C%22gg6fk5%5C%22%2C%5C%22bfou8q%5C%22%2C%5C%2221h32l%5C%22%2C%5C%22qndsds%5C%22%2C%5C%22o4yt4q%5C%22%2C%5C%2202sgqj%5C%22%2C%5C%22a349lq%5C%22%2C%5C%22lq6xsf%5C%22%2C%5C%22mb2nze%5C%22%2C%5C%2283pm0d%5C%22%2C%5C%22q18gbm%5C%22%2C%5C%229b9b10%5C%22%2C%5C%22973cci%5C%22%2C%5C%227thqz4%5C%22%2C%5C%22p5p2yo%5C%22%2C%5C%22fgwvq1%5C%22%2C%5C%22ex1xu3%5C%22%2C%5C%22qjohxf%5C%22%2C%5C%225vmpsm%5C%22%2C%5C%22ys6v1o%5C%22%2C%5C%22al4vil%5C%22%2C%5C%22cyrz7y%5C%22%2C%5C%22xo1is3%5C%22%2C%5C%2252far5%5C%22%2C%5C%22nnprhg%5C%22%2C%5C%226hf0eg%5C%22%2C%5C%22w1o2w8%5C%22%2C%5C%22iog21h%5C%22%2C%5C%2260mj7v%5C%22%2C%5C%22fkji80%5C%22%2C%5C%22w1vb4i%5C%22%2C%5C%22r73jw8%5C%22%2C%5C%229m9122%5C%22%2C%5C%22jppfwt%5C%22%2C%5C%22i0z8n1%5C%22%2C%5C%22qkjm8h%5C%22%2C%5C%22towdyv%5C%22%2C%5C%22cb79np%5C%22%2C%5C%22l7164z%5C%22%2C%5C%22vpo3wt%5C%22%2C%5C%22llqsfm%5C%22%2C%5C%22qlcuu5%5C%22%2C%5C%22fbuhsy%5C%22%2C%5C%222mrepu%5C%22%2C%5C%22s6q22n%5C%22%2C%5C%22nntg4s%5C%22%2C%5C%22zzfm0c%5C%22%2C%5C%226tkc5e%5C%22%2C%5C%22ih0468%5C%22%2C%5C%22nv7v96%5C%22%2C%5C%22ku7suo%5C%22%2C%5C%2294v69w%5C%22%2C%5C%22nv8ze5%5C%22%2C%5C%22g9ot0r%5C%22%2C%5C%22essqvy%5C%22%2C%5C%22gw8uw6%5C%22%2C%5C%2234bu7w%5C%22%2C%5C%22lr6tb6%5C%22%2C%5C%22f1x7n0%5C%22%2C%5C%22voa3hw%5C%22%2C%5C%22254k2b%5C%22%2C%5C%22mqwhqn%5C%22%2C%5C%22q7fbfw%5C%22%2C%5C%22ns7yrw%5C%22%2C%5C%2249c7br%5C%22%2C%5C%226nmgrj%5C%22%2C%5C%22hckdpv%5C%22%2C%5C%22fgi73t%5C%22%2C%5C%22b5nukb%5C%22%2C%5C%22kb5au4%5C%22%2C%5C%227dp9gx%5C%22%2C%5C%224uqz87%5C%22%2C%5C%22a7906a%5C%22%2C%5C%22eap1p5%5C%22%2C%5C%225j1idx%5C%22%2C%5C%227u7gib%5C%22%2C%5C%22q13wda%5C%22%2C%5C%22bp96i9%5C%22%2C%5C%2255gplo%5C%22%2C%5C%22b478w0%5C%22%2C%5C%22eyzj96%5C%22%2C%5C%22s2x8on%5C%22%2C%5C%22ueacbr%5C%22%2C%5C%22trf9tg%5C%22%2C%5C%22o3mm96%5C%22%2C%5C%22nbupuc%5C%22%2C%5C%22ezmx4w%5C%22%2C%5C%22dpzonz%5C%22%2C%5C%22w1yg77%5C%22%2C%5C%22jgmezl%5C%22%2C%5C%22p3y1zq%5C%22%2C%5C%2232pwc7%5C%22%2C%5C%22dkml3k%5C%22%2C%5C%22198ozm%5C%22%2C%5C%22p9ucm8%5C%22%2C%5C%221mwjfl%5C%22%2C%5C%22tsg6fc%5C%22%2C%5C%223w1vrt%5C%22%2C%5C%223btuy0%5C%22%2C%5C%22gwsdl1%5C%22%2C%5C%227yaz66%5C%22%2C%5C%22xg719z%5C%22%2C%5C%22uto6xl%5C%22%2C%5C%22rr68op%5C%22%2C%5C%22gzffp2%5C%22%2C%5C%227rx7bi%5C%22%2C%5C%22q4uvd4%5C%22%2C%5C%22covaqn%5C%22%2C%5C%22t3fmar%5C%22%2C%5C%22n6iz1t%5C%22%2C%5C%225n3iuq%5C%22%2C%5C%22vb6y54%5C%22%2C%5C%22epp8eq%5C%22%2C%5C%227nq86i%5C%22%2C%5C%22bxtwp3%5C%22%2C%5C%22zq967r%5C%22%2C%5C%224ilds0%5C%22%2C%5C%22ntn269%5C%22%2C%5C%22xm2jzo%5C%22%2C%5C%22my7i5t%5C%22%2C%5C%223yhbda%5C%22%2C%5C%22wt5b21%5C%22%2C%5C%22i3sz5h%5C%22%2C%5C%22frf6ac%5C%22%2C%5C%22kq4525%5C%22%2C%5C%22mqfj4j%5C%22%2C%5C%2200mpxl%5C%22%2C%5C%22la678j%5C%22%2C%5C%22pydmep%5C%22%2C%5C%2268fm7o%5C%22%2C%5C%222r4qug%5C%22%2C%5C%22p98jyo%5C%22%2C%5C%228e77a5%5C%22%2C%5C%22livpad%5C%22%2C%5C%22ac38c0%5C%22%2C%5C%22mhkl02%5C%22%2C%5C%22uf95jv%5C%22%2C%5C%221b5l5d%5C%22%2C%5C%22eq3hjy%5C%22%2C%5C%22dcz41w%5C%22%2C%5C%225rvmf1%5C%22%2C%5C%220a0jsz%5C%22%2C%5C%22dlu2yg%5C%22%2C%5C%22teh9fa%5C%22%2C%5C%229lrsvx%5C%22%2C%5C%222tx7su%5C%22%2C%5C%2275vfyw%5C%22%2C%5C%22u1kaei%5C%22%2C%5C%22556vjb%5C%22%2C%5C%22u9jlwp%5C%22%2C%5C%22etaac9%5C%22%2C%5C%22cmm6zr%5C%22%2C%5C%22ahn2ts%5C%22%2C%5C%22bzl252%5C%22%2C%5C%22ay1l3c%5C%22%2C%5C%220v7vxf%5C%22%2C%5C%22008v8e%5C%22%2C%5C%22ue3yko%5C%22%2C%5C%22it6k5j%5C%22%2C%5C%22og41x6%5C%22%2C%5C%225azdze%5C%22%2C%5C%22cwl85x%5C%22%2C%5C%22gfv5nk%5C%22%2C%5C%22lfeddg%5C%22%2C%5C%22u3wokd%5C%22%2C%5C%22227h6c%5C%22%2C%5C%220ad67e%5C%22%2C%5C%22lgvzhu%5C%22%2C%5C%22kqywgi%5C%22%2C%5C%22qx89h8%5C%22%2C%5C%22rdycz1%5C%22%2C%5C%22gsd8gu%5C%22%2C%5C%224euaes%5C%22%2C%5C%22nylcph%5C%22%2C%5C%22ljqkwp%5C%22%2C%5C%22s1ymgp%5C%22%2C%5C%22gjtatx%5C%22%2C%5C%22gudzb7%5C%22%2C%5C%22j56vim%5C%22%2C%5C%220khkbz%5C%22%2C%5C%229ygz84%5C%22%2C%5C%225jr1ur%5C%22%2C%5C%227a7jm3%5C%22%2C%5C%22sciq8z%5C%22%2C%5C%22dvdzap%5C%22%2C%5C%22cbdk2b%5C%22%2C%5C%22v3e972%5C%22%2C%5C%22s557se%5C%22%2C%5C%22aj7avv%5C%22%2C%5C%22rh2yho%5C%22%2C%5C%22r3rvh1%5C%22%2C%5C%22lme1i4%5C%22%2C%5C%22ou1xb8%5C%22%2C%5C%22gbnboa%5C%22%2C%5C%225gpqhx%5C%22%2C%5C%22q7wvkq%5C%22%2C%5C%22uvk21c%5C%22%2C%5C%22mtbtnn%5C%22%2C%5C%22gyzem5%5C%22%2C%5C%22ulejzu%5C%22%2C%5C%22j8lyt5%5C%22%2C%5C%22g05qlv%5C%22%2C%5C%22uujvhg%5C%22%2C%5C%22wstkum%5C%22%2C%5C%22t9nvqo%5C%22%2C%5C%22dsbyen%5C%22%2C%5C%22u5g3tb%5C%22%2C%5C%22mdhor2%5C%22%2C%5C%22sljmfx%5C%22%2C%5C%22r3kgcy%5C%22%2C%5C%22vjlqzu%5C%22%2C%5C%22doaw04%5C%22%2C%5C%2298inwa%5C%22%2C%5C%22xj2t7q%5C%22%2C%5C%22vumu0s%5C%22%2C%5C%22bt3bv2%5C%22%2C%5C%224x6ber%5C%22%2C%5C%22904jnz%5C%22%2C%5C%22b8gyi1%5C%22%2C%5C%22vl1wbo%5C%22%2C%5C%22tbkd43%5C%22%2C%5C%22btbawi%5C%22%2C%5C%228iun0t%5C%22%2C%5C%22nexzid%5C%22%2C%5C%22y9ubrl%5C%22%2C%5C%22tp1g68%5C%22%2C%5C%22t05m6z%5C%22%2C%5C%22j8mzb2%5C%22%2C%5C%22pwoff0%5C%22%2C%5C%22j77a03%5C%22%2C%5C%22evvugf%5C%22%2C%5C%22nlswq2%5C%22%2C%5C%22ue4abb%5C%22%2C%5C%22q213pt%5C%22%2C%5C%2298810w%5C%22%2C%5C%22sor0m3%5C%22%2C%5C%229w33la%5C%22%2C%5C%22sdpjgr%5C%22%2C%5C%22rdeq8t%5C%22%2C%5C%22te6757%5C%22%2C%5C%22cngeux%5C%22%2C%5C%22v1gmh9%5C%22%2C%5C%22nunr2w%5C%22%2C%5C%22c6w8qp%5C%22%2C%5C%221rki99%5C%22%2C%5C%22meqo19%5C%22%2C%5C%2222xnaq%5C%22%2C%5C%22uedfnc%5C%22%2C%5C%22a6i9w3%5C%22%2C%5C%226qmqhw%5C%22%2C%5C%22fa3gob%5C%22%2C%5C%2202kko1%5C%22%2C%5C%2248k7uc%5C%22%2C%5C%22nnmev3%5C%22%2C%5C%22ffrmny%5C%22%2C%5C%22kq4q2w%5C%22%2C%5C%2290bo0h%5C%22%2C%5C%22joj99y%5C%22%2C%5C%229zpurr%5C%22%2C%5C%22xp34yw%5C%22%2C%5C%227b6vjf%5C%22%2C%5C%22acx4g3%5C%22%2C%5C%22cra0rz%5C%22%2C%5C%22q5b1x4%5C%22%2C%5C%22e0avcx%5C%22%2C%5C%22ejmcf4%5C%22%2C%5C%22r144n1%5C%22%2C%5C%22bmouzl%5C%22%2C%5C%22tvte1l%5C%22%2C%5C%22rrxjvt%5C%22%2C%5C%2265rb8q%5C%22%2C%5C%22ltllz9%5C%22%2C%5C%22x9dv95%5C%22%2C%5C%2279a2um%5C%22%2C%5C%22mdvsf7%5C%22%2C%5C%2279x4p0%5C%22%2C%5C%22835myt%5C%22%2C%5C%22d8oes5%5C%22%2C%5C%22uzmmz7%5C%22%2C%5C%22ky3i2c%5C%22%2C%5C%22gcjubj%5C%22%2C%5C%22o5y7tg%5C%22%2C%5C%22yw8i09%5C%22%2C%5C%22u5pild%5C%22%2C%5C%229rb8bq%5C%22%2C%5C%223dy54m%5C%22%2C%5C%22eww825%5C%22%2C%5C%22nute5e%5C%22%2C%5C%22wenm84%5C%22%2C%5C%22lnxtb7%5C%22%2C%5C%22wgun1f%5C%22%2C%5C%22chbmyv%5C%22%2C%5C%22qumvh0%5C%22%2C%5C%22b4ge9e%5C%22%2C%5C%22b9nflw%5C%22%2C%5C%222rns1i%5C%22%2C%5C%22k3csuh%5C%22%2C%5C%22b9lpgd%5C%22%2C%5C%22t3d7hy%5C%22%2C%5C%22mvpl5y%5C%22%2C%5C%22a2xbq6%5C%22%2C%5C%224ecb70%5C%22%2C%5C%22j9pg61%5C%22%2C%5C%22iq7kpa%5C%22%2C%5C%22yau6qy%5C%22%2C%5C%22lt9m20%5C%22%2C%5C%22bckiht%5C%22%2C%5C%22gttbjc%5C%22%2C%5C%2243er8k%5C%22%2C%5C%22vgujbz%5C%22%2C%5C%22conwso%5C%22%2C%5C%22qhneqr%5C%22%2C%5C%22it954g%5C%22%2C%5C%22qbn8x2%5C%22%2C%5C%224q9l40%5C%22%2C%5C%22tv5det%5C%22%2C%5C%22vk1dz4%5C%22%2C%5C%22d4cn15%5C%22%2C%5C%22bljygq%5C%22%2C%5C%22af72y8%5C%22%2C%5C%22k64pus%5C%22%2C%5C%22hwdnu5%5C%22%2C%5C%227l5hpv%5C%22%2C%5C%22nc3gmf%5C%22%2C%5C%22l75rcq%5C%22%2C%5C%22zb2uaz%5C%22%2C%5C%227gslcl%5C%22%2C%5C%22n9cueu%5C%22%2C%5C%225hne4v%5C%22%2C%5C%22bxkng5%5C%22%2C%5C%22w5k17a%5C%22%2C%5C%22aqiall%5C%22%2C%5C%22hxluky%5C%22%2C%5C%22f7mzgd%5C%22%2C%5C%22dyfdn5%5C%22%2C%5C%22bwm4qt%5C%22%2C%5C%22eozxiw%5C%22%2C%5C%22sa2a6q%5C%22%2C%5C%22uc42ar%5C%22%2C%5C%229ccyt3%5C%22%2C%5C%22ov8w1o%5C%22%2C%5C%22h9085v%5C%22%2C%5C%22obrjv3%5C%22%2C%5C%22pts58q%5C%22%2C%5C%22rctp90%5C%22%2C%5C%220wbf8p%5C%22%2C%5C%22ed1cs2%5C%22%2C%5C%22bnv2ns%5C%22%2C%5C%22c5q3iu%5C%22%2C%5C%22ja55rt%5C%22%2C%5C%22jbz0uy%5C%22%2C%5C%22asj0sy%5C%22%2C%5C%22kt39pq%5C%22%2C%5C%22bkeq24%5C%22%2C%5C%22sw5ax0%5C%22%2C%5C%22w7fngn%5C%22%2C%5C%22iyqcwg%5C%22%2C%5C%22d6vj30%5C%22%2C%5C%22y0uq31%5C%22%2C%5C%22neutur%5C%22%2C%5C%22hjkifi%5C%22%2C%5C%222ksxja%5C%22%2C%5C%222rd76e%5C%22%2C%5C%222mqr6r%5C%22%2C%5C%22bpwwxo%5C%22%2C%5C%22p4a4ly%5C%22%2C%5C%22lfyiyn%5C%22%2C%5C%22ywfh04%5C%22%2C%5C%22eh8z4m%5C%22%2C%5C%22vgfso9%5C%22%2C%5C%22yjij17%5C%22%2C%5C%22hfmoib%5C%22%2C%5C%22yk7it8%5C%22%2C%5C%2274j83y%5C%22%2C%5C%22fbyv8z%5C%22%2C%5C%22nev0g8%5C%22%2C%5C%22u47wjb%5C%22%2C%5C%22wehxiu%5C%22%2C%5C%22ynqy3n%5C%22%2C%5C%220506zk%5C%22%2C%5C%22knab8m%5C%22%2C%5C%22dvcr7s%5C%22%2C%5C%22tj4q1a%5C%22%2C%5C%22an500m%5C%22%2C%5C%22vvrdxj%5C%22%2C%5C%22flcs8h%5C%22%2C%5C%222l7tdf%5C%22%2C%5C%22j1i9fl%5C%22%2C%5C%22x55bmb%5C%22%2C%5C%22lzos1d%5C%22%2C%5C%22fug67i%5C%22%2C%5C%22sokx6p%5C%22%2C%5C%22lwi7bf%5C%22%2C%5C%222gg3s2%5C%22%2C%5C%224pg8m6%5C%22%2C%5C%22pia9xj%5C%22%2C%5C%22mgk0ro%5C%22%2C%5C%22nfeum5%5C%22%2C%5C%222wld0u%5C%22%2C%5C%223309mt%5C%22%2C%5C%229heuct%5C%22%2C%5C%22o846n4%5C%22%2C%5C%22v5ce81%5C%22%2C%5C%22pra5l9%5C%22%2C%5C%2287i435%5C%22%2C%5C%22rkm46w%5C%22%2C%5C%22lt711y%5C%22%2C%5C%22qha8vq%5C%22%2C%5C%22s9vc2i%5C%22%2C%5C%2271uqev%5C%22%2C%5C%22mc5s4b%5C%22%2C%5C%22irfizf%5C%22%2C%5C%226g4avl%5C%22%2C%5C%22665v5s%5C%22%2C%5C%222tnrli%5C%22%2C%5C%22gol71b%5C%22%2C%5C%22fgamlw%5C%22%2C%5C%22ctn2v9%5C%22%2C%5C%223tcfrp%5C%22%2C%5C%22crdfjq%5C%22%2C%5C%22t9pxn7%5C%22%2C%5C%22tax17a%5C%22%2C%5C%2219jhx7%5C%22%2C%5C%22y74olp%5C%22%2C%5C%22euncuq%5C%22%2C%5C%229btuwa%5C%22%2C%5C%22h1hqsi%5C%22%2C%5C%22x53oqt%5C%22%2C%5C%229wtdt4%5C%22%2C%5C%22yr8sh8%5C%22%2C%5C%22eki1kt%5C%22%2C%5C%228ny8va%5C%22%2C%5C%22cbyjc7%5C%22%2C%5C%22tochuq%5C%22%2C%5C%22amsjo9%5C%22%2C%5C%221ycpp4%5C%22%2C%5C%223x7pav%5C%22%2C%5C%22dg00hx%5C%22%2C%5C%223vmom9%5C%22%2C%5C%22kepfi8%5C%22%2C%5C%220mfk1q%5C%22%2C%5C%22uc4lhh%5C%22%2C%5C%22ph948p%5C%22%2C%5C%22mklkzb%5C%22%2C%5C%22bw3tql%5C%22%2C%5C%22fdsghl%5C%22%2C%5C%22co2nzy%5C%22%2C%5C%22qpz1y2%5C%22%2C%5C%220rhj0x%5C%22%2C%5C%22b49gxf%5C%22%2C%5C%228cre3h%5C%22%2C%5C%22l2u3je%5C%22%2C%5C%22ib7588%5C%22%2C%5C%22voprus%5C%22%2C%5C%228ll3ol%5C%22%2C%5C%22ck5rom%5C%22%2C%5C%22p7jq7m%5C%22%2C%5C%221tem5o%5C%22%2C%5C%22b5kt6r%5C%22%2C%5C%22w43m8k%5C%22%2C%5C%22y9enps%5C%22%2C%5C%22n4zdla%5C%22%2C%5C%22633gr8%5C%22%2C%5C%226h4i0r%5C%22%2C%5C%22us1vwl%5C%22%2C%5C%225xqg72%5C%22%2C%5C%2270j2tf%5C%22%2C%5C%22xfufl0%5C%22%2C%5C%22o80y56%5C%22%2C%5C%22ypf49l%5C%22%2C%5C%22ti102j%5C%22%2C%5C%22d3i8k1%5C%22%2C%5C%22mxujx7%5C%22%2C%5C%22eekt6e%5C%22%2C%5C%22z2nh2s%5C%22%2C%5C%22mf5oe4%5C%22%2C%5C%22v42v3u%5C%22%2C%5C%2276j3t0%5C%22%5D%2C%5C%22cspNonce%5C%22%3A%5C%22884ba8d9-c881-4482-b7ef-5b279351627a%5C%22%2C%5C%22referrerHeader%5C%22%3Anull%2C%5C%22lithiumBootstrap%5C%22%3A%7B%5C%22deprecatedPersistentId%5C%22%3A%5C%221ed76492.3617.4c94.b83a.6a75ad8c9119.19B5B2526FB%5C%22%2C%5C%22version%5C%22%3A%5C%222385786954%5C%22%2C%5C%22debugTool%5C%22%3Afalse%2C%5C%22ssrLogs%5C%22%3A%5B%5D%2C%5C%22serviceCalls%5C%22%3A%5B%5D%2C%5C%22renderError%5C%22%3Anull%2C%5C%22stagingError%5C%22%3Anull%2C%5C%22staticServiceOverrides%5C%22%3A%7B%7D%2C%5C%22hmrEnabled%5C%22%3Afalse%7D%2C%5C%22clientEntrypoint%5C%22%3A%5C%22ib7588%5C%22%2C%5C%22urqlSsrData%5C%22%3A%7B%5C%22results%5C%22%3A%7B%5C%2265767282%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Internallinks_buildHreflangs%5C%5C%5C%22%3A%7B%5C%5C%5C%22hreflangs%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-GB%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.uk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-CA%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ca%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr-CA%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ffr.tripadvisor.ca%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22it%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.it%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.es%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22de%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.de%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.fr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ja%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.jp%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22zh-Hans%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcn.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-IN%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.in%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22sv%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.se%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22nl%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.nl%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22pt%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.br%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22tr%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.tr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22da%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.dk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-MX%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.mx%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-IE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ie%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ar%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Far.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ar-EG%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.eg%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22de-AT%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.at%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22el%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.gr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-AU%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.au%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-MY%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.my%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-NZ%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.nz%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-PH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.ph%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-SG%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.sg%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-ZA%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.za%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-AR%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.ar%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-CL%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.cl%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-CO%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-PE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.pe%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22es-VE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.ve%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22id%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.id%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22he%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.il%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ko%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.co.kr%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22nb%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fno.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22pt-PT%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.pt%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22ru%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ru%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22th%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fth.tripadvisor.com%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22vi%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.vn%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22zh-Hant%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.tw%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22de-CH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.ch%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr-CH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ffr.tripadvisor.ch%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22it-CH%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fit.tripadvisor.ch%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22en-HK%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fen.tripadvisor.com.hk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22fr-BE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ffr.tripadvisor.be%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22nl-BE%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.be%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22pos%5C%5C%5C%22%3A%5C%5C%5C%22zh-hk%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com.hk%2FRestaurants-g150813-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%5D%7D%7D%5C%22%7D%2C%5C%22156332295%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22currency%5C%5C%5C%22%3A%7B%5C%5C%5C%22code%5C%5C%5C%22%3A%5C%5C%5C%22USD%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22U.S.%20Dollars%5C%5C%5C%22%2C%5C%5C%5C%22symbol%5C%5C%5C%22%3A%5C%5C%5C%22%24%5C%5C%5C%22%7D%7D%5C%22%7D%2C%5C%221190521020%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22No%20results%20were%20found%20in%20Tulum%2C%20but%20here%20are%20some%20options%20nearby.%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%222772378239%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22Top%20restaurants%20in%20Tulum%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%223291687264%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22detail%5C%5C%5C%22%3A%7B%5C%5C%5C%22info%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%2C%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.21242%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.463547%7D%7D%7D%7D%5C%22%7D%2C%5C%224059846075%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22response%5C%5C%5C%22%3A%7B%5C%5C%5C%22__typename%5C%5C%5C%22%3A%5C%5C%5C%22RestaurantPresentation_NarrowGeoResponse%5C%5C%5C%22%2C%5C%5C%5C%22contentTitle%5C%5C%5C%22%3A%5C%5C%5C%22Top%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22restaurants%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A24855625%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Alma%20Verde%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.203936%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45003%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.almaverde.com.mx%2Fcopia-de-copia-de-menu-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A348%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2Fef%2Fff%2Fc4%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1152%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20231%207473%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22M%202%20Av.%20Cob%C3%A1%20Sur%20Lote%203%20Plaza%20Hunab%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20ordered%20the%20%EF%BF%B9avocado%20toast%EF%BF%BB%20and%20the%20eggs%20Benedict%2C%20both%20were%20absolutely%20deli...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Amazing%20breakfast%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%227750b59e-dde4-4fdc-8149-e1655a255ecd%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10066419%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Popular%20-%20N%C3%B4made%20Temple%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.13606%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46304%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcutt.ly%2FgbJo5nL%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22WGxlXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjYzMTI4NDQwJmdlbz0xMDA2NjQxOSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4SUxGd3licmVpVDQxZXhIZThSbHg1Tnc0bFZXVm5XZVlpZHNyWkVJS2FOOCZjcz0xNzRhMGQyOTY0ODc3YjczYjExNDE1ODZkNzQ4MmY4ZWFfUHp4%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A412%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fd8%2F44%2Fe1%2Fla-popular.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20745%200314%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277880%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%2010.5%20Tulum%20Beach%2C%20Tulum%2077880%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Excellent%20%EF%BF%B9seafood%EF%BF%BB%20by%20the%20sea%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10066419-r1001760708-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10066419-r1001760708-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9ceviche%EF%BF%BB%20was%20a%20highlight.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10066419-r1001883756-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10066419-r1001883756-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%220801772a-38e8-48f8-b065-f6148cfd254a%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A2286855%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Zebra%20Beach%20Restaurant%20And%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d2286855-Reviews-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.14566%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45949%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flazebratulum.com%2Fdining-2%2F%23menu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22QU84Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTcxODgyJmdlbz0yMjg2ODU1JmZyb209UmVzdGF1cmFudHMmYXJlYT1yZXNlcnZhdGlvbl9idXR0b24mc2xvdD0xJm1hdGNoSUQ9MSZvb3M9MCZjbnQ9MSZzaWxvPTQ1NTgyJmJ1Y2tldD05Nzk2NDEmbnJhbms9MSZjcmFuaz0xJmNsdD1SJnR0eXBlPVJlc3RhdXJhbnQmdG09MzQ2NjcyMzM0Jm1hbmFnZWQ9ZmFsc2UmY2FwcGVkPWZhbHNlJmdvc294PTlETXZpYkRUV2ROYUZlR1lMOFFUYWt5djFMQllaYUdwZUI0TUVIbi1odjJtWkJOMzc2RUhWQ0d1c1RJUnhtSjhvSHFDMldvRjBWNWEtclRpb0J6WjUwWW44NVo3d1pDRUpKclYxLVJDcm5nJmNzPTE0NzRhMzllNmM4YmY4ZmYzYTkzZDM5OTIxODQ4MDA2YV9kUlY%3D%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A6446%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fac%2Ff1%2Fd5%2Frestaurant-interior-and.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2240%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3360%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20115%204726%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum%20A%20Boca%20Paila%20Km%208.2%20La%20Zebra%20Hotel%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Provides%20a%20variety%20of%20rooms%2C%20restaurants%2C%20and%20%EF%BF%B9bar%EF%BF%BB%20as%20well%20as%20beach%20area%20to%20en...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d2286855-r1039942673-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d2286855-r1039942673-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Edgar%2C%20%EF%BF%B9tacos%EF%BF%BB%2C%20and%20drinks%E2%80%A6%20amazing!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d2286855-r1023181036-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d2286855-r1023181036-La_Zebra_Beach_Restaurant_And_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23483492%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Fit-a-licious%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23483492-Reviews-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.199438%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46239%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Flinktr.ee%2FFitaliciousmx%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2220%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A914%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F23%2Fd8%2F44%2Fa9%2Four-new-look-come-check.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1500%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20176%200165%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Aldea%20Zama%2026%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22%EF%BF%B9Arepas%EF%BF%BB%2C%20sough%20dough%20French%20toast%2C%20quesadillas%20and%20amazing%20Gluten%20free%20salmon...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23483492-r1009328024-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23483492-r1009328024-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Had%20the%20%EF%BF%B9guacamole%EF%BF%BB%20and%20smoked%20salmon%20rye...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23483492-r1040878747-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23483492-r1040878747-Fit_a_licious-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A1144210%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mezzanine%20Thai%20Restaurant%20%26%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d1144210-Reviews-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.197395%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.43733%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fmezzaninetulum.com%2Fmenus%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22eFZxXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTc5ODczJmdlbz0xMTQ0MjEwJmZyb209UmVzdGF1cmFudHMmYXJlYT1yZXNlcnZhdGlvbl9idXR0b24mc2xvdD0xJm1hdGNoSUQ9MSZvb3M9MCZjbnQ9MSZzaWxvPTQ1NTgyJmJ1Y2tldD05Nzk2NDEmbnJhbms9MSZjcmFuaz0xJmNsdD1SJnR0eXBlPVJlc3RhdXJhbnQmdG09MzQ2NjcyMzM0Jm1hbmFnZWQ9ZmFsc2UmY2FwcGVkPWZhbHNlJmdvc294PTlETXZpYkRUV2ROYUZlR1lMOFFUYWt5djFMQllaYUdwZUI0TUVIbi1odjJtWkJOMzc2RUhWQ0d1c1RJUnhtSjhyMHhUcjNzZjlFT0FaVHRxbTBOVklnUDBhdWxFcHhzSkVkLWhPcWtLcjJBJmNzPTE5NDk2ZDMwYmNhODFhODViNjA3MzdlMTIwYTk3NzJmZV9ZTVY%3D%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A4783%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F25%2F22%2F34%2F80%2Fphad-thai.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A628%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1200%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20115%204728%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum-Boca%20Paila%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22As%20was%20my%20tofu%20%EF%BF%B9green%20curry%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1144210-r1006758941-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1144210-r1006758941-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20the%20Tulum%20%EF%BF%B9salad%EF%BF%BB%2C%20tortillas%2C%20som%20tam...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1144210-r1027585849-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1144210-r1027585849-Mezzanine_Thai_Restaurant_Bar-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A12091598%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurante%20Estrada%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d12091598-Reviews-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.209045%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45844%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2221%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1566%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F30%2Ff9%2F26%2F4d%2Festrada.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1200%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1600%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20124%200011%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calle%20Centauro%20Sur%20Entre%20Gama%20Oriente%20y%20Asteroide%20Oriente%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Fantastic%20%EF%BF%B9seafood%EF%BF%BB%20restaurant%20on%20the%20way%20back%20from%20the%20cenotes%20-%20would%20highly...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d12091598-r980535510-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d12091598-r980535510-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20day%20was%20superb%20as%20was%20the%20%EF%BF%B9guacamole%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d12091598-r1041160852-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d12091598-r1041160852-Restaurante_Estrada-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A27096664%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Sexy%20Pizza%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d27096664-Reviews-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.212246%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.4588%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pizza%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pub%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beer%20restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A406%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2c%2F1a%2Ff1%2Fb0%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2096%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1179%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20177%207336%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277776%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22av%20Tulum%2016%20Av%20Principal%20de%20Tulum%2C%20Tulum%2077776%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Sexy%20Pizza%20with%20Great%20Variants%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27096664-r1043603613-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27096664-r1043603613-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Wonderful%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27096664-r1043598529-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27096664-r1043598529-Sexy_Pizza_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A25294307%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Xibak%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d25294307-Reviews-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.200266%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46343%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fbeta.menuflow.com%2F900001450%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22ZXFTXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjQ4MzAzNjk2Jmdlbz0yNTI5NDMwNyZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4RTVDanpfbmlqUGVQYjR4Ry15TFRZcnRhZVhxMVNfZmpMT21hZGNuV1lzYyZjcz0xNjMxNDQ1ZDNjOWI4YWIwMjUwMjk0NDAyOTA2NGMwMWFfTU5C%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2215%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A88%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F28%2F91%2F35%2Ff5%2Fxibak-tulum.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2691%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2953%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20310%204024%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Itzamna%20%23178%20Aldea%20Zama%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Nice%20Dinner%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d25294307-r992355448-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d25294307-r992355448-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Perfect%20Dining%20Experience%20in%20Aldea%20Zama!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d25294307-r986894307-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d25294307-r986894307-Xibak_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%22040742ce-8190-4923-be83-e9fbe6f5ee6a%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A27105003%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Lula's%20Kitchen%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d27105003-Reviews-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.145996%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45929%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flulahoteltulum.com%2Feat-drink%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22dVNmXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU3MjEyMzk1Jmdlbz0yNzEwNTAwMyZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4a1B1dmNiOFZWZndnWWZ2eVNNWmQzZkpaWGFlMzRZYUgySVk0STd4UTVKbyZjcz0xNWQ5YTYxZTcyYzkyZThmODlhNDE4ZmMxYWZkZjUzNzJfb25M%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A795%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2F22%2Ff4%2F4a%2Fterrace-view.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2199%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3298%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20210%203028%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20km%208.1%20Lula%20Seaside%20Boutique%20Hotel%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22I%20highly%20recommend%20the%20shrimp%20%EF%BF%B9tacos%EF%BF%BB!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27105003-r1036239399-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27105003-r1036239399-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9French%20toast%EF%BF%BB%2C%20bowls%2C%20and%20all%20of%20the...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27105003-r1007731489-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27105003-r1007731489-Lula_s_Kitchen-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23719230%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Distrito%20Panamera%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23719230-Reviews-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.212597%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.458786%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22TVJiXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMjAwNDkxJmdlbz0yMzcxOTIzMCZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4dHZPbXZieWNHQzBfdlNuMVNraFhlbkxrb1ZnZ2UxcXFPOS12MW51ZnRpbyZjcz0xYzg5NTY0YTE5ZDVkMjdlMjhlMjRhYjIwMmMxNWZlNmVfcnV4%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pizza%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pub%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A45%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2218%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A175%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2F4c%2F45%2F2a%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1536%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B529982277902%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%208.%205%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9room%EF%BF%BB%20was%20very%20comfortable%E2%80%94the%20bed%20was%20nice%20and%20firm%20and%20the%20air%20condition...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23719230-r1038375088-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23719230-r1038375088-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Distrito%20Awesome!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23719230-r1043395800-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23719230-r1043395800-Distrito_Panamera-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A20296875%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Negra%20Tomasa%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d20296875-Reviews-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.211016%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.459915%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flanegratomasatulum.webnode.mx%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brew%20Pub%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beer%20restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1240%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1c%2F63%2F4d%2F59%2Fphoto0jpg.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2048%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20156%209555%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Andromeda%20Oriente%20%23%2014%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20surf-and-turf%20tostada%2C%20octopus%2C%20and%20fried%20%EF%BF%B9tuna%EF%BF%BB%20tacos%20were%20incredibly%20tend...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d20296875-r1041265282-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d20296875-r1041265282-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Beautiful%20place%20for%20family%20dinner%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d20296875-r1043402272-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d20296875-r1043402272-La_Negra_Tomasa-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10066419%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22La%20Popular%20-%20N%C3%B4made%20Temple%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.13606%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46304%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcutt.ly%2FgbJo5nL%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22WTRtXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjYzMTI4NDQwJmdlbz0xMDA2NjQxOSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4SUxGd3licmVpVDQxZXhIZThSbHg1Tnc0bFZXVm5XZVlpZHNyWkVJS2FOOCZjcz0xNzRhMGQyOTY0ODc3YjczYjExNDE1ODZkNzQ4MmY4ZWFfWVpP%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A412%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fd8%2F44%2Fe1%2Fla-popular.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20745%200314%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277880%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%2010.5%20Tulum%20Beach%2C%20Tulum%2077880%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Excellent%20%EF%BF%B9seafood%EF%BF%BB%20by%20the%20sea%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10066419-r1001760708-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10066419-r1001760708-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9ceviche%EF%BF%BB%20was%20a%20highlight.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10066419-r1001883756-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10066419-r1001883756-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10252068%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mi%20Amor%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10252068-Reviews-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.194103%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.43926%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tulumhotelmiamor.com%2Frestaurant-bar%2F%23menu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22YXAyXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTg5OTQ4Jmdlbz0xMDI1MjA2OCZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4aEFVRTRBbWdOWXluZl96TEVCb3pheXJKNU9POHBYUlczZDdscEVEbTRHWSZjcz0xZGQ2N2I5MzA5NTc2MjYzMDNiMGFmODJjZmI4MDJlOTBfQlhB%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A973%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F29%2Fc9%2Fd0%2Fad%2Frestaurant.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1492%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2880%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%201%20984%20188%204273%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km.%204.1%20Zona%20Hotelera%2C%20Parque%20Nacional%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Paradise%20best%20%EF%BF%B9guacamole%EF%BF%BB%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10252068-r876554607-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10252068-r876554607-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20best%20%EF%BF%B9cheesecake%EF%BF%BB%20dessert%20and%20amazing...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10252068-r1032238441-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10252068-r1032238441-Mi_Amor_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A21408449%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tantra%20Tulum%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d21408449-Reviews-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.158932%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.455475%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tantrabeachclub.com%2Fen%2Fmenu%3Futm_source%3Dtripadvisor%26utm_medium%3Dorganic%26utm_campaign%3Dmenu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22Qm9zXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU5MDk0MTI4Jmdlbz0yMTQwODQ0OSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4TS1OMG15SDVGNFdlWF90RzR1UkdxV21BaTItdDhYQTNyRWZQZVRqamg2USZjcz0xOGRhMTJhOWE4NjU1YTkwMzdjZTNjN2YzNTVjNDdiMzVfOG9H%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1678%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2F28%2Fa4%2F11%2Ftantra-tulum.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1080%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1080%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200252%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20KM%207%20Tantra%20Bohemian%20Luxury%20Beach%20Club%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22%EF%BF%B9Anniversary%EF%BF%BB%20at%20tantra%20tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d21408449-r1023510138-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d21408449-r1023510138-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Christmas%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d21408449-r1043677535-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d21408449-r1043677535-Tantra_Tulum_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A24855625%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Alma%20Verde%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.203936%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45003%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.almaverde.com.mx%2Fcopia-de-copia-de-menu-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A348%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2Fef%2Fff%2Fc4%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1152%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20231%207473%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22M%202%20Av.%20Cob%C3%A1%20Sur%20Lote%203%20Plaza%20Hunab%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20ordered%20the%20%EF%BF%B9avocado%20toast%EF%BF%BB%20and%20the%20eggs%20Benedict%2C%20both%20were%20absolutely%20deli...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1005678062-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Amazing%20breakfast%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d24855625-r1040228052-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A15294346%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Casa%20Maria%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d15294346-Reviews-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.195839%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.43832%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fqr.imenupro.com%2Fhod-32%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22YzJpXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTc1MzI5Jmdlbz0xNTI5NDM0NiZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4R0kxV0tmYzJVYnlQekNGRXNaMWJKNllfNFBlTTZLVW1YNGFNcTZfUXlNVSZjcz0xNjQxZjAxM2ViYzRkMmY5ZDhlY2VkMDA3NzUzMDYyMTFfQmFq%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1049%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2F6a%2Fea%2Fb8%2Ffish-tacos.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4024%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6048%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20876%202115%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Boca%20Paila%20Km%202.5%20Diamante%20Hotel%20%26%20Resturant%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20had%20grilled%20%EF%BF%B9octopus%EF%BF%BB%20and%20baja%20style%20fish%20tacos.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d15294346-r1010070146-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d15294346-r1010070146-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20and%20mixed%20%EF%BF%B9ceviche%EF%BF%BB%20Mexican%20style!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d15294346-r1013051733-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d15294346-r1013051733-Casa_Maria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A27477578%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Piaggia%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d27477578-Reviews-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.212048%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45109%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ftoptablegroup.com%2Fpiaggia-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22RU1RXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU3OTUzMzU4Jmdlbz0yNzQ3NzU3OCZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4SXBiSlBod0JQM3ZFa3BzZ3RpWThfWTlzQUNoblFmT0xuY05lSzlDeXJ4byZjcz0xNmNiOTkzNDU5MTNlOGZjMzk0NDQyYTIzYWNiMjI4YjNfQjQ1%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Italian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A340%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2Fe0%2F3a%2F6c%2Fpiaggia-tulum-is-an-italo.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A3000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20310%207074%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calla%20Gamma%20Ote%20Mz%204%20Lote%201%2C%20Local%208a%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20had%20the%20a%C3%A7a%C3%AD%20bowl%20and%20the%20%EF%BF%B9French%20toast%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27477578-r1018922684-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27477578-r1018922684-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Best%20pizza%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d27477578-r1042532282-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d27477578-r1042532282-Piaggia_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10434099%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Fresco's%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d10434099-Reviews-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.156944%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45616%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Fwww.frescostulum.com%2Fmenus%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22blJSXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTg4MDI1Jmdlbz0xMDQzNDA5OSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4SWQwR3ZpUlUzQzJJd1o5RTNxbzdYOE1pRVJwS3BzMVR4bG5qN3RJREw3OCZjcz0xNzIxZmI1ZDBmOGU1ZDg2NzI1ZWJlYmRhYjJiMTg2ZGJfNE01%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1101%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2f%2F93%2F34%2F2d%2Ffresco-s-restaurant-entrnace.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1280%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1920%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20871%201141%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum-Boca%20Paila%20km%207%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22AMAZING!!%20Best%20breakfast%20in%20tulum!%20This%20place%20is%20so%20cute%20and%20the%20service%20is...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d10434099-r789410173-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d10434099-r789410173-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%2C%20fresh%20and%20DELICIOUS!%20Immaculate%20vibes%2C%20impeccable%20service%2C%20and%20SO%20SO%20SO%20FRESH!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d10434099-r1042265342-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d10434099-r1042265342-Fresco_s-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23432753%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Parallel%2020%C2%B0%20by%20Kimpton%20Aluna%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23432753-Reviews-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.201447%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45102%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22R2c0Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTkzNzQwJmdlbz0yMzQzMjc1MyZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4S2M5UDVycExIVlFzSUtZQ2dhMW1XVUFfZ2tJa2IzS2FGaWltNHBoZWFoZyZjcz0xMjYzYTU5NDhiNzMyYjRjYTEzMzExZWI2ZDM3YTIwZTZfZ0Nj%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fusion%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20Bar%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2213%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A90%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1d%2F1f%2F0e%2F98%2Fparallel-20.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1972%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2732%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20689%200430%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Av%20Itzamna%20Poligono%200014%20Lote%201-11%2C%20Aldea%20Zama%20Hotel%20Kimpton%20Aluna%20Tulum%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22He%20even%20prepared%20something%20out%20of%20menu%20for%20me!%20Alex%20in%20the%20morning%20buffet%20is...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23432753-r791998099-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23432753-r791998099-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20I%20think%20said%20he%20was%20from%20%EF%BF%B9Mexico%20City%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23432753-r875671659-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23432753-r875671659-Parallel_20_by_Kimpton_Aluna_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%228c909865-c3fe-4f79-a09d-cd6625f0a860%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23652674%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Onyx%20Tulum%20Restaurant%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23652674-Reviews-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.211458%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.460495%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2202%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2212%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A728%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1e%2F5b%2Fdf%2F3c%2Fview-of-the-restaurant.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4032%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3024%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20209%201788%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calle%20Ori%C3%B3n%20Sur%20M4%20L-009%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20%EF%BF%B9ceviche%EF%BF%BB%20and%20mushroom%20tacos%20especially%20were%20great%2C%20and%20the%20Onyx%20Mescalita...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23652674-r1011720733-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23652674-r1011720733-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20got%20the%20%EF%BF%B9guacamole%EF%BF%BB%20for%20an%20app%2C%20steak...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23652674-r1014804001-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23652674-r1014804001-Onyx_Tulum_Restaurant_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A10169568%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22El%20Suspiro%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d10169568-Reviews-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.214872%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45359%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A163%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F17%2F8f%2F84%2Fed%2Fnuestro-original-ceviche.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A3744%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A5616%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20279%201370%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Avenida%20Coba%2C%20Mz%2036%2C%20lote%204%20y%205%20Mz%2036%2C%20Lote%204%20y%205%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Alejandro%20was%20our%20Server%20with%20immediate%20attention%20and%20very%20considerable.!!!%20Our...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10169568-r769759628-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10169568-r769759628-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20pescado%20a%20%EF%BF%B9la%20parrilla%EF%BF%BB%20y%20el%20ceviche%3A...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d10169568-r1031485099-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d10169568-r1031485099-El_Suspiro-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A26265692%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Golden%20Cham%20Thai%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d26265692-Reviews-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.209072%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46888%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soups%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2214%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A321%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2a%2F02%2Fff%2Fdb%2Fit-s-a-great-place-to.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A960%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1280%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20120%203206%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Av.%20Tulum%20Entre%20Saturno%20y%20Luna%20Sur%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20had%20shrimp%20steamed%20%EF%BF%B9dumplings%EF%BF%BB%20%26%20Thai...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26265692-r1024085297-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26265692-r1024085297-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Fabulous%20in%20all%20ways!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26265692-r1033578153-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26265692-r1033578153-Golden_Cham_Thai-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23180563%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22%C3%81tico%20Rooftop%20Bar%20%26%20Lounge%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23180563-Reviews-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.205343%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.450386%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gastropub%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A59%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A158%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1c%2Fd9%2Ff1%2F51%2Finfinity-pool-at-atico.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1333%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A2000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20689%200506%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Mza%2005%20Av.%20Coba%20Lot%2001%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20restaurant%20and%20rooftop%20staff%20were%20also%20fabulous%20and%20great%20to%20chat%20with...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23180563-r792599062-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23180563-r792599062-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Great%20Service%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23180563-r892064524-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23180563-r892064524-Atico_Rooftop_Bar_Lounge-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23780639%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Rosanegra%20Beach%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d23780639-Reviews-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.151724%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45831%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Frosanegrabeachclub.com%2Fen%2Fbest-lunch-in-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22aEVwXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTgxMzAzJmdlbz0yMzc4MDYzOSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4ZkZ2THpGbXNRZzJHeDd3RE04bnpqM2Z6UjVoaWRzbXAxQ2JBeUZjTVYtYyZjcz0xMmMzMDgxZDlmODY1NTg2YTZhNDQ3M2YwZjE3Yjc0OTFfeFcz%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2211%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A238%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F22%2F0c%2F1b%2F38%2Fbest-beach-club-in-town.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A3283%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A4925%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200283%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-boca%20Paila%205-km%207%2C%20Tulum%20Beach%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22A%20perfect%20day%20at%20the%20%EF%BF%B9tulum%20beach%EF%BF%BB%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d23780639-r972205014-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d23780639-r972205014-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22GREAT!%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d23780639-r1014112900-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d23780639-r1014112900-Rosanegra_Beach_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23459171%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Cervecer%C3%ADa%20Tulum%20Brewery%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23459171-Reviews-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.232265%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.42345%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Ftulumcerveceriaartesanal.com%2Fmenu-tulum%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22clBPXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTgzNzI3Jmdlbz0yMzQ1OTE3MSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4ZmZiMWQzcUJmSktibDZPNnZtbmgxQlZrNVdyT1FQV1ZkT3NwYUE5ck9wayZjcz0xYzRhYTZlYTI2ZWE2OTZhNjdkZmZkYTM5ZTZjMjk1ZWZfajVT%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2200%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1493%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2F6c%2Fee%2F98%2Fla-mejor-cocina-mexicana.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1400%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1500%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20218%200506%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277789%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Canc%C3%BAn%20-%20Chetumal%20Km.%20230%20Parque%20Nacional%20Del%20Jaguar%20De%20Tulum%2C%20Tulum%2077789%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22The%20lager%20is%20delicious%20and%20the%20%EF%BF%B9tuna%EF%BF%BB%20tiradito%20is%20killer.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23459171-r988921293-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23459171-r988921293-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Good%20dj%2C%20nice%20ambiance%20and%20decoration...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23459171-r800587562-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23459171-r800587562-Cerveceria_Tulum_Brewery_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23524261%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Bio-natural%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23524261-Reviews-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.204794%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45029%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fbio-natural.mx%2Fen%2Fmenu%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soups%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fruit%20parlours%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Medicinal%20foods%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A305%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F23%2Fde%2F72%2Fc3%2Fbio-natural-tulum.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4032%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A3024%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20168%206138%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Av%20Coba%20Plaza%20Mayan%20Monkey%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Vegan%20friendly!%20Thanks%20to%20Harry%20and%20Antonio%20great%20people%20and%20very%20friendly%20%E2%98%BA%EF%B8%8F%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23524261-r800511136-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23524261-r800511136-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%225%20star%20for%20Bio%20natural%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23524261-r901154099-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23524261-r901154099-Bio_natural_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23280755%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Confessions%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23280755-Reviews-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.159094%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.4552%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.confessions.com.mx%2Fmixologia-bares-en-cancun-tulum%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22dWpaXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU5MDk0MDk4Jmdlbz0yMzI4MDc1NSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4eVpFVjJSZFdrUUxzcjlpT1lIWDRfRlozcTIzTDR1MWlQUHJWVmJaQTRyYyZjcz0xMWU1M2E3ZjY1MjMxNDAxZGQzZGFlNjZkOWFkYzAzMmVfRHNT%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A11776%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bars%20%26%20Pubs%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2219%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A997%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F23%2F3f%2Fc7%2Fe1%2Fconfessions.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1080%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1080%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200275%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum-%20Boca%20Paila%20Zona%20Hotelera%2C%20Km.%207%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Hands%20down%20best%20night%20scene%20in%20tulum%20ask%20for%20them!!%20Confessions%20is%20an%20amazing...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23280755-r794483951-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23280755-r794483951-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Long%20and%20large%20%EF%BF%B9bar%EF%BF%BB.%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23280755-r996936626-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23280755-r996936626-Confessions_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A26128325%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Copal%20Gastro%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d26128325-Reviews-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.20358%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46014%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fcopalgastrobar.com%2Fwp-content%2Fuploads%2F2023%2F09%2FSPRECIO-COPAL-MENU-INGLES.pdf%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22b3I4Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjU0NzA4MjQ3Jmdlbz0yNjEyODMyNSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4d2RsOGpwN095RkVma0dxWVgtR3Iyc3RCd3ZObm1qLS1LOXptM2tXNE1rbyZjcz0xN2E4MjU2Nzc5ZTc1M2UwNGYwODA4YTcxODc0YTBlYTJfaXpR%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A665%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F29%2F7c%2F05%2F1c%2Fcaption.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A4000%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A6000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20181%206970%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Calle%20Ixchel%20esquina%20Av.%20Juanek%2C%20Aldea%20Zama%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Fantastic%20fish%20%EF%BF%B9tacos%EF%BF%BB%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26128325-r954595371-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26128325-r954595371-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Amazing%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d26128325-r1043570403-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d26128325-r1043570403-Copal_Gastro_Bar-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A19458762%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Curry%20Thai%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d19458762-Reviews-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.202757%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.47367%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A732%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2F8b%2F3c%2F58%2Fstaff.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1200%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1600%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20239%203265%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22C.%207%20Sur%20La%20Veleta%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Great%20%EF%BF%B9Thai%EF%BF%BB%20food%20in%20Tulum%20and%20Fermin%20was%20a%20fantastic%20waiter%2C%20super%20attentive%2C...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19458762-r923747319-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19458762-r923747319-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%E2%80%99ve%20stayed%20in%20tulum%20now%20for%20a%20few...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19458762-r783269823-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19458762-r783269823-Curry_Thai_Restaurant-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A1775007%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Ziggy%20Beach%20Club%20%26%20Restaurant%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d1775007-Reviews-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.156145%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45642%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.ziggybeachtulum.com%2Fmenu%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22S3E4Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTcwMDIyJmdlbz0xNzc1MDA3JmZyb209UmVzdGF1cmFudHMmYXJlYT1yZXNlcnZhdGlvbl9idXR0b24mc2xvdD0xJm1hdGNoSUQ9MSZvb3M9MCZjbnQ9MSZzaWxvPTQ1NTgyJmJ1Y2tldD05Nzk2NDEmbnJhbms9MSZjcmFuaz0xJmNsdD1SJnR0eXBlPVJlc3RhdXJhbnQmdG09MzQ2NjcyMzM0Jm1hbmFnZWQ9ZmFsc2UmY2FwcGVkPWZhbHNlJmdvc294PTlETXZpYkRUV2ROYUZlR1lMOFFUYWt5djFMQllaYUdwZUI0TUVIbi1odjJtWkJOMzc2RUhWQ0d1c1RJUnhtSjgySE44VTFMWDVYM0g0LXNNUmVjdVJSWkJFeVVCN2VkMDk0WDJLb3JSVXZ3JmNzPTEyZTQ4ZTkxZTU5YWQwYjAzMDk2NmJiN2Q3MjkxY2E1NF9pc2U%3D%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2222%3A30%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2207%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A2186%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.5%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2e%2F1e%2F3d%2F74%2Fbeach-club.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A2250%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A4000%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20871%201145%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20km%207.5%20Zona%20Hotelera%2C%20Tulum%20Beach%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Stayed%20at%20the%20beach%20tulum%20and%20ate%20at%20ziggy%E2%80%99s%20for%20lunch%20and%20dinner%20many%20times...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1775007-r785116688-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1775007-r785116688-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22For%20the%20main%20dish%20had%20the%20%EF%BF%B9octopus%EF%BF%BB%2C%20grea...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d1775007-r962472267-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d1775007-r962472267-Ziggy_Beach_Club_Restaurant-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23259804%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mamazzita%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23259804-Reviews-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.15775%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.45616%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Fbit.ly%2F2PV0Uln%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fusion%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A11776%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bars%20%26%20Pubs%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22CLOSED%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Closed%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2217%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1184%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F22%2F33%2Fb4%2F1c%2Fmain-entrance-to-mamazzita.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A720%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1280%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200274%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum%20Boca%20Paila%2C%20Km.%207%20Zona%20Hotelera%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Great%20service%2C%20Carlos%20was%20our%20server.%20The%20ambiance%20and%20atmosphere%20is%20really...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23259804-r790949867-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23259804-r790949867-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20wavy%20%EF%BF%B9meatballs%EF%BF%BB%2C%20prime%20rib%20tacos%20%26%20s...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23259804-r954878154-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23259804-r954878154-Mamazzita_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A13201059%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Loco%20Tulum%20Mediterranean%20Kitchen%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g23240074-d13201059-Reviews-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.137993%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46335%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22http%3A%2F%2Flocotulum.com%2Fmenu%2F%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22ejZFXy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjMyMTgzNjc3Jmdlbz0xMzIwMTA1OSZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4czgwc2E2NGxkXzdWanU0NVRZLTg2RDFnOS16NTgycmRucHZaT3ZLZ3VIYyZjcz0xNGViMzJlYjM2N2IzMjM2MTYyNDk4OGU0ODU2YjVmOTZfN0Uz%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20Bar%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Middle%20Eastern%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2201%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A30%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A961%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.9%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1a%2Fb3%2F9e%2Ff9%2Flocos-at-loco-tulum-km.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A1400%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1400%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20309%200149%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carretera%20Tulum%20Boca%20Paila%20Km%209.3%2C%20Tulum%20Beach%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A23240074%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22We%20enjoyed%20the%20pita%20%26%20%EF%BF%B9hummus%EF%BF%BB%2C%20tuna%20tataki%2C%20shrimp%20tacos%2C%20and%20Caesar%20salad%E2%80%94eve...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d13201059-r1003566357-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d13201059-r1003566357-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22I%E2%80%99m%20very%20speechless%20how%20was%20the%20service...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g23240074-d13201059-r805223610-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g23240074-d13201059-r805223610-Loco_Tulum_Mediterranean_Kitchen-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A19855557%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Las%20Hijas%20De%20La%20Tostada%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d19855557-Reviews-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.211218%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.461555%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Flas-hijas-de-la-tostada-4.ola.click%2Fproducts%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Food%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2208%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A178%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F2b%2Fa4%2F5d%2F94%2Fshrimp-burger.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A820%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A1200%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20235%200690%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Centro%2C%20Col%20Huracanes%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Not%20too%20sweet%20and%20quality%20ingredients.%20I%20had%20a%20hibiscus%2Fmescal%20drink%20and%20a...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19855557-r737833923-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19855557-r737833923-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20and%20salmon%20%EF%BF%B9tostada%EF%BF%BB%20and%20both%20were%20su...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d19855557-r1036251845-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d19855557-r1036251845-Las_Hijas_De_La_Tostada_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A21248033%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Funky%20Geisha%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d21248033-Reviews-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.149506%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.4594%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.funkygeisha.com.mx%2Fen%2Fmenu-comida-asiatica-thai-japonesa%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Japanese%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A1457%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.8%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F1c%2F61%2F87%2Fd9%2Flounge.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A755%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A986%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22year%5C%5C%5C%22%3A2025%2C%5C%5C%5C%22awardType%5C%5C%5C%22%3A%5C%5C%5C%22COE%5C%5C%5C%22%7D%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20998%20690%200255%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277780%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Carr.%20Tulum-Boca%20Paila%20Km%207.8%20Tulum%20Beach%20-%20Hotel%20Zone%2C%20Tulum%2077780%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22...%20the%20Panang%20curry%20and%20all%20of%20our%20%EF%BF%B9sushi%EF%BF%BB%20(order%20the%20kimchi%20%EF%BF%B9sushi%EF%BF%BB!)%20was%20delic...%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d21248033-r984875421-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d21248033-r984875421-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22A%20great%20restaurant%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d21248033-r1038637138-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d21248033-r1038637138-Funky_Geisha_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A23597112%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Mamazul%20Tulum%20Mezcaleria%5C%5C%5C%22%2C%5C%5C%5C%22detailPageRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurant_Review-g150813-d23597112-Reviews-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22geoPoint%5C%5C%5C%22%3A%7B%5C%5C%5C%22latitude%5C%5C%5C%22%3A20.199795%2C%5C%5C%5C%22longitude%5C%5C%5C%22%3A-87.46075%7D%2C%5C%5C%5C%22distance%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22menu%5C%5C%5C%22%3A%7B%5C%5C%5C%22menuUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fmamazul.short.gy%2F3PHxpA%5C%5C%5C%22%2C%5C%5C%5C%22hasProvider%5C%5C%5C%22%3Atrue%7D%2C%5C%5C%5C%22offers%5C%5C%5C%22%3A%7B%5C%5C%5C%22hasDelivery%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22hasReservation%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22slot1Offer%5C%5C%5C%22%3A%7B%5C%5C%5C%22providerId%5C%5C%5C%22%3A%5C%5C%5C%2215910%5C%5C%5C%22%2C%5C%5C%5C%22provider%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22providerDisplayName%5C%5C%5C%22%3A%5C%5C%5C%22OpenTable%5C%5C%5C%22%2C%5C%5C%5C%22buttonText%5C%5C%5C%22%3A%5C%5C%5C%22Reserve%5C%5C%5C%22%2C%5C%5C%5C%22offerURL%5C%5C%5C%22%3A%5C%5C%5C%22akx4Xy9Db21tZXJjZT9wPU9wZW5UYWJsZSZzcmM9MjM3MjkwMDU2Jmdlbz0yMzU5NzExMiZmcm9tPVJlc3RhdXJhbnRzJmFyZWE9cmVzZXJ2YXRpb25fYnV0dG9uJnNsb3Q9MSZtYXRjaElEPTEmb29zPTAmY250PTEmc2lsbz00NTU4MiZidWNrZXQ9OTc5NjQxJm5yYW5rPTEmY3Jhbms9MSZjbHQ9UiZ0dHlwZT1SZXN0YXVyYW50JnRtPTM0NjY3MjMzNCZtYW5hZ2VkPWZhbHNlJmNhcHBlZD1mYWxzZSZnb3NveD05RE12aWJEVFdkTmFGZUdZTDhRVGFreXYxTEJZWmFHcGVCNE1FSG4taHYybVpCTjM3NkVIVkNHdXNUSVJ4bUo4MkUwMThPbEJrdjF0TDV0T3l4RDJELThQdmozcHVOU19uME5LTUxGc1NFayZjcz0xNzUzNjQzNGE5NTc0YjlmM2ZjNDNhOTRiYzkyMzg4ZDhfdFBw%5C%5C%5C%22%2C%5C%5C%5C%22logoUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2Fimg2%2Fbranding%2Fhotels%2FOT_Logo_1000x232.png%5C%5C%5C%22%2C%5C%5C%5C%22trackingEvent%5C%5C%5C%22%3A%5C%5C%5C%22reserve_click%5C%5C%5C%22%2C%5C%5C%5C%22canProvideTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22canLockTimeslots%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22timeSlots%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22slot2Offer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A10591%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%7D%5D%7D%2C%5C%5C%5C%22openHours%5C%5C%5C%22%3A%7B%5C%5C%5C%22openStatus%5C%5C%5C%22%3A%5C%5C%5C%22OPEN%5C%5C%5C%22%2C%5C%5C%5C%22openStatusText%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22schedule%5C%5C%5C%22%3A%7B%5C%5C%5C%22mon%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22tue%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22wed%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22thu%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22fri%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sat%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22sun%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22close_time%5C%5C%5C%22%3A%5C%5C%5C%2223%3A00%3A00%5C%5C%5C%22%2C%5C%5C%5C%22open_time%5C%5C%5C%22%3A%5C%5C%5C%2209%3A00%3A00%5C%5C%5C%22%7D%5D%7D%7D%2C%5C%5C%5C%22reviewSummary%5C%5C%5C%22%3A%7B%5C%5C%5C%22count%5C%5C%5C%22%3A52%2C%5C%5C%5C%22rating%5C%5C%5C%22%3A4.6%7D%2C%5C%5C%5C%22thumbnail%5C%5C%5C%22%3A%7B%5C%5C%5C%22photo%5C%5C%5C%22%3A%7B%5C%5C%5C%22photoSizeDynamic%5C%5C%5C%22%3A%7B%5C%5C%5C%22urlTemplate%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fdynamic-media-cdn.tripadvisor.com%2Fmedia%2Fphoto-o%2F26%2Ff5%2F5a%2F6b%2Foctopus-with-mezcal-pairing.jpg%3Fw%3D%7Bwidth%7D%26h%3D%7Bheight%7D%26s%3D1%5C%5C%5C%22%2C%5C%5C%5C%22maxHeight%5C%5C%5C%22%3A483%2C%5C%5C%5C%22maxWidth%5C%5C%5C%22%3A723%7D%7D%7D%2C%5C%5C%5C%22travelerAwards%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22taLocation%5C%5C%5C%22%3A%7B%5C%5C%5C%22contact%5C%5C%5C%22%3A%7B%5C%5C%5C%22telephone%5C%5C%5C%22%3A%5C%5C%5C%22%2B52%20984%20807%206724%5C%5C%5C%22%2C%5C%5C%5C%22streetAddress%5C%5C%5C%22%3A%7B%5C%5C%5C%22country%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22postalCode%5C%5C%5C%22%3A%5C%5C%5C%2277760%5C%5C%5C%22%2C%5C%5C%5C%22fullAddress%5C%5C%5C%22%3A%5C%5C%5C%22Aldea%20Zama%20Andador%20Kanbul%2C%20entre%20Andador%20Kaan%20y%20Calle%20Ixchel%2C%20Tulum%2077760%20Mexico%5C%5C%5C%22%7D%7D%2C%5C%5C%5C%22names%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeo%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22hierarchy%5C%5C%5C%22%3A%7B%5C%5C%5C%22parentGeoId%5C%5C%5C%22%3A150813%7D%7D%2C%5C%5C%5C%22reviewSnippetsV2%5C%5C%5C%22%3A%7B%5C%5C%5C%22reviews%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Best%20%EF%BF%B9tacos%EF%BF%BB%20ever%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23597112-r974966219-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23597112-r974966219-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%2C%7B%5C%5C%5C%22seoContentType%5C%5C%5C%22%3A%5C%5C%5C%22REVIEW_SNIPPET%5C%5C%5C%22%2C%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Mezcal%20yes%20please%5C%5C%5C%22%2C%5C%5C%5C%22review%5C%5C%5C%22%3A%7B%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowUserReviews-g150813-d23597112-r1038908163-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22absoluteUrl%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisor.com%2FShowUserReviews-g150813-d23597112-r1038908163-Mamazul_Tulum_Mezcaleria-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22PUBLISHED%5C%5C%5C%22%7D%7D%5D%7D%2C%5C%5C%5C%22isSponsoredListing%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22sponsoredListingData%5C%5C%5C%22%3A%7B%5C%5C%5C%22restaurantRequestGuid%5C%5C%5C%22%3A%5C%5C%5C%225399e40b-cf27-4758-b5a4-31ca6a50e60a%5C%5C%5C%22%7D%2C%5C%5C%5C%22isAdsPreview%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22storyboardStatus%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22isPremium%5C%5C%5C%22%3Afalse%7D%5D%2C%5C%5C%5C%22facets%5C%5C%5C%22%3A%7B%5C%5C%5C%22onlineOptions%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Online%20options%20%26%20offers%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220693%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A531%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Online%20Reservations%20%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20693%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22cuisines%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Cuisines%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225110%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A225%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mexican%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5110%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210643%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A123%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seafood%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10643%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210640%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A73%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10640%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210679%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A68%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10679%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210648%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A64%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22International%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10648%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210639%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A59%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Latin%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10639%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210642%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A51%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cafe%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10642%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%224617%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A47%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Italian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A4617%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210649%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A36%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mediterranean%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10649%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210622%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Caribbean%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10622%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210641%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pizza%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10641%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210671%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A29%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fusion%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10671%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210670%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A28%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pub%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10670%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210668%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A26%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Grill%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10668%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210655%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A25%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Spanish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10655%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229908%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A25%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22American%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9908%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210669%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A20%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Contemporary%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10669%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210749%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22South%20American%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10749%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210651%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A17%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Barbecue%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10651%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210659%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Asian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10659%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210345%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Steakhouse%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10345%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210760%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Central%20American%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10760%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225473%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A14%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Japanese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5473%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210653%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10653%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210698%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A12%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Argentinean%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10698%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210682%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10682%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225086%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22French%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5086%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210646%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fast%20Food%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10646%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210654%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22European%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10654%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221355%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beer%20restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21355%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210660%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Thai%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10660%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221353%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dining%20bars%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21353%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210621%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A6%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brew%20Pub%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10621%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210687%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A6%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Middle%20Eastern%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10687%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221343%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A5%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fruit%20parlours%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21343%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210631%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Peruvian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10631%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210683%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gastropub%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10683%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221367%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Japanese%20Fusion%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21367%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210346%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Indian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10346%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210700%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soups%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10700%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220076%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Southern-Italian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20076%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210663%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Turkish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10663%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210699%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Canadian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10699%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210772%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hawaiian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10772%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220062%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Neapolitan%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20062%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220067%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Romana%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20067%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220068%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lazio%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20068%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220069%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sicilian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20069%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220075%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Central-Italian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20075%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221350%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Medicinal%20foods%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21350%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210348%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brazilian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10348%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210633%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Moroccan%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10633%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210666%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Deli%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10666%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210675%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vietnamese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10675%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210676%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Diner%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10676%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210680%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Portuguese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10680%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210686%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Food%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10686%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210696%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Taiwanese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10696%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211739%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Central%20Asian%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11739%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220064%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Campania%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20064%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220077%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Catalan%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20077%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%225379%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A231%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chinese%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A5379%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22diets%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Dietary%20restrictions%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210665%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A160%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A285%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vegetarian%20friendly%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10665%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210697%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A138%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A285%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vegan%20options%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10697%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210992%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A94%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A285%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gluten%20free%20options%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10992%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22diningOptions%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Features%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210852%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A308%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Seating%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10852%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216547%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A305%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Table%20Service%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16547%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210602%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A250%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Reservations%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10602%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210862%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A248%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Serves%20Alcohol%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10862%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210603%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A231%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Outdoor%20Seating%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10603%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211780%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A211%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Accepts%20Credit%20Cards%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11780%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210870%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A210%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Free%20Wifi%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10870%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210863%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A191%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Full%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10863%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210601%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A160%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Takeout%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10601%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210864%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A129%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wine%20and%20Beer%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10864%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210854%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A101%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Parking%20Available%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10854%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210861%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A99%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wheelchair%20Accessible%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10861%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210867%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A98%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Visa%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10867%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210866%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A94%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mastercard%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10866%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220996%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A93%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dog%20Friendly%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20996%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210855%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A82%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10855%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210860%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A80%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Highchairs%20Available%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10860%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220989%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A76%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Live%20Music%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20989%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210865%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A73%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22American%20Express%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10865%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221271%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A69%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Family%20style%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21271%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210600%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A62%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Delivery%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10600%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210868%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A43%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Digital%20Payments%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10868%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210856%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A40%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Validated%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10856%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210858%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A40%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Free%20off-street%20parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10858%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220990%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A37%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Waterfront%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20990%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210859%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A36%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Television%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10859%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220993%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beach%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20993%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210702%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A24%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Private%20Dining%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10702%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210869%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cash%20Only%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10869%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210871%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A21%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Discover%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10871%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221379%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Non-smoking%20restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21379%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221907%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Gift%20Cards%20Available%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21907%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220992%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A14%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Drive%20Thru%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20992%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210857%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Valet%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10857%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220995%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Playgrounds%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20995%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221381%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sports%20bars%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21381%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220994%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A5%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22BYOB%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20994%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210612%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A3%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Buffet%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10612%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220991%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A234%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Jazz%20Bar%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20991%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22dishes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Dishes%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220515%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A197%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cocido%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20515%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2219954%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A121%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tacos%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A19954%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222444%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A115%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beef%20Tacos%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22444%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210937%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A96%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Shrimp%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10937%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221324%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A96%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21324%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223001%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A96%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Jumbo%20Shrimp%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23001%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216554%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A89%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Salad%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16554%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210883%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A83%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Ceviche%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10883%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223145%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A71%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Guacamole%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23145%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222950%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A65%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Desserts%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22950%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221354%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A63%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20dishes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21354%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220336%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A54%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Steak%20Frites%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20336%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220711%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A52%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Octopus%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20711%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222620%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A50%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mixed%20Seafood%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22620%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222390%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A49%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Burger%20and%20Fries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22390%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221239%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A48%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pesto%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21239%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222870%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A48%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fresh%20Pasta%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22870%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210907%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A47%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Burger%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10907%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220552%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A45%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tuna%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20552%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210678%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A44%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pasta%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10678%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222992%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A39%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mole%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22992%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216553%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A38%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fish%20Taco%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16553%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220487%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cecina%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20487%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223109%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A35%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cake%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23109%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220752%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A34%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beef%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20752%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223203%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A34%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fried%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23203%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222568%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Soup%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22568%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229911%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Juice%20%26%20Smoothies%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9911%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210915%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A32%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lobster%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10915%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222388%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A32%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Breads%20and%20Pastries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22388%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222293%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20Quesadilla%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22293%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223103%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Quesadilla%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23103%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223285%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A30%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Quesadillas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23285%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220031%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A29%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Crawfish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20031%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220041%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A28%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Nachos%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20041%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222299%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A28%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Rice%20dishes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22299%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210944%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A27%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tortillas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10944%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210685%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A25%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20Wings%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10685%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220034%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A24%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fajitas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20034%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222879%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A23%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tostadas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22879%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222901%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A23%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tostada%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22901%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220730%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Toasts%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20730%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221326%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pork%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21326%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222890%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A22%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Toasties%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22890%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210647%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sandwiches%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10647%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210878%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Burrito%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10878%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220175%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Veggie%20Burrito%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20175%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220547%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A19%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Salmon%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20547%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210932%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Ribs%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10932%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220697%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Foie%20gras%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20697%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223357%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A18%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Potato%20Wedges%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23357%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220148%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A17%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hoagie%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20148%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222521%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mango%20dishes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22521%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223331%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A16%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Beef%20Ribs%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23331%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222305%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Enchiladas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22305%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222628%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A15%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chicken%20Enchiladas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22628%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221275%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A14%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cakes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21275%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221174%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lamb%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21174%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222438%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22White%20Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22438%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222573%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Coconut%20Shrimp%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22573%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222584%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A13%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Avocado%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22584%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210902%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A12%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22French%20Toast%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10902%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220029%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chili%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20029%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221338%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Garlic%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21338%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222280%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chili%20Con%20Carne%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22280%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222572%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%20Rolls%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22572%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222651%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Snapper%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22651%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222775%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Red%20Snapper%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22775%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223089%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A11%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chilaquiles%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23089%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210921%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Omelette%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10921%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216555%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pancakes%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16555%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220027%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Calamari%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20027%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222386%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Grilled%20Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22386%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222867%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Patties%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22867%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223260%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fresh%20Fish%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23260%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223368%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A10%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fish%20Fillet%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23368%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210924%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Paella%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10924%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220181%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Curry%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20181%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220312%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Risotto%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20312%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220475%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Paella%20Valenciana%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20475%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220754%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hamburgers%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20754%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221234%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Milanese%20risotto%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21234%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222275%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Avocado%20Toast%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22275%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210923%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pad%20Thai%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10923%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210939%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Steak%20Tartare%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10939%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220699%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Prawns%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20699%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221278%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Chocolates%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21278%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221301%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Parfait%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21301%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222743%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Empanadas%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22743%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223091%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Empanada%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23091%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223125%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A8%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Vegetable%20Curry%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23125%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210893%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Crab%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10893%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220347%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fettuccine%20Alfredo%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20347%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2220483%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Eggplant%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A20483%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221037%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hanger%20steak%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21037%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221175%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Antipasti%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21175%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222319%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mixed%20Grill%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22319%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2222431%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tartare%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A22431%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2223311%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Pastries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A23311%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2226531%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A7%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A239%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Huevos%20Rancheros%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A26531%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22establishmentTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Establishment%20type%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210591%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A379%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10591%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211776%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A12%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bars%20%26%20Pubs%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11776%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229900%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A9%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Coffee%20%26%20Tea%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9900%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229909%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A6%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dessert%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9909%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216556%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Quick%20Bites%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16556%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%229901%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bakeries%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A9901%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221908%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A1%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A230%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Delivery%20Only%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21908%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22mealTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Meal%20type%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210597%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A181%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Breakfast%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10597%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210606%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A122%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Brunch%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10606%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210598%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A306%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Lunch%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10598%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210599%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A308%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A233%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Dinner%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10599%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22michelinAwards%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22MICHELIN%20Guide%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221964%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A4%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A645%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22MICHELIN%20Guide%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21964%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2221969%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A2%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A645%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22MICHELIN%20Bib%20Gourmand%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A21969%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22restaurantSpecialOffer%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22minimumTravelerRating%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Traveler%20rating%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELER_RATING_LOW%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%223%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A380%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELER_RATING_MEDIUM%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%224%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A338%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELER_RATING_HIGH%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%225%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A64%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%5D%7D%2C%5C%5C%5C%22restaurantOwnerTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Diversity%20attributes%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22neighbourhoods%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Neighborhood%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22waypoints%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Airports%20and%20theme%20parks%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%5D%7D%2C%5C%5C%5C%22travelersChoice%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Awards%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22coe%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22Travelers%E2%80%99%20Choice%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A33%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%2C%5C%5C%5C%22openNow%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%22OPEN_NOW%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Open%20now%5C%5C%5C%22%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%5D%7D%2C%5C%5C%5C%22priceTypes%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Price%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210953%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A197%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A240%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Cheap%20Eats%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10953%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210955%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%20-%20%24%24%24%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A397%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A240%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Mid-range%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10955%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Atrue%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210954%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3A%5C%5C%5C%22%24%24%24%24%5C%5C%5C%22%2C%5C%5C%5C%22count%5C%5C%5C%22%3A80%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A240%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Fine%20Dining%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10954%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Afalse%7D%5D%7D%2C%5C%5C%5C%22styles%5C%5C%5C%22%3A%7B%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Great%20for%5C%5C%5C%22%2C%5C%5C%5C%22items%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210604%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A186%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Families%20with%20children%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10604%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210614%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A132%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Romantic%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10614%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2211777%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A117%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Kids%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A11777%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210607%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A82%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Special%20occasions%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10607%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210609%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A81%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Large%20groups%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10609%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210613%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A79%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Local%20cuisine%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10613%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210608%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A72%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Bar%20scene%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10608%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210605%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A46%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Business%20meetings%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10605%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2210610%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A45%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Scenic%20view%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A10610%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2216550%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A27%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hidden%20Gems%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A16550%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%2C%7B%5C%5C%5C%22id%5C%5C%5C%22%3A%5C%5C%5C%2212504%5C%5C%5C%22%2C%5C%5C%5C%22secondaryName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22count%5C%5C%5C%22%3A23%2C%5C%5C%5C%22tag%5C%5C%5C%22%3A%7B%5C%5C%5C%22categoryId%5C%5C%5C%22%3A235%2C%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hot%20New%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22tagId%5C%5C%5C%22%3A12504%7D%2C%5C%5C%5C%22mustTry%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22selected%5C%5C%5C%22%3Anull%7D%5D%7D%7D%2C%5C%5C%5C%22banner%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22metadata%5C%5C%5C%22%3A%7B%5C%5C%5C%22nextCursor%5C%5C%5C%22%3A%5C%5C%5C%22eyJzdGFydF9hZnRlciI6WyI0LjQ0MTAxMjQiLCI0LjcxMjIyNDc2NiIsImYzNDM1ZTViLTkyZWUtNGY0My1iNjM2LTY1OWUyZjk0NTNlZSJdLCJyYWNfb2Zmc2V0IjpudWxsLCJtYXhfcmFjX29mZnNldCI6bnVsbCwib2Zmc2V0IjowLCJsaW1pdCI6MzB9%5C%5C%5C%22%2C%5C%5C%5C%22total_hits%5C%5C%5C%22%3A397%2C%5C%5C%5C%22offset%5C%5C%5C%22%3A0%7D%2C%5C%5C%5C%22racMetadata%5C%5C%5C%22%3A%7B%5C%5C%5C%22requestToken%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22defaultDatetime%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22defaultPartySize%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22currentDatetime%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22enabled%5C%5C%5C%22%3Afalse%7D%2C%5C%5C%5C%22sorts%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22POPULARITY%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Highest%20Rating%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Highest%20rated%20restaurants%20on%20Tripadvisor%2C%20based%20on%20reviews.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22RELEVANCE%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Relevance%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20ranked%20by%20how%20well%20they%20match%20your%20selections%20and%20traveler%20reviews.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22is_applied%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22FEATURED%5C%5C%5C%22%2C%5C%5C%5C%22title%5C%5C%5C%22%3A%5C%5C%5C%22Featured%5C%5C%5C%22%2C%5C%5C%5C%22tooltip%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20ranked%20according%20to%20page%20views%2C%20reviews%20and%20individual%20attributes%20such%20as%20price%20range%2C%20cuisine%20and%20location%2C%20as%20well%20as%20aggregated%20Tripadvisor%20data%20comparing%20user%20interactions%20with%20all%20the%20other%20restaurant%20pages%20for%20this%20locale.%5C%5C%5C%22%7D%5D%2C%5C%5C%5C%22searchCenter%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22geoBroadenStatus%5C%5C%5C%22%3A%7B%5C%5C%5C%22is_broadened%5C%5C%5C%22%3Afalse%2C%5C%5C%5C%22cta_message%5C%5C%5C%22%3A%7B%5C%5C%5C%22desktopText%5C%5C%5C%22%3A%5C%5C%5C%22Looking%20to%20expand%20your%20search%20outside%20of%20Tulum%3F%20We%20have%20suggestions.%5C%5C%5C%22%2C%5C%5C%5C%22mobileText%5C%5C%5C%22%3A%5C%5C%5C%22We%20found%20great%20results%20outside%20of%20Tulum.%5C%5C%5C%22%7D%2C%5C%5C%5C%22action_message%5C%5C%5C%22%3A%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Expand%20your%20search.%5C%5C%5C%22%7D%7D%7D%7D%5C%22%7D%2C%5C%224252927822%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22breadcrumbsData%5C%5C%5C%22%3A%7B%5C%5C%5C%22breadcrumbs%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Mexico%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150768-Mexico-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Yucatan%20Peninsula%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150805-Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Quintana%20Roo%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g1575485-Quintana_Roo_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Riviera%20Maya%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g659488-Riviera_Maya_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%2C%7B%5C%5C%5C%22text%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22linkRoute%5C%5C%5C%22%3A%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%5C%5C%5C%22localizedText%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22obfuscated%5C%5C%5C%22%3Afalse%7D%5D%2C%5C%5C%5C%22usePageHeader%5C%5C%5C%22%3Atrue%2C%5C%5C%5C%22additionalTrails%5C%5C%5C%22%3A%5B%5D%7D%7D%5C%22%7D%2C%5C%22100483659892248%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Theme_getTheme%5C%5C%5C%22%3Anull%7D%5C%22%7D%2C%5C%2216178272766%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Opf_getOnPageFactorsForLocale%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22errMessage%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22factors%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22META_DESCRIPTION%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Moderately%20Priced%20Restaurants%20in%20Tulum%2C%20Riviera%20Maya%3A%20Find%20Tripadvisor%20traveler%20reviews%20of%20THE%20BEST%20Tulum%20Mid%20Range%20Restaurants%20and%20search%20by%20price%2C%20location%2C%20and%20more.%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22MASTHEAD_H1%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Moderately%20Priced%20Restaurants%20in%20Tulum%2C%20Riviera%20Maya%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22TITLE%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22THE%2010%20BEST%20Moderately-Priced%20Restaurants%20in%20Tulum%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22MAIN_H1%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Mid%20Range%20Restaurants%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22IS_INDEXABLE%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5C%5C%5C%22true%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-2754358323%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AdMissionControl_getAdSlotsListForPageAndPlatform%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22slots%5C%5C%5C%22%3A%5B%5C%5C%5C%22footer%5C%5C%5C%22%2C%5C%5C%5C%22inline8%5C%5C%5C%22%2C%5C%5C%5C%22inline7%5C%5C%5C%22%2C%5C%5C%5C%22inline6%5C%5C%5C%22%2C%5C%5C%5C%22inline5%5C%5C%5C%22%2C%5C%5C%5C%22inline4%5C%5C%5C%22%2C%5C%5C%5C%22inline3%5C%5C%5C%22%2C%5C%5C%5C%22inline2%5C%5C%5C%22%2C%5C%5C%5C%22inline1%5C%5C%5C%22%2C%5C%5C%5C%22rail1%5C%5C%5C%22%2C%5C%5C%5C%22rail2%5C%5C%5C%22%2C%5C%5C%5C%22horizon%5C%5C%5C%22%2C%5C%5C%5C%22InlineNova%5C%5C%5C%22%2C%5C%5C%5C%22rail3%5C%5C%5C%22%5D%7D%5D%7D%5C%22%7D%2C%5C%227470346746%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AdMissionControl_getPageSlotSettings%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22slots%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline8%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline4%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline5%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline6%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22horizon%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline7%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22InlineNova%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline1%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline2%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22inline3%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22footer%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22rail2%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22rail1%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22slot%5C%5C%5C%22%3A%5C%5C%5C%22rail3%5C%5C%5C%22%2C%5C%5C%5C%22overrides%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-2154554899%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22locations%5C%5C%5C%22%3A%7B%5C%5C%5C%22detail%5C%5C%5C%22%3A%7B%5C%5C%5C%22info%5C%5C%5C%22%3A%7B%5C%5C%5C%22countryId%5C%5C%5C%22%3A150768%7D%7D%7D%7D%5C%22%7D%2C%5C%2211562210216%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22gptInfo%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22adBase%5C%5C%5C%22%3A%5C%5C%5C%22%2F5349%2Fta.ta.br.s%2Fna.mex.yucatan_peninsula.tulum%5C%5C%5C%22%2C%5C%5C%5C%22ppid%5C%5C%5C%22%3A%5C%5C%5C%22AZXKXUDHRA6OVGEIKKSKDSN2UX7UAA43DLPU7ICO3QWSLZDJ6LHQ%5C%5C%5C%22%2C%5C%5C%5C%22pageLevelTargeting%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22aud_id%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%2216062%5C%5C%5C%22%2C%5C%5C%5C%2250011%5C%5C%5C%22%2C%5C%5C%5C%2215434%5C%5C%5C%22%2C%5C%5C%5C%2216856%5C%5C%5C%22%2C%5C%5C%5C%2217765%5C%5C%5C%22%2C%5C%5C%5C%2215440%5C%5C%5C%22%2C%5C%5C%5C%2217323%5C%5C%5C%22%2C%5C%5C%5C%2217307%5C%5C%5C%22%2C%5C%5C%5C%2216860%5C%5C%5C%22%2C%5C%5C%5C%2220048%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22browser%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22chrome%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22country%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22150768%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22dest%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22beach%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22detail%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%220%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22geo%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22150813%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22geo_ctx%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%2211903%5C%5C%5C%22%2C%5C%5C%5C%2211912%5C%5C%5C%22%2C%5C%5C%5C%2212024%5C%5C%5C%22%2C%5C%5C%5C%2212028%5C%5C%5C%22%2C%5C%5C%5C%2212035%5C%5C%5C%22%2C%5C%5C%5C%2212067%5C%5C%5C%22%2C%5C%5C%5C%2212077%5C%5C%5C%22%2C%5C%5C%5C%2221827%5C%5C%5C%22%2C%5C%5C%5C%2221828%5C%5C%5C%22%2C%5C%5C%5C%2221840%5C%5C%5C%22%2C%5C%5C%5C%2221852%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22loctype%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22restaurants%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22PageType%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22Restaurants%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22platform%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22desktop%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22premium_category%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22modalhorizon%3Apremium%2Fhorizon%3Apremium%2Frail3%3Apremium%2Frail1%3Apremium%2Finline8%3Apremium%2Finline6%3Asufficient%2Finline7%3Asufficient%2Finlinenova%3Asufficient%2Finline1%3Asufficient%2Finline2%3Asufficient%2Ffooter%3Asufficient%2Finline3%3Apremium%2Finline4%3Apremium%2Finline5%3Apremium%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22rd%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22com%5C%5C%5C%22%5D%7D%2C%7B%5C%5C%5C%22key%5C%5C%5C%22%3A%5C%5C%5C%22region%5C%5C%5C%22%2C%5C%5C%5C%22value%5C%5C%5C%22%3A%5B%5C%5C%5C%22150813%5C%5C%5C%22%2C%5C%5C%5C%22659488%5C%5C%5C%22%2C%5C%5C%5C%221575485%5C%5C%5C%22%2C%5C%5C%5C%22150805%5C%5C%5C%22%5D%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-12449012372%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22ad_refresh_tweak_1677267844%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22control%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-9948974379%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22trips_rebrand_on_web_1753200458%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3Anull%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-2632168235%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22standalone_tips_1744116322%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22global_nav_cta_simplification_1764622874%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22control%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-612125067%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22typeahead___nested_results__icons___hard_code_list__1723050108%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3Anull%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-22033722027%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22core_combined_login_modern_ui_poc_modal_1720186558%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-1389843205%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22location%5C%5C%5C%22%3A%7B%5C%5C%5C%22detail%5C%5C%5C%22%3A%7B%5C%5C%5C%22info%5C%5C%5C%22%3A%7B%5C%5C%5C%22localizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%7D%2C%5C%5C%5C%22rateLimiting%5C%5C%5C%22%3A%7B%5C%5C%5C%22status%5C%5C%5C%22%3A%5C%5C%5C%22OK%5C%5C%5C%22%7D%7D%7D%7D%5C%22%7D%2C%5C%22-18230052403%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22routes%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Tourism%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22c0xNXy9Ub3VyaXNtLWcxNTA4MTMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEtVmFjYXRpb25zLmh0bWxfRkdx%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22HotelsFusion%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22VjNBXy9Ib3RlbHMtZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS1Ib3RlbHMuaHRtbF9NR2g%3D%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22AttractionsFusion%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22WWI5Xy9BdHRyYWN0aW9ucy1nMTUwODEzLUFjdGl2aXRpZXMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEuaHRtbF9ZakE%3D%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22OXZ1Xy9SZXN0YXVyYW50cy1nMTUwODEzLVR1bHVtX1l1Y2F0YW5fUGVuaW5zdWxhLmh0bWxfYjBB%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Cruises%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22M2pWXy9DcnVpc2VzLWcxNTA4MTMtVHVsdW1fWXVjYXRhbl9QZW5pbnN1bGEtQ3J1aXNlc19yT0Q%3D%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22ShowForum%5C%5C%5C%22%2C%5C%5C%5C%22encodedUrl%5C%5C%5C%22%3A%5C%5C%5C%22OWZJXy9TaG93Rm9ydW0tZzE1MDgxMy1UdWx1bV9ZdWNhdGFuX1Blbmluc3VsYS5odG1sX2xNbQ%3D%3D%5C%5C%5C%22%7D%5D%7D%5C%22%7D%2C%5C%228843354577%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22region%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22route%5C%5C%5C%22%3Anull%7D%5D%7D%5C%22%7D%2C%5C%2212102622365%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22GNavlinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22Root%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Global%20Nav%20Root%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Tourism%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Tourism%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTourism-g150813-Tulum_Yucatan_Peninsula-Vacations.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Hotels%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_bnb%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Bed%20and%20Breakfast%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-c2-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Attractions%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Things%20to%20Do%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAttractions-g150813-Activities-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Forum%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Travel%20Forum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Photos%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Photos%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FLocationPhotos-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TOURISM_Maps%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Map%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FLocalMaps-g150813-Tulum-Area.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_Hotels%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22All%20Tulum%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_SD%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotel%20Deals%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FSmartDeals-g150813-Tulum_Yucatan_Peninsula-Hotel-Deals.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_LM%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Last%20Minute%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FLastMinute-g150813-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_hotels_near_me%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20Near%20Me%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNearMe%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22By%20Hotel%20Type%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_motels_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Motels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfd2-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_hostels_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hostels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-c3-zff26-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_hostels_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Campgrounds%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-c3-zff29-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Glamping%20Stays%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff105-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Beach%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff10-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Business%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff7-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Family%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff4-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Green%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff24-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff8-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Luxury%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff12-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Romantic%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff3-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Casinos%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff14-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_type_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Spa%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zff13-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22By%20Hotel%20Class%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%225-Star%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfc5-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%223-Star%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfc3-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_class_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%224-Star%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfc4-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_brand_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22By%20Hotel%20Brand%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Ahau%20Collection%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb22174-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22OYO%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb11779-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Luxury%20Bahia%20Principe%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb11763-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Oasis%20Hotels%20%26%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb16281-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22InterContinental%20(IHG)%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb9205-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Marriott%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb9208-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Colibri%20Boutique%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb11845-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_brand_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Hilton%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfb9204-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_by_amenity_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Amenities%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_amenities_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Pet%20Friendly%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfa9-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_amenities_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotels%20with%20Pools%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfa3-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_amenities_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Hotels%20with%20Free%20Parking%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotels-g150813-zfa7-Tulum_Yucatan_Peninsula-Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Tulum%20Categories%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Boutique%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Boutique-Hotels-zfp1512.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Discount%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Cheap-Hotels-zfp11040.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Honeymoon%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Honeymoon-Resorts-zfp13996.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Wedding%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Wedding-Hotels-zfp16067.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22All%20Inclusive%20Hotels%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-All-Inclusive-Hotels-zfp41115.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22All%20Inclusive%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-All-Inclusive-Resorts-zfp1189394.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Adults%20Only%20and%20Adult%20Friendly%20Resorts%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Adults-Only-And-Adult-Friendly-Resorts-zfp28929004.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Golf%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Golf-Hotels-zfp28929053.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Tulum%20Singles%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Singles-Resorts-zfp28929102.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_blender_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Spring%20Break%20Resorts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsList-Tulum-Spring-Break-Resorts-zfp28929151.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Landmarks%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Tulum%20Archaeological%20Site%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Akumal%20Beach%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Cenotes%20Dos%20Ojos%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d1478385-Cenotes_Dos_Ojos-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Playa%20Paraiso%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d3321137-Playa_Paraiso-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Gran%20Cenote%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d669578-Gran_Cenote-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Yal-ku%20Lagoon%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d153288-Yal_ku_Lagoon-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Aktun%20Chen%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d153913-Aktun_Chen-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Akumal%20Monkey%20Sanctuary%20%26%20Rescued%20Animals%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g499445-d10441459-Akumal_Monkey_Sanctuary_Rescued_Animals-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Cenotes%20Sac%20Actun%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d3387166-Cenotes_Sac_Actun-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20The%20Yacht%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Airports%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Cancun%20Airport%20(CUN)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150807-qCUN-Cancun_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Felipe%20Carrillo%20Puerto%20International%20Airport%20(TQO)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150813-qTQO-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hotels%20near%20Aerodromo%20de%20Playa%20Del%20Carmen%20(PCM)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150812-qPCM-Playa_del_Carmen_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_motels_near_airports_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Motels%20near%20Aerodromo%20de%20Playa%20Del%20Carmen%20(PCM)%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelsNear-g150812-qPCM-zfd2-Playa_del_Carmen_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_categories_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Hotel%20Categories%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Ryokan%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9435-Ryokan.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Family%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft6216-Family_Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Family%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft6216%2C9672-Family_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Ski%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9672%2C19075-Ski_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Singles%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9672%2C20284-Singles_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20All-Inclusive%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9172%2C9189-All_Inclusive_Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Honeymoon%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9624%2C9672-Honeymoon_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20All-Inclusive%20Family%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft6216%2C9172%2C9672-All_Inclusive_Family_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Adults%20Only%20All-Inclusive%20Resorts%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9172%2C9672%2C18884-Adults_Only_All_Inclusive_Resorts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HOTELS_top_hotel_category_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Casino%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FHotelCategory-zft9171-Casino_Hotels.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22ATTRACTIONS_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Things%20to%20Do%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAttractions-g150813-Activities-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22All%20Tulum%20Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_popular_types_of_food_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Types%20of%20Food%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Argentinian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10698-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Street%20Food%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10686-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Peruvian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10631-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Hawaiian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10772-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Healthy%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10679-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Latin%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10639-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_cuisine_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Fast%20Food%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c10646-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dietary_restriction_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Gluten%20Free%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfz10992-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dietary_restriction_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Vegetarian%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfz10665-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dietary_restriction_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Vegan%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfz10697-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dining_option_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Spanish%20Restaurants%20with%20Outdoor%20Seating%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c36-zfp6-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dining_option_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Late%20Night%20Steakhouses%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c37-zfp8-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dining_option_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Sushi%20Restaurants%20for%20Lunch%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-c38-zfp30-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_popular_dishes_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Dishes%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Donuts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10758-Tulum_Yucatan_Peninsula-Donuts.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20French%20Toast%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10902-Tulum_Yucatan_Peninsula-French_Toast.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Chow%20Mein%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10890-Tulum_Yucatan_Peninsula-Chow_Mein.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Unagi%20%26%20Dojo%20(Eel%20%26%20Soil)%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd11724-Tulum_Yucatan_Peninsula-Unagi_Dojo_Eel_Soil.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Cuban%20Sandwich%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd19960-Tulum_Yucatan_Peninsula-Cuban_Sandwich.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Mac%20and%20cheese%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20040-Tulum_Yucatan_Peninsula-Mac_and_cheese.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Steak%20Frites%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20336-Tulum_Yucatan_Peninsula-Steak_Frites.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Burritos%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10878-Tulum_Yucatan_Peninsula-Burrito.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Gyros%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10905-Tulum_Yucatan_Peninsula-Gyros.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Crab%20Cakes%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd10894-Tulum_Yucatan_Peninsula-Crab_Cakes.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Red%20Curry%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20183-Tulum_Yucatan_Peninsula-Red_Curry.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Veggie%20Burger%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20156-Tulum_Yucatan_Peninsula-Veggie_Burger.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Tres%20Leches%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20137-Tulum_Yucatan_Peninsula-Tres_Leches.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Cucumber%20Salad%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20161-Tulum_Yucatan_Peninsula-Cucumber_Salad.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_dishes_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Best%20Lettuce%20Wraps%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfd20189-Tulum_Yucatan_Peninsula-Lettuce_Wraps.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_popular_categories_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Popular%20Restaurant%20Categories%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links_breakfast%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Breakfast%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp2-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links_lunch%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Lunch%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp30-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links_dinner%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Dinner%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp58-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20with%20a%20View%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp43-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20for%20Group%20Dining%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp9-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Buffet%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp49-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Late%20Night%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp8-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20with%20Outdoor%20Seating%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp6-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Food%20Delivery%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp19-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_categories_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Kid%20Friendly%20Restaurants%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfp5-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_establishments_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Bakeries%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfg9901-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_establishments_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Coffee%20%26%20Tea%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfg9900-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_by_establishments_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Desserts%20in%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-zfg9909-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Hotels%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Lula%20Seaside%20Boutique%20Hotel%20%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d7223463-Lula_Seaside_Boutique_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Wakax%20Hacienda%20-%20Cenote%20%26%20Boutique%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23825874-Wakax_Hacienda_Cenote_Boutique_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Hotel%20Ma'xanab%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d19006698-Hotel_Ma_xanab_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Mi%20Amor%20Colibri%20Boutique%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d7183451-Mi_Amor_Colibri_Boutique_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Mezzanine%20Tulum%2C%20a%20Small%20Luxury%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d557443-Mezzanine_Tulum_a_Small_Luxury_Hotel-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Kanan%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d17495992-Kanan_Tulum-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20La%20Zebra%2C%20an%20SLH%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g23240074-d296667-La_Zebra_an_SLH_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Secrets%20Akumal%20Riviera%20Maya%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g499445-d8285376-Secrets_Akumal_Riviera_Maya-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Suenos%20Tulum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g23240074-d557040-Suenos_Tulum-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_hotels_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20The%20Beach%20Tulum%20Hotel%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g23240074-d3979959-The_Beach_Tulum_Hotel-Tulum_Beach_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_header%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Near%20Landmarks%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Tulum%20Archaeological%20Site%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d153186-Tulum_Archaeological_Site-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Akumal%20Beach%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g499445-d153287-Akumal_Beach-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20The%20Yacht%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d17562722-The_Yacht_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20My%20Quest%20Concierge%20Private%20Tours%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23065077-My_Quest_Concierge_Private_Tours-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Dlux%20Transfer%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23852094-Dlux_Transfer-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Marine%20Life%20Akumal%20MX%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g499445-d23928640-Marine_Life_Akumal_MX-Akumal_Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Mayikal%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d23285262-Mayikal_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Parque%20Ecoturistico%20Cenote%20Aktun%20Bej%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d26344863-Parque_Ecoturistico_Cenote_Aktun_Bej-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Tulum%20Experiences%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d26394539-Tulum_Experiences-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22RESTAURANTS_near_landmarks_links%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%20near%20Playacht%20-%20Private%20Yacht%20Rentals%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurantsNear-g150813-d20967421-Playacht_Private_Yacht_Rentals-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22ARTICLES_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Travel%20Stories%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FArticles%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22CRUISES_nm_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Cruises%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FCruises%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22url%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22tours%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Tours%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAttractions-g150813-Activities-c42-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22ADD_LISTING_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Add%20a%20Place%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAddListing%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TRAVEL_FORUMS_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Travel%20Forum%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FShowForum-g150813-i155-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22AIRLINES_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Airlines%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FAirlines%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TRAVELERS_CHOICE_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Travelers'%20Choice%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTravelersChoice%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22HELP_CENTER_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Help%20Center%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22https%3A%2F%2Fwww.tripadvisorsupport.com%2Fhc%2Fen-us%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%2C%7B%5C%5C%5C%22linkId%5C%5C%5C%22%3A%5C%5C%5C%22TRIPS_default%5C%5C%5C%22%2C%5C%5C%5C%22preLocalizedName%5C%5C%5C%22%3Anull%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22Trip%20Planner%5C%5C%5C%22%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FTrips%5C%5C%5C%22%2C%5C%5C%5C%22sublinks%5C%5C%5C%22%3A%5B%5D%7D%5D%7D%5D%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22250489043532424%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22currentLocale%5C%5C%5C%22%3A%7B%5C%5C%5C%22pointOfSaleName%5C%5C%5C%22%3A%5C%5C%5C%22United%20States%5C%5C%5C%22%2C%5C%5C%5C%22ietfLocale%5C%5C%5C%22%3A%5C%5C%5C%22en%5C%5C%5C%22%2C%5C%5C%5C%22languageName%5C%5C%5C%22%3A%5C%5C%5C%22English%5C%5C%5C%22%2C%5C%5C%5C%22country%5C%5C%5C%22%3A%7B%5C%5C%5C%22locationId%5C%5C%5C%22%3A191%2C%5C%5C%5C%22name%5C%5C%5C%22%3A%5C%5C%5C%22United%20States%5C%5C%5C%22%7D%7D%7D%5C%22%7D%2C%5C%22232167697255007%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22data%5C%5C%5C%22%3A%5B%5C%5C%5C%227149%5C%5C%5C%22%2Cnull%2C%5C%5C%5C%226977%5C%5C%5C%22%2C%5C%5C%5C%226940%5C%5C%5C%22%2C%5C%5C%5C%226892%5C%5C%5C%22%2C%5C%5C%5C%227133%5C%5C%5C%22%2C%5C%5C%5C%225108%5C%5C%5C%22%2Cnull%2C%5C%5C%5C%225388%5C%5C%5C%22%2C%5C%5C%5C%227342%5C%5C%5C%22%2C%5C%5C%5C%225386%5C%5C%5C%22%2C%5C%5C%5C%226879%5C%5C%5C%22%2C%5C%5C%5C%227192%5C%5C%5C%22%2C%5C%5C%5C%227195%5C%5C%5C%22%2C%5C%5C%5C%226970%5C%5C%5C%22%2C%5C%5C%5C%226877%5C%5C%5C%22%2C%5C%5C%5C%225071%5C%5C%5C%22%2Cnull%2C%5C%5C%5C%226893%5C%5C%5C%22%2C%5C%5C%5C%226875%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%22-15724433%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22menu_links_on__r_1718132076%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3Anull%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-5481293365%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22ai_travel_assistant_fab__web__1744635547%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-16161885676%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22AbTesting_evaluateTests%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22testResults%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22experimentKey%5C%5C%5C%22%3A%5C%5C%5C%22ollie__ai_assistant__dynamic_prompts__conversation_starters_1753882029%5C%5C%5C%22%2C%5C%5C%5C%22bucket%5C%5C%5C%22%3A%5C%5C%5C%22test%5C%5C%5C%22%7D%5D%7D%5D%7D%5C%22%7D%2C%5C%22-6191652010%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22Routing_canonicalize%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%5D%7D%5C%22%7D%2C%5C%22-7144532626%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22Best%20dining%20in%20Tulum%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%22-8501101610%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22No%20restaurants%20were%20found%20near%20Tulum.%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%226865839759%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22alertSection%5C%5C%5C%22%3A%7B%5C%5C%5C%22alerts%5C%5C%5C%22%3A%5B%5D%7D%7D%5C%22%7D%2C%5C%22-9100016233%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22text%5C%5C%5C%22%3A%5B%5C%5C%5C%22Tulum%20restaurants%5C%5C%5C%22%5D%7D%5C%22%7D%2C%5C%22-12694294617%5C%22%3A%7B%5C%22data%5C%22%3A%5C%22%7B%5C%5C%5C%22paginationRoutes%5C%5C%5C%22%3A%5B%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22params%5C%5C%5C%22%3A%7B%5C%5C%5C%22offset%5C%5C%5C%22%3A%5C%5C%5C%220%5C%5C%5C%22%2C%5C%5C%5C%22geoId%5C%5C%5C%22%3A150813%2C%5C%5C%5C%22diningOptionId%5C%5C%5C%22%3A10955%7D%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa0-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa0-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%2C%7B%5C%5C%5C%22page%5C%5C%5C%22%3A%5C%5C%5C%22Restaurants%5C%5C%5C%22%2C%5C%5C%5C%22params%5C%5C%5C%22%3A%7B%5C%5C%5C%22offset%5C%5C%5C%22%3A%5C%5C%5C%2230%5C%5C%5C%22%2C%5C%5C%5C%22geoId%5C%5C%5C%22%3A150813%2C%5C%5C%5C%22diningOptionId%5C%5C%5C%22%3A10955%7D%2C%5C%5C%5C%22url%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%2C%5C%5C%5C%22webLinkUrl%5C%5C%5C%22%3A%5C%5C%5C%22%2FRestaurants-g150813-oa30-zfp10955-Tulum_Yucatan_Peninsula.html%5C%5C%5C%22%7D%5D%7D%5C%22%7D%7D%7D%7D%22)))%3B"></script></body></html>
"""

soup_tripadvisor = BeautifulSoup(html_do_tripadvisor, 'html.parser')

print("\nIniciando la extracción detallada de los restaurantes de la página de Tripadvisor...")
restaurantes_detallados = [] 

restaurant_blocks = soup_tripadvisor.find_all('div', {'data-automation': 'restaurantCard'})

if not restaurant_blocks:
    print("AVISO: No se encontró ningún bloque principal de restaurantes con los selectores configurados ({'data-automation': 'restaurantCard'}).")
    print("Por favor, **REVISE LA INSPECCIÓN MANUAL** si el diseño de la página cambió y actualice este selector.")

top_n_restaurants = restaurant_blocks[:5] 

for block in top_n_restaurants:

    nombre_link_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'ukgoS' in c)
    nombre = "Nombre no encontrado"
    if nombre_link_tag:
        nombre_div_tag = nombre_link_tag.find('div', class_=lambda c: c and 'biGQs' in c and 'OgHoE' in c)
        if nombre_div_tag:
            nombre = nombre_div_tag.get_text(strip=True)

    reviews_tag = block.find('div', {'data-automation': 'bubbleReviewCount'})
    reviews = reviews_tag.find('span').get_text(strip=True) if reviews_tag and reviews_tag.find('span') else "Reseñas no encontradas"


    rating_tag = block.find('div', {'data-automation': 'bubbleRatingValue'})
    rating = rating_tag.find('span').get_text(strip=True) if rating_tag and rating_tag.find('span') else "Calificación no encontrada"


    culinaria_precio_div = block.find('div', class_=lambda c: c and 'ZvrsW' in c and 'biqBm' in c)

    tipo_culinaria = "Tipo de cocina no encontrado"
    precio = "Precio no encontrado"

    if culinaria_precio_div:

        spans_info = culinaria_precio_div.find_all(
            'span',
            class_=lambda c: c and 'biGQs' in c and 'ZNjnF' in c
        )


        if len(spans_info) >= 1:

            tipo_culinaria = spans_info[0].get_text(strip=True)


            for i in range(1, len(spans_info)):
                text = spans_info[i].get_text(strip=True)
                if '$' in text: 
                    precio = text
                    break 


    localizacion = "Ubicación no especificada (desde el nombre)"
    if ' - ' in nombre:

        partes_nombre = nombre.split(' - ')
        if len(partes_nombre) > 1:
            localizacion = partes_nombre[-1].strip()


    link = "Enlace no encontrado"
    if nombre_link_tag and 'href' in nombre_link_tag.attrs:
        link = "https://www.tripadvisor.com.mx" + nombre_link_tag['href']


    restaurantes_detallados.append({
        "Nombre": nombre,
        "Calificación": rating,
        "Reseñas": reviews,
        "Precio": precio,
        "Tipo de Cocina": tipo_culinaria,
        "Ubicación": localizacion,
        "Enlace": link
    })


if restaurantes_detallados:
    print(f"\n--- {len(restaurantes_detallados)} Restaurantes Extraídos de Tripadvisor ---")
    for i, r in enumerate(restaurantes_detallados):
        print(f"Restaurante #{i+1}:")
        print(f"  Nombre: {r['Nombre']}")
        print(f"  Calificación: {r['Calificación']}")
        print(f"  Reseñas: {r['Reseñas']}")
        print(f"  Precio: {r['Precio']}")
        print(f"  Tipo de Cocina: {r['Tipo de Cocina']}")
        print(f"  Ubicación: {r['Ubicación']}")
        print(f"  Enlace: {r['Enlace']}")
        print("-" * 40)
else:
    print("No se extrajo ningún detalle de restaurantes. **Verifique los selectores HTML en el Bloque 4**.")

print("-" * 50)



Iniciando la extracción detallada de los restaurantes de la página de Tripadvisor...

--- 5 Restaurantes Extraídos de Tripadvisor ---
Restaurante #1:
  Nombre: Alma Verde Tulum
  Calificación: 4.9
  Reseñas: (348reviews)
  Precio: $$ - $$$
  Tipo de Cocina: Mexican, Cafe
  Ubicación: Ubicación no especificada (desde el nombre)
  Enlace: https://www.tripadvisor.com.mx/Restaurant_Review-g150813-d24855625-Reviews-Alma_Verde_Tulum-Tulum_Yucatan_Peninsula.html
----------------------------------------
Restaurante #2:
  Nombre: La Popular - Nômade Temple Tulum
  Calificación: 4.8
  Reseñas: (412reviews)
  Precio: $$ - $$$
  Tipo de Cocina: Mexican, Seafood
  Ubicación: Nômade Temple Tulum
  Enlace: https://www.tripadvisor.com.mx/Restaurant_Review-g150813-d10066419-Reviews-La_Popular_Nomade_Temple_Tulum-Tulum_Yucatan_Peninsula.html
----------------------------------------
Restaurante #3:
  Nombre: 1. La Zebra Beach Restaurant And Bar
  Calificación: 4.8
  Reseñas: (6,446reviews)
  Precio: $$ 